In [3]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import json
import re
from tqdm import tqdm
from pyspark.sql.functions import broadcast
from datetime import datetime, timedelta
import os
os.chdir('/home/y0c07y1/rpc_feature_extraction/rpc_model/')
from utils.utils import *
import gcsfs

In [4]:
import pyspark
from pyspark.sql.functions import lit
from pyspark.sql import functions as func
import pyspark.sql as pss
from pyspark.sql.functions import col, split, when
from pyspark.sql import types as t
from pyspark.sql.types import (
    StringType,
    MapType,
    LongType,
    DoubleType,
    FloatType,
    StructField,
    StructType,
    IntegerType,
    ArrayType
)
#spark = pss.SparkSession.builder.appName("dctr_test3").config(conf=pyspark.SparkConf().setAll([('spark.yarn.queue', 'root.critical')])).enableHiveSupport().getOrCreate()
def gen_spark_session(ds):
    ds_nodash = ds.replace("-", "")

    return pyspark.sql.SparkSession.builder.config(
            "hive.exec.dynamic.partition", True,
        ).config(
            "hive.exec.dynamic.partition.mode", "nonstrict"
        ).config('mapreduce.input.fileinputformat.input.dir.recursive',True
        ).config('spark.hive.mapred.supports.subdirectories',True
        ).config("spark.sql.crossJoin.enabled", "true"
        ).config("spark.sql.broadcastTimeout", "36000"
        ).config("spark.sql.sources.partitionOverwriteMode","dynamic"
        ).config("spark.hadoop.orc.overwrite.output.file",True
        ).config("spark.yarn.executor.memoryOverhead", '20g'
        ).config("spark.executor.memory",'12g'
        ).config("spark.driver.memory",'12g'
        ).config("spark.dynamicAllocation.enabled",'true'
        ).config("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation",'true'
        ).config("spark.sql.hive.convertMetastoreOrc", 'false'
        ).appName(APP_NAME % ds_nodash).enableHiveSupport().getOrCreate()


APP_NAME = 'rpc_feature_extractionjupyter1_%s'
spark = gen_spark_session('2022-12/7')
spark.sparkContext.setLogLevel("ERROR")

# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10000000)

In [4]:
aa=spark.sql("select * from fair.pt2_14day_agg_gmv_nas where ad_date = '2023-02-15' and f_nas = 'oas'").toPandas()

In [9]:
aa['sourceid_revenue'].unique()

array([2, 6, 4])

In [10]:
aa2 = aa[aa['sourceid_revenue']==2]
aa4 = aa[aa['sourceid_revenue']==4]

In [11]:
len(aa2)

58417

In [12]:
len(aa4)

9256

In [13]:
len(aa)

72286

In [69]:
dctr_v2_hive_table = 'sem.daily_criteria_traffic_revenue_v2_tmp'
ds = '2023-02-15'
aa = spark.sql("""SELECT dr.adid as adid_spend,
                       coalesce(pcoc.catalog_item_id, dr.catalog_item_id) as catalog_item_id_spend,
                       coalesce(pcoc.us_seller_id, dr.seller_id) as seller_id_spend,
                       date_string_p as date_string_p_spend,
                       source_id as source_id_spend,
                       is_pla as is_pla_spend,
                       is_mobile as is_mobile_spend,
                       0 as is_target_all_spend,
                       sum(adspend) as adspend_dctr,
                       sum(clicks) as clicks_dctr,
                       sum(impressions) as impressions_dctr,
                       sum(orders) as orders_dctr,
                       sum(gmv_revenue_adjustment) as gmv_dctr
                            FROM
                        (SELECT *,split(adid,'_')[0] as catalog_item_id
                        FROM """ + dctr_v2_hive_table + """
                         WHERE customer_id_p = 1
                        AND structure_version_p = 2
                        AND date_string_p = '""" + ds + """'
                        AND revenue_source_p = 'fair15'
                        and source_id=4
                        and adid='0') dr
                        LEFT JOIN (SELECT DISTINCT adid, catalog_item_id, us_seller_id FROM 
                                       sem.pla_catalog_offer_criteria_shopping) pcoc
                        ON dr.adid = pcoc.adid
                    GROUP BY 1,2,3,4,5,6,7,8""").toPandas()

In [70]:
aa

,adid_spend,catalog_item_id_spend,seller_id_spend,date_string_p_spend,source_id_spend,is_pla_spend,is_mobile_spend,is_target_all_spend,adspend_dctr,clicks_dctr,impressions_dctr,orders_dctr,gmv_dctr


In [105]:
spark.sql("select * from sem.daily_criteria_traffic_revenue_v2_tmp").dtypes

[('customer_id', 'int'),
 ('date_string', 'string'),
 ('adid', 'string'),
 ('source_id', 'int'),
 ('keyword_external_id', 'string'),
 ('keyword', 'string'),
 ('account_external_id', 'string'),
 ('account_name', 'string'),
 ('campaign_external_id', 'string'),
 ('campaign_name', 'string'),
 ('group_external_id', 'string'),
 ('group_name', 'string'),
 ('is_pla', 'int'),
 ('sem_product_id', 'string'),
 ('is_mobile', 'int'),
 ('seller_id', 'string'),
 ('seller_name', 'string'),
 ('is_branded', 'int'),
 ('is_target_all', 'int'),
 ('match_type', 'string'),
 ('campaign_type', 'string'),
 ('channel_id', 'int'),
 ('clicks', 'int'),
 ('impressions', 'int'),
 ('ctr', 'double'),
 ('cpc', 'double'),
 ('max_cpc', 'double'),
 ('adspend', 'double'),
 ('position', 'double'),
 ('qty', 'int'),
 ('revenue', 'double'),
 ('orders', 'int'),
 ('avg_order_size', 'double'),
 ('cost', 'double'),
 ('operating_profit', 'double'),
 ('gmv_revenue_adjustment', 'double'),
 ('qty_sd', 'int'),
 ('revenue_sd', 'double'),


In [24]:
aa['seller_id_spend'].unique()

array([None], dtype=object)

In [27]:
bb = spark.sql("""select * from fair.pt3_14day_report_nas where source_id=4 and ad_date='2023-02-15'""").toPandas()

In [30]:
bb.columns

Index(['att_gmv_retrn_cancl_brand', 'att_gmv_retrn_cancl_coop',
       'att_gmv_retrn_cancl_item', 'attributed_gmv_brand',
       'attributed_gmv_coop', 'attributed_gmv_item', 'attributed_gmv_seller',
       'attributed_gmv_seller_item', 'attributed_gmv_seller_coop',
       'attributed_gmv_seller_brand', 'att_gmv_retrn_cancl_seller',
       'att_gmv_retrn_cancl_seller_item', 'att_gmv_retrn_cancl_seller_coop',
       'att_gmv_retrn_cancl_seller_brand', 'att_lineitems_item',
       'att_lineitems_brand', 'att_lineitems_coop',
       'att_lineitems_retrn_cancl_item', 'att_lineitems_retrn_cancl_brand',
       'att_lineitems_retrn_cancl_coop', 'att_lineitems_seller_item',
       'att_lineitems_seller_brand', 'att_lineitems_seller_coop',
       'att_lineitems_seller', 'att_lineitems_retrn_cancl_seller_item',
       'att_lineitems_retrn_cancl_seller_brand',
       'att_lineitems_retrn_cancl_seller_coop',
       'att_lineitems_retrn_cancl_seller', 'same_item_orders',
       'same_brand_orders'

In [118]:
ds = '2023-02-15'
cc = spark.sql("""SELECT *,split(adid,'_')[0] as catalog_item_id
                        FROM """ + dctr_v2_hive_table + """
                         WHERE customer_id_p = 1
                        AND structure_version_p = 1
                        AND date_string_p = '""" + ds + """'
                        AND revenue_source_p = 'fair15'
                        and source_id=2""").toPandas()
cc

,customer_id,date_string,adid,source_id,keyword_external_id,keyword,account_external_id,account_name,campaign_external_id,campaign_name,group_external_id,group_name,is_pla,sem_product_id,is_mobile,seller_id,seller_name,is_branded,is_target_all,match_type,campaign_type,channel_id,clicks,impressions,ctr,cpc,max_cpc,adspend,position,qty,revenue,orders,avg_order_size,cost,operating_profit,gmv_revenue_adjustment,qty_sd,revenue_sd,orders_sd,avg_order_size_sd,cost_sd,operating_profit_sd,gmv_revenue_adjustment_sd,qty_v1,revenue_v1,orders_v1,avg_order_size_v1,cost_v1,operating_profit_v1,gmv_revenue_adjustment_v1,qty_v1_sd,revenue_v1_sd,orders_v1_sd,avg_order_size_v1_sd,cost_v1_sd,operating_profit_v1_sd,gmv_revenue_adjustment_v1_sd,qty_v2,revenue_v2,orders_v2,avg_order_size_v2,cost_v2,operating_profit_v2,gmv_revenue_adjustment_v2,qty_v2_sd,revenue_v2_sd,orders_v2_sd,avg_order_size_v2_sd,cost_v2_sd,operating_profit_v2_sd,gmv_revenue_adjustment_v2_sd,qty_v3,revenue_v3,orders_v3,avg_order_size_v3,cost_v3,operating_profit_v3,gmv_revenue_adjustment_v3,qty_v3_sd,revenue_v3_sd,orders_v3_sd,avg_order_size_v3_sd,cost_v3_sd,operating_profit_v3_sd,gmv_revenue_adjustment_v3_sd,estimated_cp,estimated_cp_sd,estimated_cp_v1,estimated_cp_v1_sd,estimated_cp_v2,estimated_cp_v2_sd,estimated_cp_v3,estimated_cp_v3_sd,actual_cp,actual_cp_sd,actual_cp_v1,actual_cp_v1_sd,actual_cp_v2,actual_cp_v2_sd,actual_cp_v3,actual_cp_v3_sd,settled_simple_cp,settled_simple_cp_sd,settled_simple_cp_v1,settled_simple_cp_v1_sd,settled_simple_cp_v2,settled_simple_cp_v2_sd,settled_simple_cp_v3,settled_simple_cp_v3_sd,customer_id_p,date_string_p,adtype_p,revenue_source_p,structure_version_p,catalog_item_id
0,1,2023-02-15,456677372,2,617140639623,puppy milk walmart,1887786223,labs 65 - seasonal,19467026094,w2ml-fy23-edlpmega-sll-b-exact,143604499566,pets,0,None,0,None,None,1,0,exact,Other,20,1,1,1.0000,0.41,0.01,0.41,0.0,0,0.00,0,0.00,0.00,0.00,0.00,0,0.0,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.0,0.00,0,0.00,0,0.00,0.00,0.0,0.00,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2023-02-15,keyword,fair15,1,456677372
1,1,2023-02-15,456677531,2,382802982369,walmart grocery online delivery,1887786223,labs 65 - seasonal,19467026094,w2ml-fy23-edlpmega-sll-b-exact,143604499886,opd,0,None,0,None,None,1,0,exact,Other,20,1,1,1.0000,0.44,2.50,0.44,0.0,20,78.28,1,78.28,78.28,0.00,78.28,0,0.0,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.00,20,78.28,1,78.28,78.28,0.0,78.28,0,0.00,0,0.00,0.00,0.0,0.00,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2023-02-15,keyword,fair15,1,456677531
2,1,2023-02-15,456692196,2,329461993329,walmart diapers,1887786223,labs 65 - seasonal,19597863249,w2ml_fy24-smlb-lifestage-baby-brand,146610244018,wipes & diapers,0,None,0,None,None,1,0,phrase,Other,20,3,22,0.1364,0.67,0.50,2.00,0.0,-1,0.00,1,0.00,0.00,0.00,0.00,0,0.0,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.00,0,0.00,0,0.00,0.00,0.00,0.00,-1,0.00,1,0.00,0.00,0.0,0.00,0,0.00,0,0.00,0.00,0.0,0.00,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2023-02-15,keyword,fair15,1,456692196
3,1,2023-02-15,456693364,2,304512042880,walmart deals,1887786223,labs 65 - seasonal,19638416603,fy23-edlpmega-glm-save-b_exact,146818291578,xcat-deals_exact,0,None,0,None,None,1,0,exact,Other,20,2,61,0.0328,0.71,0.50,1.42,0.0,11,138.70,2,69.35,121.41,17.29,174.93,11,138.7,2,69.35,121.41,17.29,174.93,4,31.72,1,31.72,14.43,17.29,67.95,4,31.72,1,31.72,14.43,17.29,67.95,7,106.98,1,106.98,106.98,0.0,106.98,7,106.98,1,106.98,106.98,0.0,106.98,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [114]:
cc

,customer_id,date_string,adid,source_id,keyword_external_id,keyword,account_external_id,account_name,campaign_external_id,campaign_name,group_external_id,group_name,is_pla,sem_product_id,is_mobile,seller_id,seller_name,is_branded,is_target_all,match_type,campaign_type,channel_id,clicks,impressions,ctr,cpc,max_cpc,adspend,position,qty,revenue,orders,avg_order_size,cost,operating_profit,gmv_revenue_adjustment,qty_sd,revenue_sd,orders_sd,avg_order_size_sd,cost_sd,operating_profit_sd,gmv_revenue_adjustment_sd,qty_v1,revenue_v1,orders_v1,avg_order_size_v1,cost_v1,operating_profit_v1,gmv_revenue_adjustment_v1,qty_v1_sd,revenue_v1_sd,orders_v1_sd,avg_order_size_v1_sd,cost_v1_sd,operating_profit_v1_sd,gmv_revenue_adjustment_v1_sd,qty_v2,revenue_v2,orders_v2,avg_order_size_v2,cost_v2,operating_profit_v2,gmv_revenue_adjustment_v2,qty_v2_sd,revenue_v2_sd,orders_v2_sd,avg_order_size_v2_sd,cost_v2_sd,operating_profit_v2_sd,gmv_revenue_adjustment_v2_sd,qty_v3,revenue_v3,orders_v3,avg_order_size_v3,cost_v3,operating_profit_v3,gmv_revenue_adjustment_v3,qty_v3_sd,revenue_v3_sd,orders_v3_sd,avg_order_size_v3_sd,cost_v3_sd,operating_profit_v3_sd,gmv_revenue_adjustment_v3_sd,estimated_cp,estimated_cp_sd,estimated_cp_v1,estimated_cp_v1_sd,estimated_cp_v2,estimated_cp_v2_sd,estimated_cp_v3,estimated_cp_v3_sd,actual_cp,actual_cp_sd,actual_cp_v1,actual_cp_v1_sd,actual_cp_v2,actual_cp_v2_sd,actual_cp_v3,actual_cp_v3_sd,settled_simple_cp,settled_simple_cp_sd,settled_simple_cp_v1,settled_simple_cp_v1_sd,settled_simple_cp_v2,settled_simple_cp_v2_sd,settled_simple_cp_v3,settled_simple_cp_v3_sd,customer_id_p,date_string_p,adtype_p,revenue_source_p,structure_version_p,catalog_item_id
0,1,2023-02-15,440744748,4,4578435189284123,,138004297,bsc-138,361260236,health_care_32,1197368853763495,999955786,1,None,1,None,None,0,0,,,23,3,21,0.1429,0.18,0.33,0.55,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2023-02-15,pla,fair15,1,440744748
1,1,2023-02-15,440744748,4,4578435189284123,,138004297,bsc-138,361260236,health_care_32,1197368853763495,999955786,1,None,0,None,None,0,0,,,22,8,200,0.0400,0.27,0.33,2.17,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2023-02-15,pla,fair15,1,440744748


In [109]:
ww = bb[['att_gmv_retrn_cancl_brand','adid', 'catalog_item_id', 'brand_nm',
       'clicks_dctr', 'impressions_dctr', 'orders_dctr', 'gmv_dctr',
       'adspend_dctr', 'click_share', 'impression_share', 'source_id',
       'is_pla', 'is_target_all', 'seller_id', 'is_mobile', 'ad_date',
       'structure_version']]
ww[ww['catalog_item_id']=='999955786']

,att_gmv_retrn_cancl_brand,adid,catalog_item_id,brand_nm,clicks_dctr,impressions_dctr,orders_dctr,gmv_dctr,adspend_dctr,click_share,impression_share,source_id,is_pla,is_target_all,seller_id,is_mobile,ad_date,structure_version
94779,13.6600,NULL,999955786,timeframecameraaccessories,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4,1,0,101221028,0,2023-02-15,1
94780,0.0000,NULL,999955786,timeframecameraaccessories,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4,1,0,0,1,2023-02-15,1
94781,0.0000,NULL,999955786,timeframecameraaccessories,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4,1,1,0,0,2023-02-15,1
94782,0.0000,NULL,999955786,timeframecameraaccessories,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4,1,0,0,0,2023-02-15,1
94783,0.0000,NULL,999955786,timeframecameraaccessories,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4,1,1,0,1,2023-02-15,1


In [110]:
adid2cat = spark.sql("""SELECT
                            adid, max(catalog_item_id_ad_oms) as catalog_item_id_ad_oms_dotcom
                        FROM
                            ( SELECT CAST(id AS INT) as adid,
                                     catalog_item_id as catalog_item_id_ad_oms
                              FROM sem.wmt_dotcom_items 
                              )
                        GROUP BY adid
                        having max(catalog_item_id_ad_oms)='999955786'
                        order by 2
                        limit 100000""").toPandas()

In [111]:
adid2cat

,adid,catalog_item_id_ad_oms_dotcom
0,440744748,999955786


In [116]:
parameterized_spend_hive_table = 'sem.daily_kgl_traffic_revenue_parameterized'
sem_daily_traffic_parameterized = 'sem.daily_traffic_parameterized'
ds = '2023-02-15'
target_all_rev = spark.sql("""
                   SELECT report_data.item_id as catalog_item_id_target_all,
                       date_string as date_string_target_all,
                       source_id as source_id_target_all,
                       sum(report_data.revenue_gmv) as gmv_dctr_target_all,
                       sum(report_data.orders) as orders_target_all
                    FROM """+parameterized_spend_hive_table+"""
                    WHERE customer_id = 1
                       AND date_string = '"""+ ds +"""'
                       AND adid = '0'
                       and source_id=4
                    GROUP BY report_data.item_id, source_id, date_string""").toPandas()

In [117]:
target_all_rev

,catalog_item_id_target_all,date_string_target_all,source_id_target_all,gmv_dctr_target_all,orders_target_all


In [102]:
zz = spark.sql("""select * from sem.daily_kgl_traffic_revenue_parameterized where customer_id = 1
                       AND report_type = 'shopping' limit 3""").toPandas()

In [103]:
zz.columns

Index(['account_external_id', 'account_name', 'campaign_external_id',
       'campaign_name', 'group_external_id', 'group_name',
       'criteria_external_id', 'keyword', 'adid', 'is_mobile', 'is_pla',
       'report_data', 'customer_id', 'date_string', 'report_type',
       'source_id'],
      dtype='object')

In [104]:
zz['report_data'][0]

{'ctr': '0.00%',
 'adspend': '0.0',
 'conversions': '0.0',
 'revenue': '0',
 'cost': '1679',
 'item_id': '100000588',
 'average_cpc': '0',
 'clicks': '0.0',
 'orders': '0',
 'impressions': 'SPEED',
 'seller_id': '0.0',
 'revenue_gmv': '9'}

In [73]:
target_all_rev

,catalog_item_id_target_all,date_string_target_all,source_id_target_all,gmv_dctr_target_all,orders_target_all
0,177043893_101010156,2023-02-15,2,0.0,0.0
1,1770506812,2023-02-15,2,0.0,0.0
2,1770509263_18988,2023-02-15,2,0.0,0.0
3,1770621258,2023-02-15,2,0.0,0.0
4,1770683901,2023-02-15,2,0.0,0.0
...,...,...,...,...,...
11469628,176988253_5692,2023-02-15,2,0.0,0.0
11469629,1769949759_101183723,2023-02-15,2,0.0,0.0
11469630,177007678_2683,2023-02-15,2,0.0,0.0
11469631,1770080294_101027889,2023-02-15,2,0.0,0.0


In [274]:
sqlCmd = """SELECT report_data.item_id as catalog_item_id_spend,
                   date_string as date_string_spend,
                   report_data.seller_id as seller_id_spend,
                   is_mobile as is_mobile_spend,
                   source_id as source_id_spend,
                   is_pla as is_pla_spend,
                   sum(report_data.adspend) as adspend_dctr,
                   sum(report_data.clicks) as clicks_dctr,
                   sum(report_data.impressions) as impressions_dctr,
                   sum(report_data.orders) as orders_dctr,
                   sum(report_data.revenue_gmv) as gmv_dctr
                FROM """ + parameterized_spend_hive_table + """
                WHERE customer_id = 1
                   AND date_string = '""" + ds + """'
                   AND report_type = 'shopping'
                   AND coalesce(adid, '-1') != '0'
                   AND coalesce(campaign_external_id, '-1') != '202077872'
                GROUP BY report_data.item_id, report_data.seller_id, source_id, is_pla, is_mobile, date_string"""

spend = spark.sql(sqlCmd)

spend = spend.withColumn("is_target_all_spend", func.lit(0))

# get OAS non target all traffic and revenue for bing
sqlCmd = """SELECT catalog_item_id as catalog_item_id_spend,
                   date_string_p as date_spend,
                   null as seller_id_spend,
                   is_mobile as is_mobile_spend,
                   source_id as source_id_spend,
                   is_pla as is_pla_spend,
                   sum(adspend) as adspend_dctr,
                   sum(clicks) as clicks_dctr,
                   sum(impressions) as impressions_dctr,
                   sum(orders) as orders_dctr,
                   sum(gmv_revenue_adjustment) as gmv_dctr
                        FROM
                    (SELECT * FROM """+dctr_v2_hive_table+"""
                     WHERE customer_id_p = 1
                        AND structure_version_p = 1
                        AND revenue_source_p = 'fair15'
                        AND source_id = 4
                        AND adid != '0'
                        AND campaign_external_id != '202077872'
                        AND date_string_p = '""" + ds + """') dr
                    left outer join
                    (SELECT distinct id, catalog_item_id
                     FROM sem.wmt_dotcom_items) wdi
                     ON dr.adid = wdi.id
                GROUP BY catalog_item_id, seller_id, is_mobile, source_id, is_pla, date_string_p"""

spend1 = spark.sql(sqlCmd)

spend1 = spend1.withColumn("is_target_all_spend", func.lit(0))

# get OAS target all traffic
target_all_spend = spark.sql("""
                   SELECT report_data.item_id as catalog_item_id_target_all,
                       date_string as date_string_target_all,
                       source_id as source_id_target_all,
                       is_mobile as is_mobile_target_all,
                       sum(report_data.cost) as adspend_target_all,
                       sum(report_data.clicks) as clicks_target_all,
                       sum(report_data.impressions) as impressions_target_all,
                       if(max(report_data['mcid'])='8175035','0',mp.seller_id) as seller_id_target_all
                    FROM {sem_daily_traffic_parameterized} traffic
                    LEFT OUTER JOIN
                        (SELECT max(seller_id) seller_id, mca_account_id
                         FROM {sem_marketplace_partners}
                         GROUP BY mca_account_id) mp
                    ON (traffic.report_data['mcid'] = mp.mca_account_id)
                    WHERE customer_id = 1
                       AND date_string = '{ds}'
                       AND report_type = 'shopping'
                       AND campaign_external_id = '202077872'
                    GROUP BY report_data.item_id, source_id, date_string, seller_id, is_mobile""".format(
    sem_daily_traffic_parameterized=sem_daily_traffic_parameterized,
    ds=ds,
    sem_marketplace_partners=sem_marketplace_partners
))

target_all_spend = target_all_spend.withColumn("is_target_all_target_all", func.lit(1))
target_all_spend = target_all_spend.withColumn("is_pla_target_all", func.lit(1))

# union non target all traffic+revenue with target all traffic
spend = spend.join(target_all_spend, (
        (spend.date_string_spend == target_all_spend.date_string_target_all) &
        (spend.is_target_all_spend == target_all_spend.is_target_all_target_all) &
        (spend.catalog_item_id_spend == target_all_spend.catalog_item_id_target_all) &
        (spend.source_id_spend == target_all_spend.source_id_target_all) &
        (coalesce(spend.seller_id_spend, lit("no_seller")) == coalesce(target_all_spend.seller_id_target_all, lit("no_seller"))) &
        (spend.is_mobile_spend == target_all_spend.is_mobile_target_all) &
        (spend.is_pla_spend == target_all_spend.is_pla_target_all)),
                   how='outer')

spend = spend.withColumn("adspend_dctr", func.coalesce(col("adspend_dctr"),
                                                       col("adspend_target_all"))).drop('adspend_target_all')

spend = spend.withColumn("clicks_dctr", func.coalesce(col("clicks_dctr"),
                                                      col("clicks_target_all"))).drop('clicks_target_all')

spend = spend.withColumn("impressions_dctr", func.coalesce(col("impressions_dctr"),
                                                           col("impressions_target_all"))).drop(
    'impressions_target_all')

spend = spend.withColumn("is_target_all_spend", func.coalesce(col("is_target_all_spend"),
                                                              col("is_target_all_target_all"))).drop(
    'is_target_all_target_all')

spend = spend.withColumn("catalog_item_id_spend", func.coalesce(col("catalog_item_id_spend"),
                                                                col("catalog_item_id_target_all"))).drop(
    'catalog_item_id_target_all')

spend = spend.withColumn("source_id_spend", func.coalesce(col("source_id_spend"),
                                                          col("source_id_target_all"))).drop('source_id_target_all')

spend = spend.withColumn("is_pla_spend", func.coalesce(col("is_pla_spend"),
                                                       col("is_pla_target_all"))).drop('is_pla_target_all')

spend = spend.withColumn("date_spend", func.coalesce(col("date_string_spend"),
                                                     col("date_string_target_all"))).drop('date_string_target_all').drop('date_string_spend')

spend = spend.withColumn("seller_id_spend", func.coalesce(col("seller_id_spend"),
                                                          col("seller_id_target_all"))).drop('seller_id_target_all')

spend = spend.withColumn("is_mobile_spend", func.coalesce(col("is_mobile_spend"),
                                                          col("is_mobile_target_all"))).drop('is_mobile_target_all')

# append OAS target all revenue to (non target all traffic+revenue) + target all traffic
target_all_rev = spark.sql("""
                   SELECT report_data.item_id as catalog_item_id_target_all,
                       date_string as date_string_target_all,
                       source_id as source_id_target_all,
                       report_data.seller_id as seller_id_target_all,
                       is_mobile as is_mobile_target_all,
                       sum(report_data.revenue_gmv) as gmv_dctr_target_all,
                       sum(report_data.orders) as orders_target_all
                    FROM """ + parameterized_spend_hive_table + """
                    WHERE customer_id = 1
                       AND date_string = '""" + ds + """'
                       AND adid = '0'
                       AND report_type = 'shopping'
                    GROUP BY report_data.item_id, report_data.seller_id, source_id, date_string, is_mobile""")

target_all_rev = target_all_rev.withColumn("is_target_all_target_all", func.lit(1))
target_all_rev = target_all_rev.withColumn("is_pla_target_all", func.lit(1))

spend = spend.join(target_all_rev, (
        (spend.date_spend == target_all_rev.date_string_target_all) &
        (spend.is_target_all_spend == target_all_rev.is_target_all_target_all) &
        (spend.catalog_item_id_spend == target_all_rev.catalog_item_id_target_all) &
        (spend.source_id_spend == target_all_rev.source_id_target_all) &
        (coalesce(spend.seller_id_spend, lit("no_seller")) == coalesce(target_all_rev.seller_id_target_all, lit("no_seller"))) &
        (spend.is_mobile_spend == target_all_rev.is_mobile_target_all) &
        (spend.is_pla_spend == target_all_rev.is_pla_target_all)),
                   how='outer')


spend = spend.withColumn("orders_dctr", func.coalesce(col("orders_dctr"),
                                                      col("orders_target_all"))).drop(
    'orders_target_all')

spend = spend.withColumn("gmv_dctr", func.coalesce(col("gmv_dctr"),
                                                   col("gmv_dctr_target_all"))).drop(
    'gmv_dctr_target_all')

spend = spend.withColumn("is_target_all_spend", func.coalesce(col("is_target_all_spend"),
                                                              col("is_target_all_target_all"))).drop(
    'is_target_all_target_all')

spend = spend.withColumn("catalog_item_id_spend", func.coalesce(col("catalog_item_id_spend"),
                                                                col("catalog_item_id_target_all"))).drop(
    'catalog_item_id_target_all')

spend = spend.withColumn("source_id_spend", func.coalesce(col("source_id_spend"),
                                                          col("source_id_target_all"))).drop('source_id_target_all')

spend = spend.withColumn("is_pla_spend", func.coalesce(col("is_pla_spend"),
                                                       col("is_pla_target_all"))).drop('is_pla_target_all')

spend = spend.withColumn("date_spend", func.coalesce(col("date_spend"),
                                                     col("date_string_target_all"))).drop('date_string_target_all')

spend = spend.withColumn("seller_id_spend", func.coalesce(col("seller_id_spend"),
                                                          col("seller_id_target_all"))).drop('seller_id_target_all')

spend = spend.withColumn("is_mobile_spend", func.coalesce(col("is_mobile_spend"),
                                                          col("is_mobile_target_all"))).drop('is_mobile_target_all')

spend = spend.union(spend1.select(spend.columns))

# add in some competitive_metrics for OAS
competitive_metrics = spark.sql("""SELECT item_id as item_id_comp,
                                          date_string_p as date_string_p_comp,
                                          is_pla as is_pla_comp,
                                          is_mobile as is_mobile_comp,
                                          source_id_p as source_id_comp,
                                          mean(impression_share) as impression_share,
                                          mean(click_share) as click_share
                                   FROM """ + sem_daily_adid_rh_metrics_v2 + """
                                    WHERE customer_id_p=1
                                        AND date_string_p = '""" + ds + """'
                                        AND structure_version_p=1
                                    GROUP BY item_id, is_pla, source_id_p, date_string_p, is_mobile""")

is_target_all_vals = spark.sql('SELECT is_target_all as is_target_all_comp FROM fair_dev.TargetAllTypes')
competitive_metrics = competitive_metrics.crossJoin(is_target_all_vals)

spend = spend.join(competitive_metrics, (
        (spend.date_spend == competitive_metrics.date_string_p_comp) &
        (spend.catalog_item_id_spend == competitive_metrics.item_id_comp) &
        (spend.source_id_spend == competitive_metrics.source_id_comp) &
        (spend.is_target_all_spend == competitive_metrics.is_target_all_comp) &
        (spend.is_mobile_spend == competitive_metrics.is_mobile_comp) &
        (spend.is_pla_spend == competitive_metrics.is_pla_comp)),
                   how='left')

spend = spend.withColumn("catalog_item_id_spend", func.coalesce(col("catalog_item_id_spend"),
                                                                col("item_id_comp"))).drop("item_id_comp")

spend = spend.withColumn("source_id_spend", func.coalesce(col("source_id_spend"),
                                                          col("source_id_comp"))).drop("source_id_comp")

spend = spend.withColumn("is_pla_spend", func.coalesce(col("is_pla_spend"),
                                                       col("is_pla_comp"))).drop("is_pla_comp")

spend = spend.withColumn("date_spend", func.coalesce(col("date_spend"),
                                                     col("date_string_p_comp"))).drop("date_string_p_comp")

spend = spend.withColumn("is_target_all_spend", func.coalesce(col("is_target_all_spend"),
                                                              col("is_target_all_comp"))).drop("is_target_all_comp")

spend = spend.withColumn("is_mobile_spend", func.coalesce(col("is_mobile_spend"),
                                                          col("is_mobile_comp"))).drop("is_mobile_comp")
spend = spend.withColumn('catalog_item_id_spend', split('catalog_item_id_spend', '_')[0])

spend_cat = spend.groupBy('catalog_item_id_spend', 'source_id_spend', 'is_mobile_spend',
                          'is_pla_spend', 'date_spend', 'is_target_all_spend',
                          'seller_id_spend').agg(
    func.sum("adspend_dctr").alias('adspend_dctr'),
    func.mean("impression_share").alias('impression_share'),
    func.mean("click_share").alias('click_share'),
    func.sum("clicks_dctr").alias('clicks_dctr'),
    func.sum("impressions_dctr").alias('impressions_dctr'),
    func.sum("orders_dctr").alias('orders_dctr'),
    func.sum("gmv_dctr").alias('gmv_dctr')
)


# get 14D gmv on offer level for OAS
revenue_canclret_oas = spark.sql("""SELECT
                        sum(attributed_gmv_item) as attributed_gmv_item,
                        sum(attributed_gmv_brand) as attributed_gmv_brand,
                        sum(attributed_gmv_coop) as attributed_gmv_coop,
                        sum(att_gmv_retrn_cancl_item) as att_gmv_retrn_cancl_item,
                        sum(att_gmv_retrn_cancl_brand) as att_gmv_retrn_cancl_brand,
                        sum(att_gmv_retrn_cancl_coop) as att_gmv_retrn_cancl_coop,

                        sum(att_lineitems_item) as att_lineitems_item,
                        sum(att_lineitems_brand) as att_lineitems_brand,
                        sum(att_lineitems_coop) as att_lineitems_coop,
                        sum(att_lineitems_retrn_cancl_item) as att_lineitems_retrn_cancl_item,
                        sum(att_lineitems_retrn_cancl_brand) as att_lineitems_retrn_cancl_brand,
                        sum(att_lineitems_retrn_cancl_coop) as att_lineitems_retrn_cancl_coop,
                    sum(attributed_gmv_seller_item) as attributed_gmv_seller_item,
                    sum(attributed_gmv_seller_brand) as attributed_gmv_seller_brand,
                    sum(attributed_gmv_seller_coop) as attributed_gmv_seller_coop,
                    sum(attributed_gmv_seller) as attributed_gmv_seller,

                    sum(att_gmv_retrn_cancl_seller_item) as att_gmv_retrn_cancl_seller_item,
                    sum(att_gmv_retrn_cancl_seller_brand) as att_gmv_retrn_cancl_seller_brand,
                    sum(att_gmv_retrn_cancl_seller_coop) as att_gmv_retrn_cancl_seller_coop,
                    sum(att_gmv_retrn_cancl_seller) as att_gmv_retrn_cancl_seller,

                    sum(att_lineitems_seller_item) as att_lineitems_seller_item,
                    sum(att_lineitems_seller_brand) as att_lineitems_seller_brand,
                    sum(att_lineitems_seller_coop) as att_lineitems_seller_coop,
                    sum(att_lineitems_seller) as att_lineitems_seller,
                    sum(att_lineitems_retrn_cancl_seller_item) as att_lineitems_retrn_cancl_seller_item,
                    sum(att_lineitems_retrn_cancl_seller_brand) as att_lineitems_retrn_cancl_seller_brand,
                    sum(att_lineitems_retrn_cancl_seller_coop) as att_lineitems_retrn_cancl_seller_coop,
                    sum(att_lineitems_retrn_cancl_seller) as att_lineitems_retrn_cancl_seller,

                    sum(same_item_orders) as same_item_orders,
                    sum(same_brand_orders) as same_brand_orders,
                    sum(same_coop_orders) as same_coop_orders,
                    sum(same_seller_item_orders) as same_seller_item_orders,
                    sum(same_seller_brand_orders) as same_seller_brand_orders,
                    sum(same_seller_coop_orders) as same_seller_coop_orders,
                    sum(same_seller_orders) as same_seller_orders,

                    catalog_item_id_revenue,
                    sourceid_revenue,
                    is_pla_revenue,
                    is_target_all_revenue,
                    sellerid_ad,
                    is_mobile_revenue,
                    ad_date
                FROM  {brand_agg_stg_hive_table}
                      WHERE ad_date = '{ds}'
                      and f_nas = 'oas'
                GROUP BY catalog_item_id_revenue,
                         is_pla_revenue,
                         is_target_all_revenue,
                         sourceid_revenue,
                         ad_date,
                         sellerid_ad,
                         is_mobile_revenue
                         """.format(brand_agg_stg_hive_table=brand_agg_stg_hive_table,
                                    ds=ds))

spend_num_sellers = revenue_canclret_oas.groupBy("catalog_item_id_revenue", "is_pla_revenue", "is_target_all_revenue",
                                  "sourceid_revenue", "ad_date", "is_mobile_revenue").count()

revenue_canclret_oas = revenue_canclret_oas.join(spend_num_sellers, (
        (revenue_canclret_oas.catalog_item_id_revenue == spend_num_sellers.catalog_item_id_revenue) &
        (revenue_canclret_oas.is_pla_revenue == spend_num_sellers.is_pla_revenue) &
        (revenue_canclret_oas.is_target_all_revenue == spend_num_sellers.is_target_all_revenue) &
        (revenue_canclret_oas.sourceid_revenue == spend_num_sellers.sourceid_revenue) &
        (revenue_canclret_oas.ad_date == spend_num_sellers.ad_date) &
        (revenue_canclret_oas.is_mobile_revenue == spend_num_sellers.is_mobile_revenue)),
                   how='left').drop(spend_num_sellers.catalog_item_id_revenue).drop(spend_num_sellers.is_pla_revenue)\
        .drop(spend_num_sellers.is_target_all_revenue).drop(spend_num_sellers.sourceid_revenue)\
        .drop(spend_num_sellers.ad_date)\
        .drop(spend_num_sellers.is_mobile_revenue)

revenue_canclret_oas = revenue_canclret_oas.withColumnRenamed("count", "num_sellers")

# append OAS traffic+revenue to OAS 14D gmv table
item_agg_oas = spend_cat.join(revenue_canclret_oas, (
        (spend_cat.is_target_all_spend == revenue_canclret_oas.is_target_all_revenue) &
        (spend_cat.date_spend == revenue_canclret_oas.ad_date) &
        (spend_cat.catalog_item_id_spend == revenue_canclret_oas.catalog_item_id_revenue) &
        (spend_cat.source_id_spend == revenue_canclret_oas.sourceid_revenue) &
        ((coalesce(spend_cat.seller_id_spend, lit("0")) == coalesce(revenue_canclret_oas.sellerid_ad, lit("0"))) | (
                    spend_cat.source_id_spend == 4)) &
        (spend_cat.is_mobile_spend == revenue_canclret_oas.is_mobile_revenue) &
        (spend_cat.is_pla_spend == revenue_canclret_oas.is_pla_revenue)),
                              how='outer')

item_agg_oas = item_agg_oas.withColumn("catalog_item_id",
                                       func.coalesce(col("catalog_item_id_spend"), col("catalog_item_id_revenue")))
item_agg_oas = item_agg_oas.withColumn("source_id", func.coalesce(col("source_id_spend"), col("sourceid_revenue")))
item_agg_oas = item_agg_oas.withColumn("is_pla", func.coalesce(col("is_pla_spend"), col("is_pla_revenue")))
item_agg_oas = item_agg_oas.withColumn("ad_date", func.coalesce(col("ad_date"), col("date_spend")))
item_agg_oas = item_agg_oas.withColumn("is_target_all",
                                       func.coalesce(col("is_target_all_spend"), col("is_target_all_revenue")))
item_agg_oas = item_agg_oas.withColumn("seller_id", func.coalesce(col("seller_id_spend"), col("sellerid_ad")))
item_agg_oas = item_agg_oas.withColumn("is_mobile", func.coalesce(col("is_mobile_spend"), col("is_mobile_revenue")))

item_agg_oas = item_agg_oas.withColumn("adspend_dctr", when(item_agg_oas.source_id == 4,
                                                            item_agg_oas.adspend_dctr / item_agg_oas.num_sellers).otherwise(
    item_agg_oas.adspend_dctr))
item_agg_oas = item_agg_oas.withColumn("clicks_dctr", when(item_agg_oas.source_id == 4,
                                                           item_agg_oas.clicks_dctr / item_agg_oas.num_sellers).otherwise(
    item_agg_oas.clicks_dctr))
item_agg_oas = item_agg_oas.withColumn("impressions_dctr", when(item_agg_oas.source_id == 4,
                                                                item_agg_oas.impressions_dctr / item_agg_oas.num_sellers).otherwise(
    item_agg_oas.impressions_dctr))
item_agg_oas = item_agg_oas.withColumn("orders_dctr", when(item_agg_oas.source_id == 4,
                                                           item_agg_oas.orders_dctr / item_agg_oas.num_sellers).otherwise(
    item_agg_oas.orders_dctr))
item_agg_oas = item_agg_oas.withColumn("gmv_dctr", when(item_agg_oas.source_id == 4,
                                                        item_agg_oas.gmv_dctr / item_agg_oas.num_sellers).otherwise(
    item_agg_oas.gmv_dctr))

item_agg_oas = item_agg_oas.fillna(0.0, subset=['adspend_dctr', 'click_share', 'impression_share',
                                                'attributed_gmv_seller_item',
                                                'attributed_gmv_seller_brand',
                                                'attributed_gmv_seller_coop',
                                                'attributed_gmv_seller',
                                                'att_gmv_retrn_cancl_seller',
                                                'att_gmv_retrn_cancl_seller_item',
                                                'att_gmv_retrn_cancl_seller_coop',
                                                'att_gmv_retrn_cancl_seller_brand',
                                                'attributed_gmv_item',
                                                'attributed_gmv_brand',
                                                'attributed_gmv_coop',
                                                'att_gmv_retrn_cancl_item',
                                                'att_gmv_retrn_cancl_brand',
                                                'att_gmv_retrn_cancl_coop',
                                                'gmv_dctr', 'att_lineitems_item',
                                               'att_lineitems_brand', 'att_lineitems_coop',
                                               'att_lineitems_retrn_cancl_item', 'att_lineitems_retrn_cancl_brand',
                                               'att_lineitems_retrn_cancl_coop', 'att_lineitems_seller_item',
                                               'att_lineitems_seller_brand', 'att_lineitems_seller_coop',
                                               'att_lineitems_seller', 'att_lineitems_retrn_cancl_seller_item',
                                               'att_lineitems_retrn_cancl_seller_brand',
                                               'att_lineitems_retrn_cancl_seller_coop',
                                               'att_lineitems_retrn_cancl_seller', 'same_item_orders',
                                               'same_brand_orders', 'same_coop_orders', 'same_seller_item_orders',
                                               'same_seller_brand_orders', 'same_seller_coop_orders',
                                               'same_seller_orders'])

item_agg_oas = item_agg_oas.fillna(0, subset=['clicks_dctr', 'impressions_dctr',
                                              'orders_dctr']).drop('num_sellers')

# assign null string to OAS adid
item_agg_oas = item_agg_oas.withColumn('adid', lit('NULL'))
item_agg_oas = item_agg_oas.fillna('0', subset=['seller_id'])
item_agg_oas = item_agg_oas.withColumn('structure_version', lit(1))



# get non target all traffic+revenue for NAS on offer level from dctr_v2_tmp
# no need to get target all ads for NAS since NAS only has non target all offers
sqlCmd = """SELECT dr.adid as adid_spend,
                   coalesce(pcoc.catalog_item_id, dr.catalog_item_id) as catalog_item_id_spend,
                   coalesce(pcoc.us_seller_id, dr.seller_id) as seller_id_spend,
                   date_string_p as date_string_p_spend,
                   source_id as source_id_spend,
                   is_pla as is_pla_spend,
                   is_mobile as is_mobile_spend,
                   0 as is_target_all_spend,
                   sum(adspend) as adspend_dctr,
                   sum(clicks) as clicks_dctr,
                   sum(impressions) as impressions_dctr,
                   sum(orders) as orders_dctr,
                   sum(gmv_revenue_adjustment) as gmv_dctr
                        FROM
                    (SELECT *,split(adid,'_')[0] as catalog_item_id
                    FROM """ + dctr_v2_hive_table + """
                     WHERE customer_id_p = 1
                    AND structure_version_p = 2
                    AND date_string_p = '""" + ds + """'
                    AND revenue_source_p = 'fair15') dr
                    LEFT JOIN (SELECT DISTINCT adid, catalog_item_id, us_seller_id FROM 
                                   sem.pla_catalog_offer_criteria_shopping) pcoc
                    ON dr.adid = pcoc.adid
                GROUP BY 1,2,3,4,5,6,7,8"""

spend_nas = spark.sql(sqlCmd)

# add in some competitive_metrics for NAS
competitive_metrics = spark.sql("""SELECT darm.adid as adid_comp, 
                                        coalesce(catalog_item_id, item_id) as item_id_comp,
                                        coalesce(us_seller_id, seller_id) as seller_id_comp,
                                          date_string_p as date_string_p_comp,
                                          is_pla as is_pla_comp,
                                          is_mobile as is_mobile_comp,
                                          source_id_p as source_id_comp,
                                          0 as is_target_all_comp,
                                          mean(impression_share) as impression_share,
                                          mean(click_share) as click_share
                                   FROM """ + sem_daily_adid_rh_metrics_v2 + """ darm 
                                   LEFT JOIN (
                                   SELECT DISTINCT adid, catalog_item_id, us_seller_id FROM 
                                   sem.pla_catalog_offer_criteria_shopping) pcoc
                                   ON darm.adid = pcoc.adid
                                    WHERE darm.customer_id_p=1
                                    AND structure_version_p = 2
                                    AND revenue_source_p = 'fair15'
                                        AND date_string_p = '""" + ds + """'
                                    GROUP BY 1,2,3,4,5,6,7,8""")

spend_nas = spend_nas.join(competitive_metrics, (
        (spend_nas.adid_spend == competitive_metrics.adid_comp) &
        (spend_nas.date_string_p_spend == competitive_metrics.date_string_p_comp) &
        (spend_nas.catalog_item_id_spend == competitive_metrics.item_id_comp) &
        (coalesce(spend_nas.seller_id_spend, lit("no_seller")) == coalesce(competitive_metrics.seller_id_comp, lit("no_seller"))) &
        (spend_nas.source_id_spend == competitive_metrics.source_id_comp) &
        (spend_nas.is_target_all_spend == competitive_metrics.is_target_all_comp) &
        (spend_nas.is_mobile_spend == competitive_metrics.is_mobile_comp) &
        (spend_nas.is_pla_spend == competitive_metrics.is_pla_comp)),
                           how='outer')

spend_nas = spend_nas.withColumn("adid_spend", func.coalesce(col("adid_spend"),
                                                                        col("adid_comp")))

spend_nas = spend_nas.withColumn("catalog_item_id_spend", func.coalesce(col("catalog_item_id_spend"),
                                                                        col("item_id_comp")))

spend_nas = spend_nas.withColumn("source_id_spend", func.coalesce(col("source_id_spend"),
                                                                  col("source_id_comp")))

spend_nas = spend_nas.withColumn("is_pla_spend", func.coalesce(col("is_pla_spend"),
                                                               col("is_pla_comp")))

spend_nas = spend_nas.withColumn("date_spend", func.coalesce(col("date_string_p_spend"),
                                                             col("date_string_p_comp")))

spend_nas = spend_nas.withColumn("is_target_all_spend", func.coalesce(col("is_target_all_spend"),
                                                                      col("is_target_all_comp")))

spend_nas = spend_nas.withColumn("seller_id_spend", func.coalesce(col("seller_id_spend"),
                                                                  col("seller_id_comp")))

spend_nas = spend_nas.withColumn("is_mobile_spend", func.coalesce(col("is_mobile_spend"),
                                                                  col("is_mobile_comp")))

spend_nas_cat = spend_nas.groupBy('adid_spend', 'catalog_item_id_spend', 'source_id_spend',
                                  'is_pla_spend', 'date_spend', 'is_target_all_spend',
                                  'seller_id_spend', 'is_mobile_spend').agg(
    func.sum("adspend_dctr").alias('adspend_dctr'),
    func.mean("impression_share").alias('impression_share'),
    func.mean("click_share").alias('click_share'),
    func.sum("clicks_dctr").alias('clicks_dctr'),
    func.sum("impressions_dctr").alias('impressions_dctr'),
    func.sum("orders_dctr").alias('orders_dctr'),
    func.sum("gmv_dctr").alias('gmv_dctr')
)

# get 14D gmv on offer level for NAS
revenue_canclret_nas = spark.sql("""SELECT
                        sum(attributed_gmv_item) as attributed_gmv_item,
                        sum(attributed_gmv_brand) as attributed_gmv_brand,
                        sum(attributed_gmv_coop) as attributed_gmv_coop,
                        sum(att_gmv_retrn_cancl_item) as att_gmv_retrn_cancl_item,
                        sum(att_gmv_retrn_cancl_brand) as att_gmv_retrn_cancl_brand,
                        sum(att_gmv_retrn_cancl_coop) as att_gmv_retrn_cancl_coop,
                        sum(att_lineitems_item) as att_lineitems_item,
                        sum(att_lineitems_brand) as att_lineitems_brand,
                        sum(att_lineitems_coop) as att_lineitems_coop,
                        sum(att_lineitems_retrn_cancl_item) as att_lineitems_retrn_cancl_item,
                        sum(att_lineitems_retrn_cancl_brand) as att_lineitems_retrn_cancl_brand,
                        sum(att_lineitems_retrn_cancl_coop) as att_lineitems_retrn_cancl_coop,
                    sum(attributed_gmv_seller_item) as attributed_gmv_seller_item,
                    sum(attributed_gmv_seller_brand) as attributed_gmv_seller_brand,
                    sum(attributed_gmv_seller_coop) as attributed_gmv_seller_coop,
                    sum(attributed_gmv_seller) as attributed_gmv_seller,

                    sum(att_gmv_retrn_cancl_seller_item) as att_gmv_retrn_cancl_seller_item,
                    sum(att_gmv_retrn_cancl_seller_brand) as att_gmv_retrn_cancl_seller_brand,
                    sum(att_gmv_retrn_cancl_seller_coop) as att_gmv_retrn_cancl_seller_coop,
                    sum(att_gmv_retrn_cancl_seller) as att_gmv_retrn_cancl_seller,

                    sum(att_lineitems_seller_item) as att_lineitems_seller_item,
                    sum(att_lineitems_seller_brand) as att_lineitems_seller_brand,
                    sum(att_lineitems_seller_coop) as att_lineitems_seller_coop,
                    sum(att_lineitems_seller) as att_lineitems_seller,
                    sum(att_lineitems_retrn_cancl_seller_item) as att_lineitems_retrn_cancl_seller_item,
                    sum(att_lineitems_retrn_cancl_seller_brand) as att_lineitems_retrn_cancl_seller_brand,
                    sum(att_lineitems_retrn_cancl_seller_coop) as att_lineitems_retrn_cancl_seller_coop,
                    sum(att_lineitems_retrn_cancl_seller) as att_lineitems_retrn_cancl_seller,

                    sum(same_item_orders) as same_item_orders,
                    sum(same_brand_orders) as same_brand_orders,
                    sum(same_coop_orders) as same_coop_orders,
                    sum(same_seller_item_orders) as same_seller_item_orders,
                    sum(same_seller_brand_orders) as same_seller_brand_orders,
                    sum(same_seller_coop_orders) as same_seller_coop_orders,
                    sum(same_seller_orders) as same_seller_orders,

                    adid, 
                    catalog_item_id_revenue,
                    sourceid_revenue,
                    is_pla_revenue,
                    is_target_all_revenue,
                    sellerid_ad,
                    is_mobile_revenue,
                    ad_date
                FROM  {brand_agg_stg_hive_table}
                      WHERE ad_date = '{ds}'
                      and f_nas = 'nas'
                GROUP BY adid, 
                         catalog_item_id_revenue,
                         is_pla_revenue,
                         is_target_all_revenue,
                         sourceid_revenue,
                         ad_date,
                         sellerid_ad,
                         is_mobile_revenue
                         """.format(brand_agg_stg_hive_table=brand_agg_stg_hive_table,
                                    ds=ds))

# append NAS traffic+revenue to NAS 14D gmv table
item_agg_nas = spend_nas_cat.join(revenue_canclret_nas, (
        (spend_nas_cat.is_target_all_spend == revenue_canclret_nas.is_target_all_revenue) &
        (spend_nas_cat.date_spend == revenue_canclret_nas.ad_date) &
        (spend_nas_cat.catalog_item_id_spend == revenue_canclret_nas.catalog_item_id_revenue) &
        (spend_nas_cat.adid_spend == revenue_canclret_nas.adid) &
        (spend_nas_cat.source_id_spend == revenue_canclret_nas.sourceid_revenue) &
        (coalesce(spend_nas_cat.seller_id_spend, lit("0")) == coalesce(revenue_canclret_nas.sellerid_ad, lit("0"))) &
        (spend_nas_cat.is_mobile_spend == revenue_canclret_nas.is_mobile_revenue) &
        (spend_nas_cat.is_pla_spend == revenue_canclret_nas.is_pla_revenue)),
                                  how='outer')

item_agg_nas = item_agg_nas.withColumn("catalog_item_id",
                                       func.coalesce(col("catalog_item_id_spend"), col("catalog_item_id_revenue")))
item_agg_nas = item_agg_nas.withColumn("source_id", func.coalesce(col("source_id_spend"), col("sourceid_revenue")))
item_agg_nas = item_agg_nas.withColumn("is_pla", func.coalesce(col("is_pla_spend"), col("is_pla_revenue")))
item_agg_nas = item_agg_nas.withColumn("ad_date", func.coalesce(col("ad_date"), col("date_spend")))
item_agg_nas = item_agg_nas.withColumn("is_target_all",
                                       func.coalesce(col("is_target_all_spend"), col("is_target_all_revenue")))
item_agg_nas = item_agg_nas.withColumn("seller_id", func.coalesce(col("seller_id_spend"), col("sellerid_ad")))
item_agg_nas = item_agg_nas.withColumn("is_mobile", func.coalesce(col("is_mobile_spend"), col("is_mobile_revenue")))
item_agg_nas = item_agg_nas.withColumn("adid", func.coalesce(col("adid_spend"), col("adid")))

item_agg_nas = item_agg_nas.fillna(0.0, subset=['adspend_dctr', 'click_share', 'impression_share',
                                                'attributed_gmv_seller_item',
                                                'attributed_gmv_seller_brand',
                                                'attributed_gmv_seller_coop',
                                                'attributed_gmv_seller',
                                                'att_gmv_retrn_cancl_seller',
                                                'att_gmv_retrn_cancl_seller_item',
                                                'att_gmv_retrn_cancl_seller_coop',
                                                'att_gmv_retrn_cancl_seller_brand',
                                                'attributed_gmv_item',
                                                'attributed_gmv_brand',
                                                'attributed_gmv_coop',
                                                'att_gmv_retrn_cancl_item',
                                                'att_gmv_retrn_cancl_brand',
                                                'att_gmv_retrn_cancl_coop',
                                                'gmv_dctr', 'att_lineitems_item',
                                               'att_lineitems_brand', 'att_lineitems_coop',
                                               'att_lineitems_retrn_cancl_item', 'att_lineitems_retrn_cancl_brand',
                                               'att_lineitems_retrn_cancl_coop', 'att_lineitems_seller_item',
                                               'att_lineitems_seller_brand', 'att_lineitems_seller_coop',
                                               'att_lineitems_seller', 'att_lineitems_retrn_cancl_seller_item',
                                               'att_lineitems_retrn_cancl_seller_brand',
                                               'att_lineitems_retrn_cancl_seller_coop',
                                               'att_lineitems_retrn_cancl_seller', 'same_item_orders',
                                               'same_brand_orders', 'same_coop_orders', 'same_seller_item_orders',
                                               'same_seller_brand_orders', 'same_seller_coop_orders',
                                               'same_seller_orders'])

item_agg_nas = item_agg_nas.fillna(0, subset=['clicks_dctr', 'impressions_dctr',
                                              'orders_dctr'])

item_agg_nas = item_agg_nas.withColumn('structure_version', lit(2))
item_agg_nas = item_agg_nas.fillna(0, subset=['seller_id'])
item_agg_nas = item_agg_nas.select(item_agg_oas.columns)
item_agg = item_agg_nas.union(item_agg_oas)

# get brand for each item
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x: x).map(
    lambda x: x.replace("ts=", "")).max()

item_info_dfm = spark.sql("""SELECT item_id as catlg_item_id,
                                    max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as brand_nm
            FROM dmas.dfm
            WHERE ts='""" + partitionDate + """'
            GROUP BY item_id""")

item_agg = item_agg.join(item_info_dfm,
                         (item_agg.catalog_item_id == item_info_dfm.catlg_item_id),
                         how='left').drop(item_info_dfm.catlg_item_id)

# 10. Save to aggregated table
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
columns = ['att_gmv_retrn_cancl_brand',
           'att_gmv_retrn_cancl_coop',
           'att_gmv_retrn_cancl_item',
           'attributed_gmv_brand',
           'attributed_gmv_coop',
           'attributed_gmv_item',
           'attributed_gmv_seller',
           'attributed_gmv_seller_item',
           'attributed_gmv_seller_coop',
           'attributed_gmv_seller_brand',
           'att_gmv_retrn_cancl_seller',
           'att_gmv_retrn_cancl_seller_item',
           'att_gmv_retrn_cancl_seller_coop',
           'att_gmv_retrn_cancl_seller_brand',

           'att_lineitems_item',
           'att_lineitems_brand',
           'att_lineitems_coop',
           'att_lineitems_retrn_cancl_item',
           'att_lineitems_retrn_cancl_brand',
           'att_lineitems_retrn_cancl_coop',
           'att_lineitems_seller_item',
           'att_lineitems_seller_brand',
           'att_lineitems_seller_coop',
           'att_lineitems_seller',
           'att_lineitems_retrn_cancl_seller_item',
           'att_lineitems_retrn_cancl_seller_brand',
           'att_lineitems_retrn_cancl_seller_coop',
           'att_lineitems_retrn_cancl_seller',

           'same_item_orders', 'same_brand_orders',
           'same_coop_orders', 'same_seller_item_orders',
           'same_seller_brand_orders', 'same_seller_coop_orders',
           'same_seller_orders',

           'adid', 'catalog_item_id', 'brand_nm',
           'clicks_dctr', 'impressions_dctr', 'orders_dctr', 'gmv_dctr', 'adspend_dctr',
           'click_share', 'impression_share',
           'source_id', 'is_pla', 'is_target_all', 'seller_id', 'is_mobile',
           'ad_date', 'structure_version']
item_agg = item_agg.select(columns)

In [273]:
ds = '2023-03-20'

In [204]:
aa = spend100.toPandas()

In [206]:
aa['num_sellers'].unique()

array([1, 2, 3])

In [209]:
aa[aa['num_sellers']==3]

,catalog_item_id_spend,source_id_spend,is_mobile_spend,is_pla_spend,date_spend,is_target_all_spend,seller_id_spend,adspend_dctr,impression_share,click_share,clicks_dctr,impressions_dctr,orders_dctr,gmv_dctr,num_sellers
11,1070752164,2,1,1,2023-02-15,1,1259,0.0,None,None,0.0,1.0,0.0,0.0,3
26,11975404,2,0,1,2023-02-15,0,15303,0.0,None,None,0.0,2.0,0.0,0.0,3


In [ ]:

    spend_num_sellers = spend.groupBy("catalog_item_id_spend", "source_id_spend", "is_pla_spend",
                                      "date_spend", "is_mobile_spend", "is_target_all_spend").count()

    spend = spend.join(spend_num_sellers, (
            (spend.date_spend == spend_num_sellers.date_spend) &
            (spend.catalog_item_id_spend == spend_num_sellers.catalog_item_id_spend) &
            (spend.source_id_spend == spend_num_sellers.source_id_spend) &
            (spend.is_target_all_spend == spend_num_sellers.is_target_all_spend) &
            (spend.is_mobile_spend == spend_num_sellers.is_mobile_spend) &
            (spend.is_pla_spend == spend_num_sellers.is_pla_spend)),
                       how='left').drop(spend_num_sellers.date_spend).drop(spend_num_sellers.catalog_item_id_spend)\
            .drop(spend_num_sellers.source_id_spend).drop(spend_num_sellers.is_target_all_spend)\
            .drop(spend_num_sellers.is_mobile_spend)\
            .drop(spend_num_sellers.is_pla_spend)
    spend = spend.withColumnRenamed("count", "num_sellers")

    spend1 = spend1.withColumnRenamed("date_string_p_spend", "date_spend")

In [63]:
v1 = spark.sql("""select catalog_item_id, source_id, sum(attributed_gmv_brand) from 
(select fair.source_id,fair.catalog_item_id, attributed_gmv_brand from fair.pt3_14day_report_brand fair
inner join
casesci_sem.coop_keyword_item_info coop
on
fair.catalog_item_id = coop.catalog_item_id and 
fair.source_id = coop.source_id and
fair.ad_date = coop.date_string and
fair.ad_date = "2023-03-26") a
group by source_id,catalog_item_id""").toPandas()

In [64]:
v1.columns = ['catalog_item_id','source_id', 'attributed_gmv_brand_v1']

In [65]:
v2 = spark.sql("""select catalog_item_id,source_id, sum(attributed_gmv_brand) from 
(select fair.source_id, fair.catalog_item_id,attributed_gmv_brand from fair.pt3_14day_report_nas fair
inner join
casesci_sem.coop_keyword_item_info coop
on
fair.catalog_item_id = coop.catalog_item_id and 
fair.source_id = coop.source_id and
fair.ad_date = coop.date_string and
fair.ad_date = "2023-03-26") a
group by source_id,catalog_item_id""").toPandas()

In [66]:
v2.columns = ['catalog_item_id','source_id', 'attributed_gmv_brand_v2']

In [67]:
v_join = pd.merge(v1,v2, how='outer', on = ['catalog_item_id','source_id'])

In [68]:
v_join[(v_join['attributed_gmv_brand_v2']!=v_join['attributed_gmv_brand_v1'])&(v_join['source_id']==4)]

,catalog_item_id,source_id,attributed_gmv_brand_v1,attributed_gmv_brand_v2
961,997083873,4,170.8400,42.7100


In [69]:
spark.sql("""select source_id, sum(attributed_gmv_brand) from 
(select fair.source_id,attributed_gmv_brand from fair.pt3_14day_report_brand fair
inner join
casesci_sem.coop_keyword_item_info coop
on
fair.catalog_item_id = coop.catalog_item_id and 
fair.source_id = coop.source_id and
fair.ad_date = coop.date_string and
fair.ad_date = "2023-03-26") a
group by source_id""").toPandas()

,source_id,sum(attributed_gmv_brand)
0,4,9800.4233
1,2,335577.4199


In [71]:
spark.sql("""select source_id, sum(attributed_gmv_brand) from 
(select fair.source_id,attributed_gmv_brand from fair.pt3_14day_report_nas fair
inner join
casesci_sem.coop_keyword_item_info coop
on
fair.catalog_item_id = coop.catalog_item_id and 
fair.source_id = coop.source_id and
fair.ad_date = coop.date_string and
fair.ad_date = "2023-03-26") a
group by source_id""").toPandas()

,source_id,sum(attributed_gmv_brand)
0,4,9672.2933
1,2,643933.5148


In [96]:
dctr_hive_table = 'sem.daily_criteria_traffic_revenue'
sem_wmt_dotcom_items='sem.wmt_dotcom_items'
ds = '2023-03-20'
dctr_v1 = spark.sql("""SELECT catalog_item_id as catalog_item_id_spend,
                        is_mobile,
                       date_string_p as date_string_p_spend,
                       source_id as source_id_spend,
                       is_pla as is_pla_spend,
                       sum(adspend) as adspend_dctr,
                       sum(clicks) as clicks_dctr,
                       sum(impressions) as impressions_dctr,
                       sum(orders) as orders_dctr,
                               sum(gmv_revenue_adjustment) as gmv_dctr
                            FROM
                        (SELECT * FROM """+dctr_hive_table+"""
                         WHERE customer_id_p = 1
                            AND adid != '0'
                            AND campaign_external_id != '202077872'
                            AND date_string_p = '""" + ds + """') dr
                        inner join
                        (SELECT distinct id, catalog_item_id
                         FROM """+sem_wmt_dotcom_items+"""
                         where catalog_item_id = '587182688') wdi
                         ON dr.adid = wdi.id
                    GROUP BY catalog_item_id, source_id, is_pla, date_string_p,is_mobile""").toPandas()
dctr_v1

,catalog_item_id_spend,is_mobile,date_string_p_spend,source_id_spend,is_pla_spend,adspend_dctr,clicks_dctr,impressions_dctr,orders_dctr,gmv_dctr
0,587182688,1,2023-03-20,2,1,1742.77,2269,168112,53,6084.99
1,587182688,0,2023-03-20,2,1,218.53,244,26930,6,1190.06
2,587182688,1,2023-03-20,4,1,1.30,13,381,0,0.00
3,587182688,0,2023-03-20,4,1,15.14,61,5886,3,570.00


In [97]:
parameterized_spend_hive_table = 'sem.daily_kgl_traffic_revenue_parameterized'
dctr_v2 = spark.sql("""SELECT report_data.item_id as catalog_item_id_spend,
                   date_string as date_string_spend,
                   report_data.seller_id as seller_id_spend,
                   is_mobile as is_mobile_spend,
                   source_id as source_id_spend,
                   is_pla as is_pla_spend,
                   sum(report_data.adspend) as adspend_dctr,
                   sum(report_data.clicks) as clicks_dctr,
                   sum(report_data.impressions) as impressions_dctr,
                   sum(report_data.orders) as orders_dctr,
                   sum(report_data.revenue_gmv) as gmv_dctr
                FROM """ + parameterized_spend_hive_table + """
                WHERE customer_id = 1
                   AND date_string = '""" + ds + """'
                   and report_data.item_id = '587182688'
                   AND report_type = 'shopping'
                   AND coalesce(adid, '-1') != '0'
                   AND coalesce(campaign_external_id, '-1') != '202077872'
                GROUP BY report_data.item_id, report_data.seller_id, source_id, is_pla, is_mobile, date_string""").toPandas()
dctr_v2

,catalog_item_id_spend,date_string_spend,seller_id_spend,is_mobile_spend,source_id_spend,is_pla_spend,adspend_dctr,clicks_dctr,impressions_dctr,orders_dctr,gmv_dctr
0,587182688,2023-03-20,0,0,2,1,218.529999,244.0,26930.0,4.0,1190.059998
1,587182688,2023-03-20,0,1,2,1,1742.770020,2269.0,168112.0,38.0,7138.549997


In [ ]:
catalog_item_id	source_id	attributed_gmv_brand_v1	attributed_gmv_brand_v2
132195807	4	0.0000	49.9900
587182688	4	536.0000	914.0000
963321253	4	0.0000	22.7500
437543077	4	0.0000	129.0000
50753947	4	0.0000	73.0000
523117746	4	349.0000	813.0000
34621243	4	0.0000	22.4250
408186523	4	0.0000	26.9600
903393993	4	0.0000	35.9600
454609278	4	0.0000	74.0000
913748766	4	177.0000	354.0000
46368155	4	0.0000	326.9600
128173335	4	0.0000	29.9400
269430405	4	0.0000	346.4775
160692525	4	0.0000	297.0000
314022535	4	0.0000	354.0000
782010436	4	0.0000	179.9900
682511182	4	0.0000	22.9600
50753941	4	328.0000	437.0000
483978365	4	269.0000	538.0000
291404933	4	0.0000	349.0000
200485831	4	43.6500	14.5500

In [107]:
pt1_v1 = spark.sql("""select *
from fair.pt1_14day_ads_lineitems
where order_date = '2023-03-26' and catalog_item_id_ad='997083873' and ad_date = '2023-03-26'""").toPandas()
pt1_v1[(pt1_v1['purchase_brand']==pt1_v1['ad_item_brand'])&(pt1_v1['source_id']==4)]

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_nm,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,order_date
1,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,997083873,997083873,UNSCHEDULED_DELIVERY,1,deltachildren,deltachildren,2474,2474,0,0,1,200010802154653,170.840000000,170.840000000,4,22,1,0,1,1,1,1,1,1,2,2023-03-26


In [108]:
pt1_v2 = spark.sql("""select *
from fair.pt1_14day_ads_lineitems_nas
where order_date = '2023-03-26' and catalog_item_id_ad='997083873' and ad_date = '2023-03-26'""").toPandas()
pt1_v2[(pt1_v2['purchase_brand']==pt1_v2['ad_item_brand'])&(pt1_v2['f_nas']=='oas')&(pt1_v2['source_id']==4)]

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_nm,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,is_mobile,f_nas,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,same_item,same_brand,same_coop,same_seller,same_seller_item,same_seller_brand,same_seller_coop,order_date
2,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,997083873,997083873,UNSCHEDULED_DELIVERY,1,deltachildren,deltachildren,2474,2474,0,0,1,200010802154653,170.840000000,170.840000000,4,22,1,0,0,oas,1,4,1,1,4,1,5,1,1,1,1,1,1,1,2023-03-26


In [76]:
fair_daily_auth_order_to_ref_vehicle = 'fair.daily_auth_order_to_ref_vehicle'
sem_customers = 'sem.customers'
ds = '2023-03-26'
fair_daily_auth_order_to_ref_vehicle = 'fair.daily_auth_order_to_ref_vehicle'
fair_daily_auth_order_to_visit_chain = 'fair.daily_auth_order_to_visit_chain'

In [103]:
# v1 oms
spark.sql("""SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id,
           if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
        WHERE ds = '{ds}'
        and order_no = '200010802154653'""".format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                    ds=ds,
                                    sem_customers=sem_customers,
                                    extract_adid_lia = r'substring(reflector_id, channel_id_position+2)',
                                    extract_adid_pla = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)'
                                   )).toPandas()

,order_no_oms,order_date_oms,reflector_id,adid_oms,channel_id_oms,ad_date_oms
0,200010802154653,2023-03-26,imp_R80SzAzSZxyNUjNVVhy4-XQEUkAWDbVNUwmuxU0,SZxyNUjNVVhy4-XQEUkAWDbVNUwmuxU0,None,2023-03-26


In [104]:
# v2 oms
spark.sql("""select order_no_oms, order_date_oms, channel_id_oms, ad_date_oms, f_nas_oms, adid_oms,reflector_id,
        if(reflector_id RLIKE '{regex_lia_nas}' or reflector_id RLIKE '{regex_pla_nas}', split(adid_oms,'_')[0], NULL) as catalog_item_id_ad_oms,
        if(reflector_id RLIKE '{regex_pla_nas}', split(adid_oms,'_')[1], '0') as sellerid_ad_oms
        FROM (
            SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id, 
            if(reflector_id RLIKE '{regex_lia_nas}' or reflector_id RLIKE '{regex_pla_nas}', 'nas', 'oas') as f_nas_oms,
            if(reflector_id RLIKE '{regex_pla_nas}', {extract_adid_nas}, if(reflector_id RLIKE '{regex_lia_nas}', {extract_adid_nas}, if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}))) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
    WHERE ds = '{ds}')
    where order_no_oms = '200010802154653'""".format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                 ds=ds,
                                 sem_customers=sem_customers,
                                 regex_lia_nas=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                 regex_pla_nas=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+_[0-9]+$',
                                 extract_adid_lia=r'substring(reflector_id, channel_id_position+2)',
                                 extract_adid_pla=r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                 extract_adid_nas=r'substring(reflector_id, channel_id_position+2)',
                                 )).toPandas()

,order_no_oms,order_date_oms,channel_id_oms,ad_date_oms,f_nas_oms,adid_oms,reflector_id,catalog_item_id_ad_oms,sellerid_ad_oms
0,200010802154653,2023-03-26,None,2023-03-26,oas,SZxyNUjNVVhy4-XQEUkAWDbVNUwmuxU0,imp_R80SzAzSZxyNUjNVVhy4-XQEUkAWDbVNUwmuxU0,None,0


In [105]:
# v1 omni
spark.sql("""SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,adid,
       if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}) AS adid_omni,
       if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}) AS channel_id_omni,
       coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '') sellerid_ad,
       split(coalesce(parse_url(page_url, 'QUERY','wl12'), ''),'_')[0] catalog_item_id_ad_omni
       FROM {fair_daily_auth_order_to_visit_chain},
       (
                SELECT id customer_id,
                       reflector_identifier,
                       channel_id_position,
                       adid_length
                FROM {sem_customers} c where id=1
        ) c
       WHERE visit_vehicle='labs sem'
           AND length(adid)-length(replace(adid,'_',''))=0
           and order_no = '200010802154653'
           AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2
           AND ds = '{ds}'""".format(ds = ds,
                                      fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                      sem_customers=sem_customers,
                                      regex_lia = r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                      extract_adid_lia = r'substring(adid, channel_id_position+2)',
                                      extract_adid_pla = r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                      extract_adid_reflector = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                      channel_id_plalia = r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                      channel_id_reflector = r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                     )).toPandas()


,order_no_omni,page_url,visit_hit_time_gmt,order_date_omni,adid,adid_omni,channel_id_omni,sellerid_ad,catalog_item_id_ad_omni
0,200010802154653,https://www.walmart.com/ip/Round-Coffee-Table-...,1679372994,2023-03-26,22222222222524753293,524753293,22,101103537,256151387
1,200010802154653,https://www.walmart.com/ip/KOHLER-CB-CLR1620FS...,1675634270,2023-03-26,22222222227000000000,0,27,101219667,1857740207
2,200010802154653,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,22222222227000000000,0,27,0,854547111
3,200010802154653,https://www.walmart.com/ip/Truly-Soft-Everyday...,1675749648,2023-03-26,22222222227000000000,0,27,0,482573538
4,200010802154653,https://www.walmart.com/ip/SAFAVIEH-Glamour-Pi...,1675210059,2023-03-26,22222222222345061631,345061631,22,17374,898106547
5,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,22222222222404313442,404313442,22,0,997083873
6,200010802154653,https://www.walmart.com/ip/Unisex-Baby-Fleece-...,1679631393,2023-03-26,22222222227,None,None,101093689,393156732


In [106]:
# v2 omni
spark.sql("""SELECT order_no_omni,page_url,visit_hit_time_gmt, order_date_omni, adid_omni, channel_id_omni,adid,
if(adid RLIKE '{regex_pla}', split(adid_omni,'_')[0], split(catalog_item_id_ad_omni_old,'_')[0]) as catalog_item_id_ad_omni,
coalesce(if(adid RLIKE '{regex_pla}', split(adid_omni,'_')[1], sellerid_ad_old), '0') as sellerid_ad_omni,
if(adid RLIKE '{regex_lia}' or adid RLIKE '{regex_pla}', 'nas', 'oas') as f_nas_omni
FROM (
SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,adid,
if(adid RLIKE '{regex_pla}', {extract_adid_lia}, if(adid RLIKE '{regex_lia}', {extract_adid_lia}, if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}))) AS adid_omni,
if(adid RLIKE '{regex_pla}', {channel_id_plalia}, if(adid RLIKE '{regex_lia}', {channel_id_plalia}, if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}))) AS channel_id_omni,
coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '0') sellerid_ad_old,
coalesce(parse_url(page_url, 'QUERY','wl12'), '') catalog_item_id_ad_omni_old
   FROM {fair_daily_auth_order_to_visit_chain},
   (
            SELECT id customer_id,
                   reflector_identifier,
                   channel_id_position,
                   adid_length
            FROM {sem_customers} c where id=1
    ) c
   WHERE visit_vehicle='labs sem'
       AND ds = '{ds}'
       and adid is not null
       AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2)
       where order_no_omni = '200010802154653'""".format(ds=ds,
                                  sem_customers=sem_customers,
                                  fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                  regex_lia=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                  regex_pla=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+_[0-9]+$',
                                  extract_adid_lia=r'substring(adid, channel_id_position+2)',
                                  extract_adid_pla=r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                  extract_adid_reflector=r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                  channel_id_plalia=r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                  channel_id_reflector=r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                  )).toPandas()

,order_no_omni,page_url,visit_hit_time_gmt,order_date_omni,adid_omni,channel_id_omni,adid,catalog_item_id_ad_omni,sellerid_ad_omni,f_nas_omni
0,200010802154653,https://www.walmart.com/ip/Round-Coffee-Table-...,1679372994,2023-03-26,524753293,22,22222222222524753293,256151387,101103537,oas
1,200010802154653,https://www.walmart.com/ip/Delta-Children-Case...,1679872739,2023-03-26,653389093_117755028669_12420145346,77,22222222277653389093_117755028669_12420145346,653389093,0,nas
2,200010802154653,https://www.walmart.com/ip/KOHLER-CB-CLR1620FS...,1675634270,2023-03-26,0,27,22222222227000000000,1857740207,101219667,oas
3,200010802154653,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,27,22222222227000000000,854547111,0,oas
4,200010802154653,https://www.walmart.com/ip/Delta-Children-Haye...,1679283091,2023-03-26,1511381225_117755028669_12420145346,77,222222222771511381225_117755028669_12420145346,1511381225,0,nas
5,200010802154653,https://www.walmart.com/ip/Truly-Soft-Everyday...,1675749648,2023-03-26,0,27,22222222227000000000,482573538,0,oas
6,200010802154653,https://www.walmart.com/ip/Delta-Children-Haye...,1679624348,2023-03-26,1511381225_117755028669_12420145346,77,222222222771511381225_117755028669_12420145346,1511381225,0,nas
7,200010802154653,https://www.walmart.com/ip/SAFAVIEH-Glamour-Pi...,1675210059,2023-03-26,345061631,22,22222222222345061631,898106547,17374,oas
8,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,22222222222404313442,997083873,0,oas
9,200010802154653,https://www.walmart.com/ip/Better-Homes-Garden...,1674499045,2023-03-26,138450655_117755028669_12420145346,77,22222222277138450655_117755028669_12420145346,138450655,0,nas


In [77]:
NUM_ATTRIBUTION_DAYS = 14
ds = '2023-03-26'
def is_dst(when):
    '''Given the name of Timezone will determine if that timezone is in Daylight Saving Time now (DST)'''
    return when.dst() != timedelta(0)
import pytz
sem_customers = 'sem.customers'
fair_daily_auth_order_to_visit_chain = 'fair.daily_auth_order_to_visit_chain'
fair_daily_auth_order_to_ref_vehicle = 'fair.daily_auth_order_to_ref_vehicle'
ww_crew_dl_rpt_vm_cnsld_order_item = 'ww_crew_dl_rpt_vm.cnsld_order_item'

In [113]:
sqlCmd = """SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,
   if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}) AS adid_omni,
   if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}) AS channel_id_omni,
   coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '') sellerid_ad,
   coalesce(parse_url(page_url, 'QUERY','wl12'), '') catalog_item_id_ad_omni
   FROM {fair_daily_auth_order_to_visit_chain},
   (
            SELECT id customer_id,
                   reflector_identifier,
                   channel_id_position,
                   adid_length
            FROM {sem_customers} c where id=1
    ) c
   WHERE visit_vehicle='labs sem'
       AND length(adid)-length(replace(adid,'_',''))=0
       AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2
       and order_no = '200010802154653'
       AND ds = '{ds}'""".format(ds = ds,
                                  fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                  sem_customers=sem_customers,
                                  regex_lia = r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                  extract_adid_lia = r'substring(adid, channel_id_position+2)',
                                  extract_adid_pla = r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                  extract_adid_reflector = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                  channel_id_plalia = r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                  channel_id_reflector = r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                 )

visit_chain = spark.sql(sqlCmd)

# determine if ad is_target_all
visit_chain = visit_chain.withColumn("is_target_all_omni", func.when((col("adid_omni")=='0'), func.lit(1)
                                                          ).otherwise(func.lit(0)))

# filter out to only have pla
visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                                          ).otherwise(func.lit(0)))

visit_chain = visit_chain.withColumn("is_pla_omni", func.when(col("channel_id_omni").isin([54, 55, 64, 65]), func.lit(2)
                                                          ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.withColumn("is_pla_omni", func.when(col("channel_id_omni").isin([37, 38, 77, 78]), func.lit(4)
                                                          ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.filter(col('is_pla_omni')>0)

# get the sourceid from channel id google = 2 bing =4, other = 6
visit_chain = visit_chain.withColumn("sourceid_omni", func.lit(6))
visit_chain = visit_chain.withColumn("sourceid_omni", func.when(col("channel_id_omni").isin([20, 21, 27, 28, 37, 38, 77, 78]), func.lit(2)
                                                          ).otherwise(col('sourceid_omni')))
visit_chain = visit_chain.withColumn("sourceid_omni", func.when(col("channel_id_omni").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                                          ).otherwise(col('sourceid_omni')))

visit_chain = visit_chain.withColumn('catalog_item_id_ad_omni',
                                 func.when(col('sourceid_omni')==4,
                                 func.split(col('catalog_item_id_ad_omni'), '_')[0]).otherwise(col('catalog_item_id_ad_omni')))

# 2. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds) or 8 hours depending on whether there is daylight savings
# Zhiyu Xie confirms that all ads in the visit_chain are before orders on the seconds level
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
datetime_pst = pytz.timezone('US/Pacific').localize(datetime.strptime(ds, "%Y-%m-%d"))
is_dst_val = is_dst(datetime_pst)
if(is_dst_val):
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format((visit_chain.visit_hit_time_gmt - 25200).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))
else:
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format((visit_chain.visit_hit_time_gmt - 28800).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

visit_chain = visit_chain.withColumn("datediff_browseorder", func.datediff(col("order_date_omni"),col("ad_date_omni")))
visit_chain = visit_chain.filter(col('datediff_browseorder')<NUM_ATTRIBUTION_DAYS)

visit_chain_test = visit_chain.toPandas()

# 3. now get only OMS
oms_auth_order = spark.sql("""SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id,
           if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
        WHERE ds = '{ds}'
        and order_no = '200010802154653'
        """.format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                    ds=ds,
                                    sem_customers=sem_customers,
                                    extract_adid_lia = r'substring(reflector_id, channel_id_position+2)',
                                    extract_adid_pla = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)'
                                   ))

# add is_pla and sourceid
oms_auth_order = oms_auth_order.drop_duplicates(['order_no_oms'])

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                                          ).otherwise(func.lit(0)))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([54, 55, 64, 65]), func.lit(2)
                                                          ).otherwise(col('is_pla_oms')))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([37, 38, 77, 78]), func.lit(4)
                                                          ).otherwise(col('is_pla_oms')))

# get the sourceid from channel id google = 2 bing =4, other = 6
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.lit(6))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.when(col("channel_id_oms").isin([20, 21, 27, 28, 37, 38, 77, 78]), func.lit(2)
                                                          ).otherwise(col('sourceid_oms')))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.when(col("channel_id_oms").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                                          ).otherwise(col('sourceid_oms')))

oms_auth_order = oms_auth_order.withColumn("is_target_all_oms", func.when((col("adid_oms")=='0'), func.lit(1)
                                                          ).otherwise(func.lit(0)))

# get the catalog_item_id from the adid
adid2cat = spark.sql("""SELECT
                            adid, max(catalog_item_id_ad_oms) as catalog_item_id_ad_oms
                        FROM
                            ( SELECT CAST(id AS INT) as adid,
                                     catalog_item_id as catalog_item_id_ad_oms
                              FROM sem.wmt_dotcom_items )
                        GROUP BY adid""")

oms_auth_order = oms_auth_order.join(adid2cat,
            (oms_auth_order.adid_oms == adid2cat.adid),
             how='left').drop(adid2cat.adid)

oms_auth_order_test = oms_auth_order.join(adid2cat,
            (oms_auth_order.adid_oms == adid2cat.adid),
             how='left').drop(adid2cat.adid).toPandas()

# first filter out the omni orders to only have orders that also appear in oms
# supposedly if an order doesn't appear in oms - then it isn't valid/authorized
valid_orders = oms_auth_order.select(col("order_no_oms").alias("valid_order"))
valid_orders = valid_orders.drop_duplicates()
visit_chain = valid_orders.join(visit_chain,
                    (valid_orders.valid_order == visit_chain.order_no_omni),
                     how='left').drop(valid_orders.valid_order)

# now join together oms + visit_chain
ads = oms_auth_order.join(visit_chain,
                          ((oms_auth_order.order_no_oms == visit_chain.order_no_omni) &
                          (oms_auth_order.catalog_item_id_ad_oms == visit_chain.catalog_item_id_ad_omni) &
                          (oms_auth_order.order_date_oms == visit_chain.order_date_omni)),
                          how='outer')

# filter out to only have pla/dsa
ads = ads.withColumn("is_pla", func.coalesce(col("is_pla_omni"), col("is_pla_oms")))
ads = ads.filter(col('is_pla')>0)

ads_test = ads.withColumn("is_target_all", func.coalesce(col("is_target_all_omni"), col("is_target_all_oms"))).toPandas()



In [114]:
visit_chain_test

,order_no_omni,page_url,visit_hit_time_gmt,order_date_omni,adid_omni,channel_id_omni,sellerid_ad,catalog_item_id_ad_omni,is_target_all_omni,is_pla_omni,sourceid_omni,ad_date_omni,datediff_browseorder
0,200010802154653,https://www.walmart.com/ip/Round-Coffee-Table-...,1679372994,2023-03-26,524753293,22,101103537,256151387,0,1,4,2023-03-20,6
1,200010802154653,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,27,0,854547111,1,1,2,2023-03-26,0
2,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,0,997083873,0,1,4,2023-03-26,0


In [112]:
oms_auth_order_test

,order_no_oms,order_date_oms,reflector_id,adid_oms,channel_id_oms,ad_date_oms,is_pla_oms,sourceid_oms,is_target_all_oms,catalog_item_id_ad_oms,catalog_item_id_ad_oms
0,200010802154653,2023-03-26,imp_R80SzAzSZxyNUjNVVhy4-XQEUkAWDbVNUwmuxU0,SZxyNUjNVVhy4-XQEUkAWDbVNUwmuxU0,None,2023-03-26,0,6,0,None,None


In [110]:
ads_test

,order_no_oms,order_date_oms,reflector_id,adid_oms,channel_id_oms,ad_date_oms,is_pla_oms,sourceid_oms,is_target_all_oms,catalog_item_id_ad_oms,order_no_omni,page_url,visit_hit_time_gmt,order_date_omni,adid_omni,channel_id_omni,sellerid_ad,catalog_item_id_ad_omni,is_target_all_omni,is_pla_omni,sourceid_omni,ad_date_omni,datediff_browseorder,is_pla,is_target_all
0,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,0,997083873,0,1,4,2023-03-26,0,1,0
1,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Round-Coffee-Table-...,1679372994,2023-03-26,524753293,22,101103537,256151387,0,1,4,2023-03-20,6,1,0
2,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,27,0,854547111,1,1,2,2023-03-26,0,1,1


In [117]:
ads_test['order_no_omni'].unique()

array(['200010802154653'], dtype=object)

In [115]:
# 3. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ads = ads.withColumn("ad_date", func.coalesce(col("ad_date_omni"), col("ad_date_oms")))
ads = ads.withColumn("order_date", func.coalesce(col("order_date_omni"), col("order_date_oms")))
ads = ads.withColumn("datediff_browseorder", func.datediff(col("order_date"),col("ad_date")))
ads = ads.filter(col('datediff_browseorder')<NUM_ATTRIBUTION_DAYS)

# some coalescing that needs to be done after connecting oms and fair
ads = ads.withColumn("adid", func.coalesce(col("adid_omni"), col("adid_oms")))
ads = ads.withColumn("catalog_item_id_ad", func.coalesce(col("catalog_item_id_ad_omni"), col("catalog_item_id_ad_oms")))

ads = ads.withColumn("source_id", func.coalesce(col("sourceid_omni"), col("sourceid_oms")))
ads = ads.withColumn("channel_id", func.coalesce(col("channel_id_omni"), col("channel_id_oms")))
ads = ads.withColumn("order_no", func.coalesce(col("order_no_omni"), col("order_no_oms")))

#for oms - will just assume the seller is walmart - since there is no seller indicator
ads = ads.withColumn("sellerid_ad", func.coalesce(col("sellerid_ad"), func.lit("0")))


# 4. COIM: get line items + GMV per order listed in the visit chain
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
sqlCmd =  """SELECT order_line_nbr, order_nbr as sales_order_grp_num,
                    sum(gmv_plcd_amt) as gmv_plcd_amt,
                    sum(gmv_order_amt) as gmv_order_amt,
                    max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
                    max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
                    max(slr_org_cd) as slr_org_cd_purchase,
                    max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
             FROM """+ww_crew_dl_rpt_vm_cnsld_order_item+"""
             WHERE op_cmpny_cd = 'WMT.COM'
                   AND GMV_IND = 1
                   AND gmv_cr_ind = 1
                   AND order_nbr IS NOT NULL
                   AND trim(order_nbr) != ''
                   AND order_plcd_dt = '"""+ds+"""'
                   and order_nbr = '200010802154653'
             GROUP BY order_nbr, order_line_nbr"""

transact = spark.sql(sqlCmd)

ad_trans_pairs_test = ads.join(transact,
                (ads.order_no == transact.sales_order_grp_num),
                 how='inner').drop(transact.sales_order_grp_num).toPandas()

In [116]:
ad_trans_pairs_test[ad_trans_pairs_test['source_id']==4]

,order_no_oms,order_date_oms,reflector_id,adid_oms,channel_id_oms,ad_date_oms,is_pla_oms,sourceid_oms,is_target_all_oms,catalog_item_id_ad_oms,order_no_omni,page_url,visit_hit_time_gmt,order_date_omni,adid_omni,channel_id_omni,sellerid_ad,catalog_item_id_ad_omni,is_target_all_omni,is_pla_omni,sourceid_omni,ad_date_omni,datediff_browseorder,is_pla,ad_date,order_date,adid,catalog_item_id_ad,source_id,channel_id,order_no,order_line_nbr,gmv_plcd_amt,gmv_order_amt,purchase_brand,catlg_item_id_purchase,slr_org_cd_purchase,svc_fulfmt_type_desc
0,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,0,997083873,0,1,4,2023-03-26,0,1,2023-03-26,2023-03-26,404313442,997083873,4,22,200010802154653,3,6.640000000,6.640000000,littlestarorganic,142918905,0,UNSCHEDULED_DELIVERY
1,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,0,997083873,0,1,4,2023-03-26,0,1,2023-03-26,2023-03-26,404313442,997083873,4,22,200010802154653,2,18.920000000,18.920000000,gerber,451560646,0,UNSCHEDULED_DELIVERY
2,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Round-Coffee-Table-...,1679372994,2023-03-26,524753293,22,101103537,256151387,0,1,4,2023-03-20,6,1,2023-03-20,2023-03-26,524753293,256151387,4,22,200010802154653,3,6.640000000,6.640000000,littlestarorganic,142918905,0,UNSCHEDULED_DELIVERY
3,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Round-Coffee-Table-...,1679372994,2023-03-26,524753293,22,101103537,256151387,0,1,4,2023-03-20,6,1,2023-03-20,2023-03-26,524753293,256151387,4,22,200010802154653,2,18.920000000,18.920000000,gerber,451560646,0,UNSCHEDULED_DELIVERY


In [99]:
sqlCmd = """SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,
   if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}) AS adid_omni,
   if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}) AS channel_id_omni,
   coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '') sellerid_ad,
   coalesce(parse_url(page_url, 'QUERY','wl12'), '') catalog_item_id_ad_omni
   FROM {fair_daily_auth_order_to_visit_chain},
   (
            SELECT id customer_id,
                   reflector_identifier,
                   channel_id_position,
                   adid_length
            FROM {sem_customers} c where id=1
    ) c
   WHERE visit_vehicle='labs sem'
       AND length(adid)-length(replace(adid,'_',''))=0
       AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2
       and order_no = '200010802154653'
       AND ds = '{ds}'""".format(ds = ds,
                                  fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                  sem_customers=sem_customers,
                                  regex_lia = r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                  extract_adid_lia = r'substring(adid, channel_id_position+2)',
                                  extract_adid_pla = r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                  extract_adid_reflector = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                  channel_id_plalia = r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                  channel_id_reflector = r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                 )

visit_chain = spark.sql(sqlCmd)

# determine if ad is_target_all
visit_chain = visit_chain.withColumn("is_target_all_omni", func.when((col("adid_omni")=='0'), func.lit(1)
                                                          ).otherwise(func.lit(0)))

# filter out to only have pla
visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                                          ).otherwise(func.lit(0)))

visit_chain = visit_chain.withColumn("is_pla_omni", func.when(col("channel_id_omni").isin([54, 55, 64, 65]), func.lit(2)
                                                          ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.withColumn("is_pla_omni", func.when(col("channel_id_omni").isin([37, 38, 77, 78]), func.lit(4)
                                                          ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.filter(col('is_pla_omni')>0)

# get the sourceid from channel id google = 2 bing =4, other = 6
visit_chain = visit_chain.withColumn("sourceid_omni", func.lit(6))
visit_chain = visit_chain.withColumn("sourceid_omni", func.when(col("channel_id_omni").isin([20, 21, 27, 28, 37, 38, 77, 78]), func.lit(2)
                                                          ).otherwise(col('sourceid_omni')))
visit_chain = visit_chain.withColumn("sourceid_omni", func.when(col("channel_id_omni").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                                          ).otherwise(col('sourceid_omni')))

visit_chain = visit_chain.withColumn('catalog_item_id_ad_omni',
                                 func.when(col('sourceid_omni')==4,
                                 func.split(col('catalog_item_id_ad_omni'), '_')[0]).otherwise(col('catalog_item_id_ad_omni')))

# 2. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds) or 8 hours depending on whether there is daylight savings
# Zhiyu Xie confirms that all ads in the visit_chain are before orders on the seconds level
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
datetime_pst = pytz.timezone('US/Pacific').localize(datetime.strptime(ds, "%Y-%m-%d"))
is_dst_val = is_dst(datetime_pst)
if(is_dst_val):
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format((visit_chain.visit_hit_time_gmt - 25200).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))
else:
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format((visit_chain.visit_hit_time_gmt - 28800).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

visit_chain = visit_chain.withColumn("datediff_browseorder", func.datediff(col("order_date_omni"),col("ad_date_omni")))
visit_chain = visit_chain.filter(col('datediff_browseorder')<NUM_ATTRIBUTION_DAYS)

# 3. now get only OMS
oms_auth_order = spark.sql("""SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id,
           if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
        WHERE ds = '{ds}'
        and order_no = '200010802154653'
        """.format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                    ds=ds,
                                    sem_customers=sem_customers,
                                    extract_adid_lia = r'substring(reflector_id, channel_id_position+2)',
                                    extract_adid_pla = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)'
                                   ))

# add is_pla and sourceid
oms_auth_order = oms_auth_order.drop_duplicates(['order_no_oms'])

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                                          ).otherwise(func.lit(0)))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([54, 55, 64, 65]), func.lit(2)
                                                          ).otherwise(col('is_pla_oms')))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([37, 38, 77, 78]), func.lit(4)
                                                          ).otherwise(col('is_pla_oms')))

# get the sourceid from channel id google = 2 bing =4, other = 6
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.lit(6))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.when(col("channel_id_oms").isin([20, 21, 27, 28, 37, 38, 77, 78]), func.lit(2)
                                                          ).otherwise(col('sourceid_oms')))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.when(col("channel_id_oms").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                                          ).otherwise(col('sourceid_oms')))

oms_auth_order = oms_auth_order.withColumn("is_target_all_oms", func.when((col("adid_oms")=='0'), func.lit(1)
                                                          ).otherwise(func.lit(0)))

# get the catalog_item_id from the adid
adid2cat = spark.sql("""SELECT
                            adid, max(catalog_item_id_ad_oms) as catalog_item_id_ad_oms
                        FROM
                            ( SELECT CAST(id AS INT) as adid,
                                     catalog_item_id as catalog_item_id_ad_oms
                              FROM sem.wmt_dotcom_items )
                        GROUP BY adid""")

oms_auth_order = oms_auth_order.join(adid2cat,
            (oms_auth_order.adid_oms == adid2cat.adid),
             how='left').drop(adid2cat.adid)

# first filter out the omni orders to only have orders that also appear in oms
# supposedly if an order doesn't appear in oms - then it isn't valid/authorized
valid_orders = oms_auth_order.select(col("order_no_oms").alias("valid_order"))
valid_orders = valid_orders.drop_duplicates()
visit_chain = valid_orders.join(visit_chain,
                    (valid_orders.valid_order == visit_chain.order_no_omni),
                     how='left').drop(valid_orders.valid_order)

# now join together oms + visit_chain
ads = oms_auth_order.join(visit_chain,
                          ((oms_auth_order.order_no_oms == visit_chain.order_no_omni) &
                          (oms_auth_order.catalog_item_id_ad_oms == visit_chain.catalog_item_id_ad_omni) &
                          (oms_auth_order.order_date_oms == visit_chain.order_date_omni)),
                          how='outer')

# filter out to only have pla/dsa
ads = ads.withColumn("is_pla", func.coalesce(col("is_pla_omni"), col("is_pla_oms")))
ads = ads.filter(col('is_pla')>0)

ads = ads.withColumn("is_target_all", func.coalesce(col("is_target_all_omni"), col("is_target_all_oms")))

# 3. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ads = ads.withColumn("ad_date", func.coalesce(col("ad_date_omni"), col("ad_date_oms")))
ads = ads.withColumn("order_date", func.coalesce(col("order_date_omni"), col("order_date_oms")))
ads = ads.withColumn("datediff_browseorder", func.datediff(col("order_date"),col("ad_date")))
ads = ads.filter(col('datediff_browseorder')<NUM_ATTRIBUTION_DAYS)

# some coalescing that needs to be done after connecting oms and fair
ads = ads.withColumn("adid", func.coalesce(col("adid_omni"), col("adid_oms")))
ads = ads.withColumn("catalog_item_id_ad", func.coalesce(col("catalog_item_id_ad_omni"), col("catalog_item_id_ad_oms")))

ads = ads.withColumn("source_id", func.coalesce(col("sourceid_omni"), col("sourceid_oms")))
ads = ads.withColumn("channel_id", func.coalesce(col("channel_id_omni"), col("channel_id_oms")))
ads = ads.withColumn("order_no", func.coalesce(col("order_no_omni"), col("order_no_oms")))

#for oms - will just assume the seller is walmart - since there is no seller indicator
ads = ads.withColumn("sellerid_ad", func.coalesce(col("sellerid_ad"), func.lit("0")))


# 4. COIM: get line items + GMV per order listed in the visit chain
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
sqlCmd =  """SELECT order_line_nbr, order_nbr as sales_order_grp_num,
                    sum(gmv_plcd_amt) as gmv_plcd_amt,
                    sum(gmv_order_amt) as gmv_order_amt,
                    max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
                    max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
                    max(slr_org_cd) as slr_org_cd_purchase,
                    max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
             FROM """+ww_crew_dl_rpt_vm_cnsld_order_item+"""
             WHERE op_cmpny_cd = 'WMT.COM'
                   AND GMV_IND = 1
                   AND gmv_cr_ind = 1
                   AND order_nbr IS NOT NULL
                   AND trim(order_nbr) != ''
                   AND order_plcd_dt = '"""+ds+"""'
                   and order_nbr = '200010802154653'
             GROUP BY order_nbr, order_line_nbr"""

transact = spark.sql(sqlCmd)

ad_trans_pairs = ads.join(transact,
                (ads.order_no == transact.sales_order_grp_num),
                 how='inner').drop(transact.sales_order_grp_num)


# 3. Get the brand name for the ads
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x:x).map(lambda x : x.replace("ts=","")).max()

item_info_dfm = spark.sql("""SELECT item_id as catlg_item_id,
                                    max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
            FROM dmas.dfm
            WHERE ts='"""+partitionDate+"""'
            GROUP BY item_id""")

ad_trans_pairs = ad_trans_pairs.join(item_info_dfm,
            (ad_trans_pairs.catalog_item_id_ad == item_info_dfm.catlg_item_id),
             how='left').drop(item_info_dfm.catlg_item_id)

# 3. Get the coop event id when relevant
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
coop_items = spark.sql("""SELECT wdi.catalog_item_id as catalog_item_id_coop,
                                 max(co.coop_event_id) as coop_event_id
                         FROM
                         ((
                            SELECT distinct adid as adid, coop_event_id as coop_event_id
                            FROM casesci_sem.coop_split_ad_links
                            WHERE date_string = '{ds}') co
                            INNER JOIN
                                (SELECT CAST(id AS INT) as adid, catalog_item_id
                                FROM sem.wmt_dotcom_items) wdi
                                ON co.adid = wdi.adid)
                            GROUP BY wdi.catalog_item_id""".format(ds = ds))

coop_items_ad = coop_items.select(col('catalog_item_id_coop'),
                                  col('coop_event_id').alias('coop_event_id_ad'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_ad,
                (ad_trans_pairs.catalog_item_id_ad == coop_items_ad.catalog_item_id_coop),
                 how='left').drop(coop_items_ad.catalog_item_id_coop)

coop_items_purchase = coop_items.select(col('catalog_item_id_coop'),
                                  col('coop_event_id').alias('coop_event_id_purchase'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_purchase,
                (ad_trans_pairs.catlg_item_id_purchase == coop_items_purchase.catalog_item_id_coop),
                 how='left').drop(coop_items_purchase.catalog_item_id_coop)

# 5. Mark which line items are the same advertiser and the precise same item as the adid (featured item)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

ad_trans_pairs = ad_trans_pairs.withColumn("same_item", func.when(col("catlg_item_id_purchase")==col("catalog_item_id_ad"),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_brand", func.when(col("purchase_brand")==col("ad_item_brand"),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_coop", func.when((col("coop_event_id_ad")==col("coop_event_id_purchase"))
                                                                    & col('coop_event_id_ad').isNotNull(),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller", func.when(col("slr_org_cd_purchase")==col("sellerid_ad"),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_item", func.when((col("same_seller")==1) & (col("same_item")==1),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_brand", func.when((col("same_seller")==1) & (col("same_brand")==1),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_coop", func.when((col("same_seller")==1) & (col("same_coop")==1),
                                                        func.lit(1)).otherwise(func.lit(0)))

# filter out irrelevant items

ad_trans_pairs_1 = ad_trans_pairs.filter((col('same_item') +
                                        col('same_brand') +
                                        col('same_coop') +
                                        col('same_seller'))>0).toPandas()

In [118]:
# v1 pipeline
sqlCmd = """SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,
   if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}) AS adid_omni,
   if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}) AS channel_id_omni,
   coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '') sellerid_ad,
   coalesce(parse_url(page_url, 'QUERY','wl12'), '') catalog_item_id_ad_omni
   FROM {fair_daily_auth_order_to_visit_chain},
   (
            SELECT id customer_id,
                   reflector_identifier,
                   channel_id_position,
                   adid_length
            FROM {sem_customers} c where id=1
    ) c
   WHERE visit_vehicle='labs sem'
       AND length(adid)-length(replace(adid,'_',''))=0
       AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2
       and order_no = '200010802154653'
       AND ds = '{ds}'""".format(ds = ds,
                                  fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                  sem_customers=sem_customers,
                                  regex_lia = r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                  extract_adid_lia = r'substring(adid, channel_id_position+2)',
                                  extract_adid_pla = r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                  extract_adid_reflector = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                  channel_id_plalia = r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                  channel_id_reflector = r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                 )

visit_chain = spark.sql(sqlCmd)

# determine if ad is_target_all
visit_chain = visit_chain.withColumn("is_target_all_omni", func.when((col("adid_omni")=='0'), func.lit(1)
                                                          ).otherwise(func.lit(0)))

# filter out to only have pla
visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                                          ).otherwise(func.lit(0)))

visit_chain = visit_chain.withColumn("is_pla_omni", func.when(col("channel_id_omni").isin([54, 55, 64, 65]), func.lit(2)
                                                          ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.withColumn("is_pla_omni", func.when(col("channel_id_omni").isin([37, 38, 77, 78]), func.lit(4)
                                                          ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.filter(col('is_pla_omni')>0)

# get the sourceid from channel id google = 2 bing =4, other = 6
visit_chain = visit_chain.withColumn("sourceid_omni", func.lit(6))
visit_chain = visit_chain.withColumn("sourceid_omni", func.when(col("channel_id_omni").isin([20, 21, 27, 28, 37, 38, 77, 78]), func.lit(2)
                                                          ).otherwise(col('sourceid_omni')))
visit_chain = visit_chain.withColumn("sourceid_omni", func.when(col("channel_id_omni").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                                          ).otherwise(col('sourceid_omni')))

visit_chain = visit_chain.withColumn('catalog_item_id_ad_omni',
                                 func.when(col('sourceid_omni')==4,
                                 func.split(col('catalog_item_id_ad_omni'), '_')[0]).otherwise(col('catalog_item_id_ad_omni')))

# 2. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds) or 8 hours depending on whether there is daylight savings
# Zhiyu Xie confirms that all ads in the visit_chain are before orders on the seconds level
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
datetime_pst = pytz.timezone('US/Pacific').localize(datetime.strptime(ds, "%Y-%m-%d"))
is_dst_val = is_dst(datetime_pst)
if(is_dst_val):
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format((visit_chain.visit_hit_time_gmt - 25200).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))
else:
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format((visit_chain.visit_hit_time_gmt - 28800).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

visit_chain = visit_chain.withColumn("datediff_browseorder", func.datediff(col("order_date_omni"),col("ad_date_omni")))
visit_chain = visit_chain.filter(col('datediff_browseorder')<NUM_ATTRIBUTION_DAYS)

# 3. now get only OMS
oms_auth_order = spark.sql("""SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id,
           if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
        WHERE ds = '{ds}'
        and order_no = '200010802154653'
        """.format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                    ds=ds,
                                    sem_customers=sem_customers,
                                    extract_adid_lia = r'substring(reflector_id, channel_id_position+2)',
                                    extract_adid_pla = r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)'
                                   ))

# add is_pla and sourceid
oms_auth_order = oms_auth_order.drop_duplicates(['order_no_oms'])

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                                          ).otherwise(func.lit(0)))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([54, 55, 64, 65]), func.lit(2)
                                                          ).otherwise(col('is_pla_oms')))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms", func.when(col("channel_id_oms").isin([37, 38, 77, 78]), func.lit(4)
                                                          ).otherwise(col('is_pla_oms')))

# get the sourceid from channel id google = 2 bing =4, other = 6
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.lit(6))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.when(col("channel_id_oms").isin([20, 21, 27, 28, 37, 38, 77, 78]), func.lit(2)
                                                          ).otherwise(col('sourceid_oms')))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.when(col("channel_id_oms").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                                          ).otherwise(col('sourceid_oms')))

oms_auth_order = oms_auth_order.withColumn("is_target_all_oms", func.when((col("adid_oms")=='0'), func.lit(1)
                                                          ).otherwise(func.lit(0)))

# get the catalog_item_id from the adid
adid2cat = spark.sql("""SELECT
                            adid, max(catalog_item_id_ad_oms) as catalog_item_id_ad_oms
                        FROM
                            ( SELECT CAST(id AS INT) as adid,
                                     catalog_item_id as catalog_item_id_ad_oms
                              FROM sem.wmt_dotcom_items )
                        GROUP BY adid""")

oms_auth_order = oms_auth_order.join(adid2cat,
            (oms_auth_order.adid_oms == adid2cat.adid),
             how='left').drop(adid2cat.adid)

# first filter out the omni orders to only have orders that also appear in oms
# supposedly if an order doesn't appear in oms - then it isn't valid/authorized
valid_orders = oms_auth_order.select(col("order_no_oms").alias("valid_order"))
valid_orders = valid_orders.drop_duplicates()
visit_chain = valid_orders.join(visit_chain,
                    (valid_orders.valid_order == visit_chain.order_no_omni),
                     how='left').drop(valid_orders.valid_order)

# now join together oms + visit_chain
ads = oms_auth_order.join(visit_chain,
                          ((oms_auth_order.order_no_oms == visit_chain.order_no_omni) &
                          (oms_auth_order.catalog_item_id_ad_oms == visit_chain.catalog_item_id_ad_omni) &
                          (oms_auth_order.order_date_oms == visit_chain.order_date_omni)),
                          how='outer')

# filter out to only have pla/dsa
ads = ads.withColumn("is_pla", func.coalesce(col("is_pla_omni"), col("is_pla_oms")))
ads = ads.filter(col('is_pla')>0)

ads = ads.withColumn("is_target_all", func.coalesce(col("is_target_all_omni"), col("is_target_all_oms")))

# 3. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ads = ads.withColumn("ad_date", func.coalesce(col("ad_date_omni"), col("ad_date_oms")))
ads = ads.withColumn("order_date", func.coalesce(col("order_date_omni"), col("order_date_oms")))
ads = ads.withColumn("datediff_browseorder", func.datediff(col("order_date"),col("ad_date")))
ads = ads.filter(col('datediff_browseorder')<NUM_ATTRIBUTION_DAYS)

# some coalescing that needs to be done after connecting oms and fair
ads = ads.withColumn("adid", func.coalesce(col("adid_omni"), col("adid_oms")))
ads = ads.withColumn("catalog_item_id_ad", func.coalesce(col("catalog_item_id_ad_omni"), col("catalog_item_id_ad_oms")))

ads = ads.withColumn("source_id", func.coalesce(col("sourceid_omni"), col("sourceid_oms")))
ads = ads.withColumn("channel_id", func.coalesce(col("channel_id_omni"), col("channel_id_oms")))
ads = ads.withColumn("order_no", func.coalesce(col("order_no_omni"), col("order_no_oms")))

#for oms - will just assume the seller is walmart - since there is no seller indicator
ads = ads.withColumn("sellerid_ad", func.coalesce(col("sellerid_ad"), func.lit("0")))


# 4. COIM: get line items + GMV per order listed in the visit chain
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
sqlCmd =  """SELECT order_line_nbr, order_nbr as sales_order_grp_num,
                    sum(gmv_plcd_amt) as gmv_plcd_amt,
                    sum(gmv_order_amt) as gmv_order_amt,
                    max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
                    max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
                    max(slr_org_cd) as slr_org_cd_purchase,
                    max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
             FROM """+ww_crew_dl_rpt_vm_cnsld_order_item+"""
             WHERE op_cmpny_cd = 'WMT.COM'
                   AND GMV_IND = 1
                   AND gmv_cr_ind = 1
                   AND order_nbr IS NOT NULL
                   AND trim(order_nbr) != ''
                   AND order_plcd_dt = '"""+ds+"""'
                   and order_nbr = '200010802154653'
             GROUP BY order_nbr, order_line_nbr"""

transact = spark.sql(sqlCmd)

ad_trans_pairs = ads.join(transact,
                (ads.order_no == transact.sales_order_grp_num),
                 how='inner').drop(transact.sales_order_grp_num)


# 3. Get the brand name for the ads
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x:x).map(lambda x : x.replace("ts=","")).max()

item_info_dfm = spark.sql("""SELECT item_id as catlg_item_id,
                                    max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
            FROM dmas.dfm
            WHERE ts='"""+partitionDate+"""'
            GROUP BY item_id""")

ad_trans_pairs = ad_trans_pairs.join(item_info_dfm,
            (ad_trans_pairs.catalog_item_id_ad == item_info_dfm.catlg_item_id),
             how='left').drop(item_info_dfm.catlg_item_id)

# 3. Get the coop event id when relevant
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
coop_items = spark.sql("""SELECT wdi.catalog_item_id as catalog_item_id_coop,
                                 max(co.coop_event_id) as coop_event_id
                         FROM
                         ((
                            SELECT distinct adid as adid, coop_event_id as coop_event_id
                            FROM casesci_sem.coop_split_ad_links
                            WHERE date_string = '{ds}') co
                            INNER JOIN
                                (SELECT CAST(id AS INT) as adid, catalog_item_id
                                FROM sem.wmt_dotcom_items) wdi
                                ON co.adid = wdi.adid)
                            GROUP BY wdi.catalog_item_id""".format(ds = ds))

coop_items_ad = coop_items.select(col('catalog_item_id_coop'),
                                  col('coop_event_id').alias('coop_event_id_ad'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_ad,
                (ad_trans_pairs.catalog_item_id_ad == coop_items_ad.catalog_item_id_coop),
                 how='left').drop(coop_items_ad.catalog_item_id_coop)

coop_items_purchase = coop_items.select(col('catalog_item_id_coop'),
                                  col('coop_event_id').alias('coop_event_id_purchase'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_purchase,
                (ad_trans_pairs.catlg_item_id_purchase == coop_items_purchase.catalog_item_id_coop),
                 how='left').drop(coop_items_purchase.catalog_item_id_coop)

# 5. Mark which line items are the same advertiser and the precise same item as the adid (featured item)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

ad_trans_pairs = ad_trans_pairs.withColumn("same_item", func.when(col("catlg_item_id_purchase")==col("catalog_item_id_ad"),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_brand", func.when(col("purchase_brand")==col("ad_item_brand"),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_coop", func.when((col("coop_event_id_ad")==col("coop_event_id_purchase"))
                                                                    & col('coop_event_id_ad').isNotNull(),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller", func.when(col("slr_org_cd_purchase")==col("sellerid_ad"),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_item", func.when((col("same_seller")==1) & (col("same_item")==1),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_brand", func.when((col("same_seller")==1) & (col("same_brand")==1),
                                                        func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_coop", func.when((col("same_seller")==1) & (col("same_coop")==1),
                                                        func.lit(1)).otherwise(func.lit(0)))

# filter out irrelevant items

ad_trans_pairs = ad_trans_pairs.filter((col('same_item') +
                                        col('same_brand') +
                                        col('same_coop') +
                                        col('same_seller'))>0)

# 6. Determine how many other ads can claim they influenced this purchase
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
lineitem_claimants = ad_trans_pairs.groupBy('order_no', 'order_line_nbr').agg(
                func.sum("same_item").alias('num_claimants_item'),
                func.sum("same_brand").alias('num_claimants_brand'),
                func.sum("same_coop").alias('num_claimants_coop'),
                func.sum("same_seller_item").alias('num_claimants_seller_item'),
                func.sum("same_seller_brand").alias('num_claimants_seller_brand'),
                func.sum("same_seller_coop").alias('num_claimants_seller_coop'),
                func.sum("same_seller").alias('num_claimants_seller')
                )


for atttype in ['item', 'brand', 'coop', 'seller_item', 'seller_brand', 'seller_coop', 'seller']:

    lineitem_claimants_i = lineitem_claimants.select(col("order_line_nbr").alias("order_line_nbr_summary"),
                                               col("order_no").alias("order_no_summary"),
                                               col("num_claimants_"+atttype))

    ad_trans_pairs = ad_trans_pairs.join(lineitem_claimants_i, (
                    (ad_trans_pairs.order_line_nbr == lineitem_claimants_i.order_line_nbr_summary) &
                    (ad_trans_pairs.order_no == lineitem_claimants_i.order_no_summary) &
                    (ad_trans_pairs['same_' + atttype] == 1)
                    ),
                     how='left').drop(lineitem_claimants_i.order_line_nbr_summary).drop(lineitem_claimants_i.order_no_summary)

ad_trans_pairs = ad_trans_pairs.fillna(0, subset=['num_claimants_item',
                                                  'num_claimants_brand',
                                                  'num_claimants_coop',
                                                  'num_claimants_seller_item',
                                                  'num_claimants_seller_brand',
                                                  'num_claimants_seller_coop',
                                                  'num_claimants_seller'
                                                 ])

# determine if purchase is 1p
ad_trans_pairs = ad_trans_pairs.withColumn("purchase_1p", func.when(col("slr_org_cd_purchase")=="0",
                                                        func.lit(1)).otherwise(func.lit(0)))

# 8. Save to item-level table - by order date
# ~~~~~~~~~~~~~~~~~~~~
columns_raw_table = ['datediff_browseorder', 'page_url', 'visit_hit_time_gmt', 'ad_date', 'adid',
                     'catlg_item_id_purchase', 'catalog_item_id_ad',
                     'svc_fulfmt_type_desc', 'purchase_1p',
                     'purchase_brand', 'ad_item_brand',
                     'coop_event_id_ad', 'coop_event_id_purchase',
                     'slr_org_cd_purchase', 'sellerid_ad',
                     'order_line_nbr', 'order_no',
                     'gmv_plcd_amt', 'gmv_order_amt',
                     'source_id', 'channel_id', 'is_pla', 'is_target_all',

                     'num_claimants_item', 'num_claimants_brand', 'num_claimants_coop',
                     'num_claimants_seller_item', 'num_claimants_seller_brand', 'num_claimants_seller_coop',
                     'num_claimants_seller',

                     'order_date']

coop_out = ad_trans_pairs.select(columns_raw_table)

In [119]:
qq = coop_out.toPandas()

In [120]:
qq

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_fulfmt_type_desc,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,order_date
0,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,142918905,997083873,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,2474,None,0,0,3,200010802154653,6.640000000,6.640000000,4,22,1,0,0,0,0,0,0,0,2,2023-03-26
1,0,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,142918905,854547111,UNSCHEDULED_DELIVERY,1,littlestarorganic,davinci,None,None,0,0,3,200010802154653,6.640000000,6.640000000,2,27,1,1,0,0,0,0,0,0,2,2023-03-26
2,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,451560646,997083873,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,2474,None,0,0,2,200010802154653,18.920000000,18.920000000,4,22,1,0,0,0,0,0,0,0,2,2023-03-26
3,0,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,451560646,854547111,UNSCHEDULED_DELIVERY,1,gerber,davinci,None,None,0,0,2,200010802154653,18.920000000,18.920000000,2,27,1,1,0,0,0,0,0,0,2,2023-03-26


In [95]:
sqlCmd = """SELECT order_no_omni,page_url,visit_hit_time_gmt, order_date_omni, adid_omni, channel_id_omni,
if(adid RLIKE '{regex_pla}', split(adid_omni,'_')[0], split(catalog_item_id_ad_omni_old,'_')[0]) as catalog_item_id_ad_omni,
coalesce(if(adid RLIKE '{regex_pla}', split(adid_omni,'_')[1], sellerid_ad_old), '0') as sellerid_ad_omni,
if(adid RLIKE '{regex_lia}' or adid RLIKE '{regex_pla}', 'nas', 'oas') as f_nas_omni
FROM (
SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,adid,
if(adid RLIKE '{regex_pla}', {extract_adid_lia}, if(adid RLIKE '{regex_lia}', {extract_adid_lia}, if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}))) AS adid_omni,
if(adid RLIKE '{regex_pla}', {channel_id_plalia}, if(adid RLIKE '{regex_lia}', {channel_id_plalia}, if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}))) AS channel_id_omni,
coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '0') sellerid_ad_old,
coalesce(parse_url(page_url, 'QUERY','wl12'), '') catalog_item_id_ad_omni_old
   FROM {fair_daily_auth_order_to_visit_chain},
   (
            SELECT id customer_id,
                   reflector_identifier,
                   channel_id_position,
                   adid_length
            FROM {sem_customers} c where id=1
    ) c
   WHERE visit_vehicle='labs sem'
       AND ds = '{ds}'
       and adid is not null
       AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2)
       where order_no_omni = '200010802154653'""".format(ds=ds,
                                  sem_customers=sem_customers,
                                  fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                  regex_lia=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                  regex_pla=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+_[0-9]+$',
                                  extract_adid_lia=r'substring(adid, channel_id_position+2)',
                                  extract_adid_pla=r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                  extract_adid_reflector=r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                  channel_id_plalia=r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                  channel_id_reflector=r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                  )
visit_chain = spark.sql(sqlCmd)
# determine if ad is_target_all
visit_chain = visit_chain.withColumn("is_target_all_omni", func.when((col("adid_omni") == '0'), func.lit(1)
                                                                     ).otherwise(func.lit(0)))

# filter out to only have pla
visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                               ).otherwise(func.lit(0)))

visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([54, 55, 64, 65]), func.lit(2)
                                               ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([37, 38, 77, 78]), func.lit(4)
                                               ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.filter(col('is_pla_omni') > 0)

# get the sourceid from channel id google = 2 bing =4, other = 6
visit_chain = visit_chain.withColumn("sourceid_omni", func.lit(6))
visit_chain = visit_chain.withColumn("sourceid_omni",
                                     func.when(col("channel_id_omni").isin([20, 21, 27, 28, 37, 38, 77, 78]),
                                               func.lit(2)
                                               ).otherwise(col('sourceid_omni')))
visit_chain = visit_chain.withColumn("sourceid_omni",
                                     func.when(col("channel_id_omni").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                               ).otherwise(col('sourceid_omni')))

visit_chain = visit_chain.withColumn('catalog_item_id_ad_omni',
                                     func.when(col('sourceid_omni') == 4,
                                               func.split(col('catalog_item_id_ad_omni'), '_')[0]).otherwise(
                                         col('catalog_item_id_ad_omni')))
# get is_mobile from channel id
visit_chain = visit_chain.withColumn("is_mobile_omni", func.when(
    col("channel_id_omni").isin([21, 23, 25, 28, 81, 85, 55, 65, 52, 38, 78]), func.lit(1)
    ).otherwise(lit(0)))

# 2. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds) or 8 hours depending on whether there is daylight savings
# Zhiyu Xie confirms that all ads in the visit_chain are before orders on the seconds level
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
datetime_pst = pytz.timezone('US/Pacific').localize(datetime.strptime(ds, "%Y-%m-%d"))
is_dst_val = is_dst(datetime_pst)
if (is_dst_val):
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format(
        (visit_chain.visit_hit_time_gmt - 25200).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))
else:
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format(
        (visit_chain.visit_hit_time_gmt - 28800).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

visit_chain = visit_chain.withColumn("datediff_browseorder",
                                     func.datediff(col("order_date_omni"), col("ad_date_omni")))
visit_chain = visit_chain.filter(col('datediff_browseorder') < NUM_ATTRIBUTION_DAYS)

# 3. now get only OMS
oms_auth_order = spark.sql("""select order_no_oms, order_date_oms, channel_id_oms, ad_date_oms, f_nas_oms, adid_oms,
        if(reflector_id RLIKE '{regex_lia_nas}' or reflector_id RLIKE '{regex_pla_nas}', split(adid_oms,'_')[0], NULL) as catalog_item_id_ad_oms,
        if(reflector_id RLIKE '{regex_pla_nas}', split(adid_oms,'_')[1], '0') as sellerid_ad_oms
        FROM (
            SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id, 
            if(reflector_id RLIKE '{regex_lia_nas}' or reflector_id RLIKE '{regex_pla_nas}', 'nas', 'oas') as f_nas_oms,
            if(reflector_id RLIKE '{regex_pla_nas}', {extract_adid_nas}, if(reflector_id RLIKE '{regex_lia_nas}', {extract_adid_nas}, if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}))) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
    WHERE ds = '{ds}')
    where order_no_oms = '200010802154653'""".format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                 ds=ds,
                                 sem_customers=sem_customers,
                                 regex_lia_nas=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                 regex_pla_nas=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+_[0-9]+$',
                                 extract_adid_lia=r'substring(reflector_id, channel_id_position+2)',
                                 extract_adid_pla=r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                 extract_adid_nas=r'substring(reflector_id, channel_id_position+2)',
                                 ))

# add is_pla and sourceid
oms_auth_order = oms_auth_order.drop_duplicates(['order_no_oms'])

oms_auth_order = oms_auth_order.withColumn("is_pla_oms",
                                           func.when(col("channel_id_oms").isin([81, 82, 27, 28, 22, 23]),
                                                     func.lit(1)
                                                     ).otherwise(func.lit(0)))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms",
                                           func.when(col("channel_id_oms").isin([54, 55, 64, 65]), func.lit(2)
                                                     ).otherwise(col('is_pla_oms')))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms",
                                           func.when(col("channel_id_oms").isin([37, 38, 77, 78]), func.lit(4)
                                                     ).otherwise(col('is_pla_oms')))

# get the sourceid from channel id google = 2 bing =4, other = 6
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.lit(6))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms",
                                           func.when(col("channel_id_oms").isin([20, 21, 27, 28, 37, 38, 77, 78]),
                                                     func.lit(2)
                                                     ).otherwise(col('sourceid_oms')))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms",
                                           func.when(col("channel_id_oms").isin([22, 23, 24, 25, 64, 65]),
                                                     func.lit(4)
                                                     ).otherwise(col('sourceid_oms')))

oms_auth_order = oms_auth_order.withColumn("is_target_all_oms", func.when((col("adid_oms") == '0'), func.lit(1)
                                                                          ).otherwise(func.lit(0)))
# get is_mobile from channel id
oms_auth_order = oms_auth_order.withColumn("is_mobile_oms", func.when(
    col("channel_id_oms").isin([21, 23, 25, 28, 81, 85, 55, 65, 52, 38, 78]), func.lit(1)
    ).otherwise(lit(0)))

# get the catalog_item_id from the adid for oas
adid2cat = spark.sql("""SELECT
                            adid, max(catalog_item_id_ad_oms) as catalog_item_id_ad_oms_dotcom
                        FROM
                            ( SELECT CAST(id AS INT) as adid,
                                     catalog_item_id as catalog_item_id_ad_oms
                              FROM sem.wmt_dotcom_items )
                        GROUP BY adid""")

oms_auth_order = oms_auth_order.join(adid2cat,
                                     (oms_auth_order.adid_oms == adid2cat.adid),
                                     how='left').drop(adid2cat.adid)
oms_auth_order = oms_auth_order.withColumn("catalog_item_id_ad_oms", func.coalesce(col("catalog_item_id_ad_oms"),
                                                                                   col("catalog_item_id_ad_oms_dotcom"))).drop(
    'catalog_item_id_ad_oms_dotcom')

# first filter out the omni orders to only have orders that also appear in oms
# supposedly if an order doesn't appear in oms - then it isn't valid/authorized
valid_orders = oms_auth_order.select(col("order_no_oms").alias("valid_order"))
valid_orders = valid_orders.drop_duplicates()
visit_chain = valid_orders.join(visit_chain,
                                (valid_orders.valid_order == visit_chain.order_no_omni),
                                how='left').drop(valid_orders.valid_order)

# now join together oms + visit_chain
ads = oms_auth_order.join(visit_chain,
                          ((oms_auth_order.order_no_oms == visit_chain.order_no_omni) &
                           # (oms_auth_order.catalog_item_id_ad_oms == visit_chain.catalog_item_id_ad_omni) &
                           # (oms_auth_order.sellerid_ad_oms == visit_chain.sellerid_ad_omni) &
                           (oms_auth_order.channel_id_oms == visit_chain.channel_id_omni) &
                           (oms_auth_order.adid_oms == visit_chain.adid_omni) &
                           (oms_auth_order.order_date_oms == visit_chain.order_date_omni)),
                          how='outer')

# filter out to only have pla/dsa
ads = ads.withColumn("is_pla", func.coalesce(col("is_pla_omni"), col("is_pla_oms")))
ads = ads.filter(col('is_pla') > 0)

ads = ads.withColumn("is_target_all", func.coalesce(col("is_target_all_omni"), col("is_target_all_oms")))

# 3. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ads = ads.withColumn("ad_date", func.coalesce(col("ad_date_omni"), col("ad_date_oms")))
ads = ads.withColumn("order_date", func.coalesce(col("order_date_omni"), col("order_date_oms")))
ads = ads.withColumn("datediff_browseorder", func.datediff(col("order_date"), col("ad_date")))
ads = ads.filter(col('datediff_browseorder') < NUM_ATTRIBUTION_DAYS)

# some coalescing that needs to be done after connecting oms and fair
ads = ads.withColumn("adid", func.coalesce(col("adid_omni"), col("adid_oms")))
ads = ads.withColumn("catalog_item_id_ad",
                     func.coalesce(col("catalog_item_id_ad_omni"), col("catalog_item_id_ad_oms")))

ads = ads.withColumn("source_id", func.coalesce(col("sourceid_omni"), col("sourceid_oms")))
ads = ads.withColumn("channel_id", func.coalesce(col("channel_id_omni"), col("channel_id_oms")))
ads = ads.withColumn("order_no", func.coalesce(col("order_no_omni"), col("order_no_oms")))
ads = ads.withColumn("is_mobile", func.coalesce(col("is_mobile_omni"), col("is_mobile_oms")))
ads = ads.withColumn("f_nas", func.coalesce(col("f_nas_omni"), col("f_nas_oms")))

# for oms - will just assume the seller is walmart - since there is no seller indicator
ads = ads.withColumn("sellerid_ad", func.coalesce(col("sellerid_ad_omni"), col("sellerid_ad_oms")))

# make sure the NAS adid has the same sellerid+cat_id as in PCOC, keep OAS sellerid+cat_id as extracted from page_url
PCOC = spark.sql('select distinct adid, catalog_item_id, us_seller_id from sem.pla_catalog_offer_criteria_shopping')
ads_nas = ads.filter(ads.f_nas == 'nas')
ads_oas = ads.filter(ads.f_nas == 'oas')
ads_nas = ads_nas.join(PCOC, (ads_nas.adid == PCOC.adid),
                       how='left').drop(PCOC.adid)
ads_nas = ads_nas.withColumn("catalog_item_id_ad",
                             func.coalesce(col("catalog_item_id"), col("catalog_item_id_ad")))
ads_nas = ads_nas.withColumn("sellerid_ad", func.coalesce(col("us_seller_id"), col("sellerid_ad")))
ads_nas = ads_nas.drop('catalog_item_id').drop('us_seller_id')
ads_nas = ads_nas.select(ads_oas.columns)
ads = ads_nas.union(ads_oas)


# 4. COIM: get line items + GMV per order listed in the visit chain
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# sqlCmd = """SELECT fulfmt_order_line_nbr as order_line_nbr, txn_nbr as sales_order_grp_num,
#                     sum(gmv_plcd_amt) as gmv_plcd_amt,
#                     sum(gmv_order_amt) as gmv_order_amt,
#                     max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
#                     max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
#                     max(slr_org_cd) as slr_org_cd_purchase,
#                     max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
#              FROM """ + us_secure_dl_transactions_wmt_cust_scan_dotcom_l2_cbb_catalog + """
#               WHERE gmv_ind = 1
#                    AND gmv_cr_ind = 1
#                    AND txn_nbr IS NOT NULL
#                    AND trim(txn_nbr) != ''
#                    AND txn_dt = '""" + ds + """'
#              GROUP BY txn_nbr, fulfmt_order_line_nbr"""

sqlCmd =  """SELECT order_line_nbr, order_nbr as sales_order_grp_num,
                    sum(gmv_plcd_amt) as gmv_plcd_amt,
                    sum(gmv_order_amt) as gmv_order_amt,
                    max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
                    max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
                    max(slr_org_cd) as slr_org_cd_purchase,
                    max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
             FROM """+ww_crew_dl_rpt_vm_cnsld_order_item+"""
             WHERE op_cmpny_cd = 'WMT.COM'
                   AND GMV_IND = 1
                   AND gmv_cr_ind = 1
                   AND order_nbr IS NOT NULL
                   AND trim(order_nbr) != ''
                   and order_nbr = '200010802154653'
                   AND order_plcd_dt = '"""+ds+"""'
             GROUP BY order_nbr, order_line_nbr"""

transact = spark.sql(sqlCmd)

ad_trans_pairs = ads.join(transact,
                          (ads.order_no == transact.sales_order_grp_num),
                          how='inner').drop(transact.sales_order_grp_num)

# 3. Get the brand name for the ads
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x: x).map(
    lambda x: x.replace("ts=", "")).max()

item_info_dfm = spark.sql("""SELECT item_id as catlg_item_id,
                                    max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
            FROM dmas.dfm
            WHERE ts='""" + partitionDate + """'
            GROUP BY item_id""")

ad_trans_pairs = ad_trans_pairs.join(item_info_dfm,
                                     (ad_trans_pairs.catalog_item_id_ad == item_info_dfm.catlg_item_id),
                                     how='left').drop(item_info_dfm.catlg_item_id)

# 3. Get the coop event id when relevant for OAS and NAS
# For OAS, we grab coop event id for each item for bing pla
# For NAS, we grab coop event id ffor each offer for google pla
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
coop_items_oas = spark.sql("""SELECT catalog_item_id as catalog_item_id_coop,
                                 max(coop_event_id) as coop_event_id
                            FROM casesci_sem.coop_split_ad_links_v2
                            WHERE date_string = '{ds}'
                                AND is_assigned=1
                            AND (source_id=4 or seller_id is NULL)
                            GROUP BY catalog_item_id
                            """.format(ds=ds))

coop_items_nas = spark.sql("""SELECT catalog_item_id as catalog_item_id_coop, seller_id as seller_id_coop,
                                 max(coop_event_id) as coop_event_id
                         FROM casesci_sem.coop_split_ad_links_v2
                            WHERE date_string = '{ds}'
                            AND is_assigned = 1
                            AND is_pla = 1
                            AND source_id=2
                            AND seller_id is NOT NULL
                            GROUP BY catalog_item_id, seller_id""".format(ds=ds))

# append coop event id for ad click
coop_items_ad_oas = coop_items_oas.select(col('catalog_item_id_coop'),
                                          col('coop_event_id').alias('coop_event_id_ad_oas'))

coop_items_ad_nas = coop_items_nas.select(col('catalog_item_id_coop'), col('seller_id_coop'),
                                          col('coop_event_id').alias('coop_event_id_ad_nas'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_ad_oas,
                                     (ad_trans_pairs.catalog_item_id_ad == coop_items_ad_oas.catalog_item_id_coop),
                                     how='left').drop(coop_items_ad_oas.catalog_item_id_coop)

ad_trans_pairs = ad_trans_pairs.join(coop_items_ad_nas,
                                     [ad_trans_pairs.catalog_item_id_ad == coop_items_ad_nas.catalog_item_id_coop,
                                      ad_trans_pairs.sellerid_ad == coop_items_ad_nas.seller_id_coop],
                                     how='left').drop(coop_items_ad_nas.catalog_item_id_coop).drop(
    coop_items_ad_nas.seller_id_coop)

ad_trans_pairs = ad_trans_pairs.withColumn("coop_event_id_ad", func.coalesce(col("coop_event_id_ad_nas"),
                                                                             col("coop_event_id_ad_oas"))).drop(
    'coop_event_id_ad_nas').drop('coop_event_id_ad_oas')

# append coop event id for purchase
coop_items_purchase_oas = coop_items_oas.select(col('catalog_item_id_coop'),
                                                col('coop_event_id').alias('coop_event_id_purchase_oas'))

coop_items_purchase_nas = coop_items_nas.select(col('catalog_item_id_coop'), col('seller_id_coop'),
                                                col('coop_event_id').alias('coop_event_id_purchase_nas'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_purchase_oas,
                                     (
                                                 ad_trans_pairs.catlg_item_id_purchase == coop_items_ad_oas.catalog_item_id_coop),
                                     how='left').drop(coop_items_purchase_oas.catalog_item_id_coop)

ad_trans_pairs = ad_trans_pairs.join(coop_items_purchase_nas,
                                     [
                                         ad_trans_pairs.catlg_item_id_purchase == coop_items_purchase_nas.catalog_item_id_coop,
                                         ad_trans_pairs.slr_org_cd_purchase == coop_items_purchase_nas.seller_id_coop],
                                     how='left').drop(coop_items_purchase_nas.catalog_item_id_coop).drop(
    coop_items_purchase_nas.seller_id_coop)

ad_trans_pairs = ad_trans_pairs.withColumn("coop_event_id_purchase",
                                           func.coalesce(col("coop_event_id_purchase_nas"),
                                                         col("coop_event_id_purchase_oas"))).drop(
    'coop_event_id_purchase_nas').drop('coop_event_id_purchase_oas')

######################################################################################################

# 5. Mark which line items are the same advertiser and the precise same item as the adid (featured item)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

ad_trans_pairs = ad_trans_pairs.withColumn("same_item",
                                           func.when(col("catlg_item_id_purchase") == col("catalog_item_id_ad"),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_brand", func.when(col("purchase_brand") == col("ad_item_brand"),
                                                                   func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_coop",
                                           func.when((col("coop_event_id_ad") == col("coop_event_id_purchase"))
                                                     & col('coop_event_id_ad').isNotNull(),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller",
                                           func.when(col("slr_org_cd_purchase") == col("sellerid_ad"),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_item",
                                           func.when((col("same_seller") == 1) & (col("same_item") == 1),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_brand",
                                           func.when((col("same_seller") == 1) & (col("same_brand") == 1),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_coop",
                                           func.when((col("same_seller") == 1) & (col("same_coop") == 1),
                                                     func.lit(1)).otherwise(func.lit(0)))

# filter out irrelevant items

ad_trans_pairs_2 = ad_trans_pairs.filter((col('same_item') +
                                        col('same_brand') +
                                        col('same_coop') +
                                        col('same_seller')) > 0).toPandas()

In [97]:
ad_trans_pairs_2[ad_trans_pairs_2['source_id']==4]

,order_no_oms,order_date_oms,channel_id_oms,ad_date_oms,f_nas_oms,adid_oms,catalog_item_id_ad_oms,sellerid_ad_oms,is_pla_oms,sourceid_oms,is_target_all_oms,is_mobile_oms,order_no_omni,page_url,visit_hit_time_gmt,order_date_omni,adid_omni,channel_id_omni,catalog_item_id_ad_omni,sellerid_ad_omni,f_nas_omni,is_target_all_omni,is_pla_omni,sourceid_omni,is_mobile_omni,ad_date_omni,datediff_browseorder,is_pla,is_target_all,ad_date,order_date,adid,catalog_item_id_ad,source_id,channel_id,order_no,is_mobile,f_nas,sellerid_ad,order_line_nbr,gmv_plcd_amt,gmv_order_amt,purchase_brand,catlg_item_id_purchase,slr_org_cd_purchase,svc_fulfmt_type_desc,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,same_item,same_brand,same_coop,same_seller,same_seller_item,same_seller_brand,same_seller_coop
4,None,None,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,997083873,0,oas,0,1,4,0,2023-03-26,0,1,0,2023-03-26,2023-03-26,404313442,997083873,4,22,200010802154653,0,oas,0,2,18.920000000,18.920000000,gerber,451560646,0,UNSCHEDULED_DELIVERY,deltachildren,2474,None,0,0,0,1,0,0,0
9,None,None,None,None,None,None,None,None,None,None,None,None,200010802154653,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,22,997083873,0,oas,0,1,4,0,2023-03-26,0,1,0,2023-03-26,2023-03-26,404313442,997083873,4,22,200010802154653,0,oas,0,3,6.640000000,6.640000000,littlestarorganic,142918905,0,UNSCHEDULED_DELIVERY,deltachildren,2474,None,0,0,0,1,0,0,0


In [100]:
ad_trans_pairs[ad_trans_pairs['source_id']==4]

DataFrame[order_no_oms: string, order_date_oms: string, reflector_id: string, adid_oms: string, channel_id_oms: string, ad_date_oms: date, is_pla_oms: int, sourceid_oms: int, is_target_all_oms: int, catalog_item_id_ad_oms: string, order_no_omni: string, page_url: string, visit_hit_time_gmt: bigint, order_date_omni: string, adid_omni: string, channel_id_omni: string, sellerid_ad: string, catalog_item_id_ad_omni: string, is_target_all_omni: int, is_pla_omni: int, sourceid_omni: int, ad_date_omni: string, datediff_browseorder: int, is_pla: int, is_target_all: int, ad_date: string, order_date: string, adid: string, catalog_item_id_ad: string, source_id: int, channel_id: string, order_no: string, order_line_nbr: bigint, gmv_plcd_amt: decimal(38,9), gmv_order_amt: decimal(38,9), purchase_brand: string, catlg_item_id_purchase: string, slr_org_cd_purchase: string, svc_fulfmt_type_desc: string, ad_item_brand: string, coop_event_id_ad: string, coop_event_id_purchase: string, same_item: int, same

In [84]:
pt1_v1[(pt1_v1['purchase_brand']==pt1_v1['ad_item_brand'])&(pt1_v1['source_id']==4)]

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_nm,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,order_date
1,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,997083873,997083873,UNSCHEDULED_DELIVERY,1,deltachildren,deltachildren,2474,2474,0,0,1,200010802154653,170.840000000,170.840000000,4,22,1,0,1,1,1,1,1,1,2,2023-03-26


In [86]:
pt1_v1

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_nm,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,order_date
0,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,451560646,997083873,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,2474,None,0,0,2,200010802154653,9.460000000,9.460000000,4,22,1,0,0,0,0,0,0,0,2,2023-03-26
1,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,997083873,997083873,UNSCHEDULED_DELIVERY,1,deltachildren,deltachildren,2474,2474,0,0,1,200010802154653,170.840000000,170.840000000,4,22,1,0,1,1,1,1,1,1,2,2023-03-26
2,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,142918905,997083873,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,2474,None,0,0,3,200010802154653,6.640000000,6.640000000,4,22,1,0,0,0,0,0,0,0,2,2023-03-26


In [85]:
pt1_v2[(pt1_v2['purchase_brand']==pt1_v2['ad_item_brand'])&(pt1_v2['source_id']==4)]

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_nm,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,is_mobile,f_nas,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,same_item,same_brand,same_coop,same_seller,same_seller_item,same_seller_brand,same_seller_coop,order_date
2,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,997083873,997083873,UNSCHEDULED_DELIVERY,1,deltachildren,deltachildren,2474,2474,0,0,1,200010802154653,170.840000000,170.840000000,4,22,1,0,0,oas,1,4,1,1,4,1,5,1,1,1,1,1,1,1,2023-03-26


In [87]:
pt1_v2

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_nm,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,is_mobile,f_nas,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,same_item,same_brand,same_coop,same_seller,same_seller_item,same_seller_brand,same_seller_coop,order_date
0,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,142918905,997083873,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,2474,None,0,0,3,200010802154653,6.640000000,6.640000000,4,22,1,0,0,oas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
1,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,451560646,997083873,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,2474,None,0,0,2,200010802154653,9.460000000,9.460000000,4,22,1,0,0,oas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
2,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,997083873,997083873,UNSCHEDULED_DELIVERY,1,deltachildren,deltachildren,2474,2474,0,0,1,200010802154653,170.840000000,170.840000000,4,22,1,0,0,oas,1,4,1,1,4,1,5,1,1,1,1,1,1,1,2023-03-26


In [121]:
# v2 pipeline



sqlCmd = """SELECT order_no_omni,page_url,visit_hit_time_gmt, order_date_omni, adid_omni, channel_id_omni,
if(adid RLIKE '{regex_pla}', split(adid_omni,'_')[0], split(catalog_item_id_ad_omni_old,'_')[0]) as catalog_item_id_ad_omni,
coalesce(if(adid RLIKE '{regex_pla}', split(adid_omni,'_')[1], sellerid_ad_old), '0') as sellerid_ad_omni,
if(adid RLIKE '{regex_lia}' or adid RLIKE '{regex_pla}', 'nas', 'oas') as f_nas_omni
FROM (
SELECT order_no as order_no_omni, page_url, visit_hit_time_gmt, ds as order_date_omni,adid,
if(adid RLIKE '{regex_pla}', {extract_adid_lia}, if(adid RLIKE '{regex_lia}', {extract_adid_lia}, if(adid RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_reflector}))) AS adid_omni,
if(adid RLIKE '{regex_pla}', {channel_id_plalia}, if(adid RLIKE '{regex_lia}', {channel_id_plalia}, if(adid RLIKE c.reflector_identifier, {channel_id_plalia}, {channel_id_reflector}))) AS channel_id_omni,
coalesce(parse_url(lower(page_url), 'QUERY','selectedsellerid'), '0') sellerid_ad_old,
coalesce(parse_url(page_url, 'QUERY','wl12'), '') catalog_item_id_ad_omni_old
   FROM {fair_daily_auth_order_to_visit_chain},
   (
            SELECT id customer_id,
                   reflector_identifier,
                   channel_id_position,
                   adid_length
            FROM {sem_customers} c where id=1
    ) c
   WHERE visit_vehicle='labs sem'
       AND ds = '{ds}'
       and adid is not null
       AND length(coalesce(parse_url(page_url, 'QUERY','wl12'), ''))>2)
       where order_no_omni = '200010802154653'""".format(ds=ds,
                                  sem_customers=sem_customers,
                                  fair_daily_auth_order_to_visit_chain=fair_daily_auth_order_to_visit_chain,
                                  regex_lia=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                  regex_pla=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+_[0-9]+$',
                                  extract_adid_lia=r'substring(adid, channel_id_position+2)',
                                  extract_adid_pla=r'cast(cast(substring(adid, channel_id_position+2, adid_length) AS int) AS string)',
                                  extract_adid_reflector=r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                  channel_id_plalia=r'cast(cast(substring(adid, channel_id_position, 2) AS int) AS string)',
                                  channel_id_reflector=r'cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string)'
                                  )
visit_chain = spark.sql(sqlCmd)
# determine if ad is_target_all
visit_chain = visit_chain.withColumn("is_target_all_omni", func.when((col("adid_omni") == '0'), func.lit(1)
                                                                     ).otherwise(func.lit(0)))

# filter out to only have pla
visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([81, 82, 27, 28, 22, 23]), func.lit(1)
                                               ).otherwise(func.lit(0)))

visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([54, 55, 64, 65]), func.lit(2)
                                               ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.withColumn("is_pla_omni",
                                     func.when(col("channel_id_omni").isin([37, 38, 77, 78]), func.lit(4)
                                               ).otherwise(col('is_pla_omni')))

visit_chain = visit_chain.filter(col('is_pla_omni') > 0)

# get the sourceid from channel id google = 2 bing =4, other = 6
visit_chain = visit_chain.withColumn("sourceid_omni", func.lit(6))
visit_chain = visit_chain.withColumn("sourceid_omni",
                                     func.when(col("channel_id_omni").isin([20, 21, 27, 28, 37, 38, 77, 78]),
                                               func.lit(2)
                                               ).otherwise(col('sourceid_omni')))
visit_chain = visit_chain.withColumn("sourceid_omni",
                                     func.when(col("channel_id_omni").isin([22, 23, 24, 25, 64, 65]), func.lit(4)
                                               ).otherwise(col('sourceid_omni')))

visit_chain = visit_chain.withColumn('catalog_item_id_ad_omni',
                                     func.when(col('sourceid_omni') == 4,
                                               func.split(col('catalog_item_id_ad_omni'), '_')[0]).otherwise(
                                         col('catalog_item_id_ad_omni')))
# get is_mobile from channel id
visit_chain = visit_chain.withColumn("is_mobile_omni", func.when(
    col("channel_id_omni").isin([21, 23, 25, 28, 81, 85, 55, 65, 52, 38, 78]), func.lit(1)
    ).otherwise(lit(0)))

# 2. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds) or 8 hours depending on whether there is daylight savings
# Zhiyu Xie confirms that all ads in the visit_chain are before orders on the seconds level
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
datetime_pst = pytz.timezone('US/Pacific').localize(datetime.strptime(ds, "%Y-%m-%d"))
is_dst_val = is_dst(datetime_pst)
if (is_dst_val):
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format(
        (visit_chain.visit_hit_time_gmt - 25200).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))
else:
    visit_chain = visit_chain.withColumn('ad_date_omni', func.date_format(
        (visit_chain.visit_hit_time_gmt - 28800).cast(dataType=t.TimestampType()), "yyyy-MM-dd"))

visit_chain = visit_chain.withColumn("datediff_browseorder",
                                     func.datediff(col("order_date_omni"), col("ad_date_omni")))
visit_chain = visit_chain.filter(col('datediff_browseorder') < NUM_ATTRIBUTION_DAYS)

# 3. now get only OMS
oms_auth_order = spark.sql("""select order_no_oms, order_date_oms, channel_id_oms, ad_date_oms, f_nas_oms, adid_oms,
        if(reflector_id RLIKE '{regex_lia_nas}' or reflector_id RLIKE '{regex_pla_nas}', split(adid_oms,'_')[0], NULL) as catalog_item_id_ad_oms,
        if(reflector_id RLIKE '{regex_pla_nas}', split(adid_oms,'_')[1], '0') as sellerid_ad_oms
        FROM (
            SELECT order_no as order_no_oms, ds as order_date_oms, reflector_id, 
            if(reflector_id RLIKE '{regex_lia_nas}' or reflector_id RLIKE '{regex_pla_nas}', 'nas', 'oas') as f_nas_oms,
            if(reflector_id RLIKE '{regex_pla_nas}', {extract_adid_nas}, if(reflector_id RLIKE '{regex_lia_nas}', {extract_adid_nas}, if(reflector_id RLIKE c.reflector_identifier, {extract_adid_pla}, {extract_adid_lia}))) AS adid_oms,
           cast(cast(substring(reflector_id, channel_id_position, 2) AS int) AS string) AS channel_id_oms,
           to_date(reflector_created_at) as ad_date_oms
        FROM {fair_daily_auth_order_to_ref_vehicle},
           (
                    SELECT id customer_id,
                           reflector_identifier,
                           channel_id_position,
                           adid_length
                    FROM {sem_customers} c where id=1
            ) c
    WHERE ds = '{ds}')
    where order_no_oms = '200010802154653'""".format(fair_daily_auth_order_to_ref_vehicle=fair_daily_auth_order_to_ref_vehicle,
                                 ds=ds,
                                 sem_customers=sem_customers,
                                 regex_lia_nas=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+$',
                                 regex_pla_nas=r'^2{7}22[0-9]{2}[0-9]+_[0-9]+_[0-9]+_[0-9]+$',
                                 extract_adid_lia=r'substring(reflector_id, channel_id_position+2)',
                                 extract_adid_pla=r'cast(cast(substring(reflector_id, channel_id_position+2, adid_length) AS int) AS string)',
                                 extract_adid_nas=r'substring(reflector_id, channel_id_position+2)',
                                 ))

# add is_pla and sourceid
oms_auth_order = oms_auth_order.drop_duplicates(['order_no_oms'])

oms_auth_order = oms_auth_order.withColumn("is_pla_oms",
                                           func.when(col("channel_id_oms").isin([81, 82, 27, 28, 22, 23]),
                                                     func.lit(1)
                                                     ).otherwise(func.lit(0)))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms",
                                           func.when(col("channel_id_oms").isin([54, 55, 64, 65]), func.lit(2)
                                                     ).otherwise(col('is_pla_oms')))

oms_auth_order = oms_auth_order.withColumn("is_pla_oms",
                                           func.when(col("channel_id_oms").isin([37, 38, 77, 78]), func.lit(4)
                                                     ).otherwise(col('is_pla_oms')))

# get the sourceid from channel id google = 2 bing =4, other = 6
oms_auth_order = oms_auth_order.withColumn("sourceid_oms", func.lit(6))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms",
                                           func.when(col("channel_id_oms").isin([20, 21, 27, 28, 37, 38, 77, 78]),
                                                     func.lit(2)
                                                     ).otherwise(col('sourceid_oms')))
oms_auth_order = oms_auth_order.withColumn("sourceid_oms",
                                           func.when(col("channel_id_oms").isin([22, 23, 24, 25, 64, 65]),
                                                     func.lit(4)
                                                     ).otherwise(col('sourceid_oms')))

oms_auth_order = oms_auth_order.withColumn("is_target_all_oms", func.when((col("adid_oms") == '0'), func.lit(1)
                                                                          ).otherwise(func.lit(0)))
# get is_mobile from channel id
oms_auth_order = oms_auth_order.withColumn("is_mobile_oms", func.when(
    col("channel_id_oms").isin([21, 23, 25, 28, 81, 85, 55, 65, 52, 38, 78]), func.lit(1)
    ).otherwise(lit(0)))

# get the catalog_item_id from the adid for oas
adid2cat = spark.sql("""SELECT
                            adid, max(catalog_item_id_ad_oms) as catalog_item_id_ad_oms_dotcom
                        FROM
                            ( SELECT CAST(id AS INT) as adid,
                                     catalog_item_id as catalog_item_id_ad_oms
                              FROM sem.wmt_dotcom_items )
                        GROUP BY adid""")

oms_auth_order = oms_auth_order.join(adid2cat,
                                     (oms_auth_order.adid_oms == adid2cat.adid),
                                     how='left').drop(adid2cat.adid)
oms_auth_order = oms_auth_order.withColumn("catalog_item_id_ad_oms", func.coalesce(col("catalog_item_id_ad_oms"),
                                                                                   col("catalog_item_id_ad_oms_dotcom"))).drop(
    'catalog_item_id_ad_oms_dotcom')

# first filter out the omni orders to only have orders that also appear in oms
# supposedly if an order doesn't appear in oms - then it isn't valid/authorized
valid_orders = oms_auth_order.select(col("order_no_oms").alias("valid_order"))
valid_orders = valid_orders.drop_duplicates()
visit_chain = valid_orders.join(visit_chain,
                                (valid_orders.valid_order == visit_chain.order_no_omni),
                                how='left').drop(valid_orders.valid_order)

# now join together oms + visit_chain
ads = oms_auth_order.join(visit_chain,
                          ((oms_auth_order.order_no_oms == visit_chain.order_no_omni) &
                           # (oms_auth_order.catalog_item_id_ad_oms == visit_chain.catalog_item_id_ad_omni) &
                           # (oms_auth_order.sellerid_ad_oms == visit_chain.sellerid_ad_omni) &
                           (oms_auth_order.channel_id_oms == visit_chain.channel_id_omni) &
                           (oms_auth_order.adid_oms == visit_chain.adid_omni) &
                           (oms_auth_order.order_date_oms == visit_chain.order_date_omni)),
                          how='outer')

# filter out to only have pla/dsa
ads = ads.withColumn("is_pla", func.coalesce(col("is_pla_omni"), col("is_pla_oms")))
ads = ads.filter(col('is_pla') > 0)

ads = ads.withColumn("is_target_all", func.coalesce(col("is_target_all_omni"), col("is_target_all_oms")))

# 3. Remove ads that are associated with an order more than 14 days later
# ds is PST so need to gmt to pst by subtracting 7 hours (25200 seconds)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ads = ads.withColumn("ad_date", func.coalesce(col("ad_date_omni"), col("ad_date_oms")))
ads = ads.withColumn("order_date", func.coalesce(col("order_date_omni"), col("order_date_oms")))
ads = ads.withColumn("datediff_browseorder", func.datediff(col("order_date"), col("ad_date")))
ads = ads.filter(col('datediff_browseorder') < NUM_ATTRIBUTION_DAYS)

# some coalescing that needs to be done after connecting oms and fair
ads = ads.withColumn("adid", func.coalesce(col("adid_omni"), col("adid_oms")))
ads = ads.withColumn("catalog_item_id_ad",
                     func.coalesce(col("catalog_item_id_ad_omni"), col("catalog_item_id_ad_oms")))

ads = ads.withColumn("source_id", func.coalesce(col("sourceid_omni"), col("sourceid_oms")))
ads = ads.withColumn("channel_id", func.coalesce(col("channel_id_omni"), col("channel_id_oms")))
ads = ads.withColumn("order_no", func.coalesce(col("order_no_omni"), col("order_no_oms")))
ads = ads.withColumn("is_mobile", func.coalesce(col("is_mobile_omni"), col("is_mobile_oms")))
ads = ads.withColumn("f_nas", func.coalesce(col("f_nas_omni"), col("f_nas_oms")))

# for oms - will just assume the seller is walmart - since there is no seller indicator
ads = ads.withColumn("sellerid_ad", func.coalesce(col("sellerid_ad_omni"), col("sellerid_ad_oms")))

# make sure the NAS adid has the same sellerid+cat_id as in PCOC, keep OAS sellerid+cat_id as extracted from page_url
PCOC = spark.sql('select distinct adid, catalog_item_id, us_seller_id from sem.pla_catalog_offer_criteria_shopping')
ads_nas = ads.filter(ads.f_nas == 'nas')
ads_oas = ads.filter(ads.f_nas == 'oas')
ads_nas = ads_nas.join(PCOC, (ads_nas.adid == PCOC.adid),
                       how='left').drop(PCOC.adid)
ads_nas = ads_nas.withColumn("catalog_item_id_ad",
                             func.coalesce(col("catalog_item_id"), col("catalog_item_id_ad")))
ads_nas = ads_nas.withColumn("sellerid_ad", func.coalesce(col("us_seller_id"), col("sellerid_ad")))
ads_nas = ads_nas.drop('catalog_item_id').drop('us_seller_id')
ads_nas = ads_nas.select(ads_oas.columns)
ads = ads_nas.union(ads_oas)


# 4. COIM: get line items + GMV per order listed in the visit chain
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# sqlCmd = """SELECT fulfmt_order_line_nbr as order_line_nbr, txn_nbr as sales_order_grp_num,
#                     sum(gmv_plcd_amt) as gmv_plcd_amt,
#                     sum(gmv_order_amt) as gmv_order_amt,
#                     max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
#                     max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
#                     max(slr_org_cd) as slr_org_cd_purchase,
#                     max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
#              FROM """ + us_secure_dl_transactions_wmt_cust_scan_dotcom_l2_cbb_catalog + """
#               WHERE gmv_ind = 1
#                    AND gmv_cr_ind = 1
#                    AND txn_nbr IS NOT NULL
#                    AND trim(txn_nbr) != ''
#                    AND txn_dt = '""" + ds + """'
#              GROUP BY txn_nbr, fulfmt_order_line_nbr"""

sqlCmd =  """SELECT order_line_nbr, order_nbr as sales_order_grp_num,
                    sum(gmv_plcd_amt) as gmv_plcd_amt,
                    sum(gmv_order_amt) as gmv_order_amt,
                    max(lower(REGEXP_REPLACE(brand_nm, '[^0-9A-Za-z]', ''))) as purchase_brand,
                    max(cast(catlg_item_id AS STRING)) as catlg_item_id_purchase,
                    max(slr_org_cd) as slr_org_cd_purchase,
                    max(svc_fulfmt_type_desc) as svc_fulfmt_type_desc
             FROM """+ww_crew_dl_rpt_vm_cnsld_order_item+"""
             WHERE op_cmpny_cd = 'WMT.COM'
                   AND GMV_IND = 1
                   AND gmv_cr_ind = 1
                   AND order_nbr IS NOT NULL
                   AND trim(order_nbr) != ''
                   and order_nbr = '200010802154653'
                   AND order_plcd_dt = '"""+ds+"""'
             GROUP BY order_nbr, order_line_nbr"""

transact = spark.sql(sqlCmd)

ad_trans_pairs = ads.join(transact,
                          (ads.order_no == transact.sales_order_grp_num),
                          how='inner').drop(transact.sales_order_grp_num)

# 3. Get the brand name for the ads
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x: x).map(
    lambda x: x.replace("ts=", "")).max()

item_info_dfm = spark.sql("""SELECT item_id as catlg_item_id,
                                    max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
            FROM dmas.dfm
            WHERE ts='""" + partitionDate + """'
            GROUP BY item_id""")

ad_trans_pairs = ad_trans_pairs.join(item_info_dfm,
                                     (ad_trans_pairs.catalog_item_id_ad == item_info_dfm.catlg_item_id),
                                     how='left').drop(item_info_dfm.catlg_item_id)

# 3. Get the coop event id when relevant for OAS and NAS
# For OAS, we grab coop event id for each item for bing pla
# For NAS, we grab coop event id ffor each offer for google pla
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
coop_items_oas = spark.sql("""SELECT catalog_item_id as catalog_item_id_coop,
                                 max(coop_event_id) as coop_event_id
                            FROM casesci_sem.coop_split_ad_links_v2
                            WHERE date_string = '{ds}'
                                AND is_assigned=1
                            AND (source_id=4 or seller_id is NULL)
                            GROUP BY catalog_item_id
                            """.format(ds=ds))

coop_items_nas = spark.sql("""SELECT catalog_item_id as catalog_item_id_coop, seller_id as seller_id_coop,
                                 max(coop_event_id) as coop_event_id
                         FROM casesci_sem.coop_split_ad_links_v2
                            WHERE date_string = '{ds}'
                            AND is_assigned = 1
                            AND is_pla = 1
                            AND source_id=2
                            AND seller_id is NOT NULL
                            GROUP BY catalog_item_id, seller_id""".format(ds=ds))

# append coop event id for ad click
coop_items_ad_oas = coop_items_oas.select(col('catalog_item_id_coop'),
                                          col('coop_event_id').alias('coop_event_id_ad_oas'))

coop_items_ad_nas = coop_items_nas.select(col('catalog_item_id_coop'), col('seller_id_coop'),
                                          col('coop_event_id').alias('coop_event_id_ad_nas'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_ad_oas,
                                     (ad_trans_pairs.catalog_item_id_ad == coop_items_ad_oas.catalog_item_id_coop),
                                     how='left').drop(coop_items_ad_oas.catalog_item_id_coop)

ad_trans_pairs = ad_trans_pairs.join(coop_items_ad_nas,
                                     [ad_trans_pairs.catalog_item_id_ad == coop_items_ad_nas.catalog_item_id_coop,
                                      ad_trans_pairs.sellerid_ad == coop_items_ad_nas.seller_id_coop],
                                     how='left').drop(coop_items_ad_nas.catalog_item_id_coop).drop(
    coop_items_ad_nas.seller_id_coop)

ad_trans_pairs = ad_trans_pairs.withColumn("coop_event_id_ad", func.coalesce(col("coop_event_id_ad_nas"),
                                                                             col("coop_event_id_ad_oas"))).drop(
    'coop_event_id_ad_nas').drop('coop_event_id_ad_oas')

# append coop event id for purchase
coop_items_purchase_oas = coop_items_oas.select(col('catalog_item_id_coop'),
                                                col('coop_event_id').alias('coop_event_id_purchase_oas'))

coop_items_purchase_nas = coop_items_nas.select(col('catalog_item_id_coop'), col('seller_id_coop'),
                                                col('coop_event_id').alias('coop_event_id_purchase_nas'))

ad_trans_pairs = ad_trans_pairs.join(coop_items_purchase_oas,
                                     (
                                                 ad_trans_pairs.catlg_item_id_purchase == coop_items_ad_oas.catalog_item_id_coop),
                                     how='left').drop(coop_items_purchase_oas.catalog_item_id_coop)

ad_trans_pairs = ad_trans_pairs.join(coop_items_purchase_nas,
                                     [
                                         ad_trans_pairs.catlg_item_id_purchase == coop_items_purchase_nas.catalog_item_id_coop,
                                         ad_trans_pairs.slr_org_cd_purchase == coop_items_purchase_nas.seller_id_coop],
                                     how='left').drop(coop_items_purchase_nas.catalog_item_id_coop).drop(
    coop_items_purchase_nas.seller_id_coop)

ad_trans_pairs = ad_trans_pairs.withColumn("coop_event_id_purchase",
                                           func.coalesce(col("coop_event_id_purchase_nas"),
                                                         col("coop_event_id_purchase_oas"))).drop(
    'coop_event_id_purchase_nas').drop('coop_event_id_purchase_oas')

######################################################################################################

# 5. Mark which line items are the same advertiser and the precise same item as the adid (featured item)
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

ad_trans_pairs = ad_trans_pairs.withColumn("same_item",
                                           func.when(col("catlg_item_id_purchase") == col("catalog_item_id_ad"),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_brand", func.when(col("purchase_brand") == col("ad_item_brand"),
                                                                   func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_coop",
                                           func.when((col("coop_event_id_ad") == col("coop_event_id_purchase"))
                                                     & col('coop_event_id_ad').isNotNull(),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller",
                                           func.when(col("slr_org_cd_purchase") == col("sellerid_ad"),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_item",
                                           func.when((col("same_seller") == 1) & (col("same_item") == 1),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_brand",
                                           func.when((col("same_seller") == 1) & (col("same_brand") == 1),
                                                     func.lit(1)).otherwise(func.lit(0)))

ad_trans_pairs = ad_trans_pairs.withColumn("same_seller_coop",
                                           func.when((col("same_seller") == 1) & (col("same_coop") == 1),
                                                     func.lit(1)).otherwise(func.lit(0)))

# filter out irrelevant items

ad_trans_pairs = ad_trans_pairs.filter((col('same_item') +
                                        col('same_brand') +
                                        col('same_coop') +
                                        col('same_seller')) > 0)

# 6. Determine how many other ads can claim they influenced this purchase of this unit of item
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
lineitem_claimants = ad_trans_pairs.groupBy('order_no', 'order_line_nbr').agg(
    func.sum("same_item").alias('num_claimants_item'),
    func.sum("same_brand").alias('num_claimants_brand'),
    func.sum("same_coop").alias('num_claimants_coop'),
    func.sum("same_seller_item").alias('num_claimants_seller_item'),
    func.sum("same_seller_brand").alias('num_claimants_seller_brand'),
    func.sum("same_seller_coop").alias('num_claimants_seller_coop'),
    func.sum("same_seller").alias('num_claimants_seller')
)

for atttype in ['item', 'brand', 'coop', 'seller_item', 'seller_brand', 'seller_coop', 'seller']:
    lineitem_claimants_i = lineitem_claimants.select(col("order_line_nbr").alias("order_line_nbr_summary"),
                                                     col("order_no").alias("order_no_summary"),
                                                     col("num_claimants_" + atttype))

    ad_trans_pairs = ad_trans_pairs.join(lineitem_claimants_i, (
            (ad_trans_pairs.order_line_nbr == lineitem_claimants_i.order_line_nbr_summary) &
            (ad_trans_pairs.order_no == lineitem_claimants_i.order_no_summary) &
            (ad_trans_pairs['same_' + atttype] == 1)
    ),
                                         how='left').drop(lineitem_claimants_i.order_line_nbr_summary).drop(
        lineitem_claimants_i.order_no_summary)

ad_trans_pairs = ad_trans_pairs.fillna(0, subset=['num_claimants_item',
                                                  'num_claimants_brand',
                                                  'num_claimants_coop',
                                                  'num_claimants_seller_item',
                                                  'num_claimants_seller_brand',
                                                  'num_claimants_seller_coop',
                                                  'num_claimants_seller'
                                                  ])

# determine if purchase is 1p
ad_trans_pairs = ad_trans_pairs.withColumn("purchase_1p", func.when(col("slr_org_cd_purchase") == "0",
                                                                    func.lit(1)).otherwise(func.lit(0)))

# 8. Save to item-level table - by order date
# ~~~~~~~~~~~~~~~~~~~~
columns_raw_table = ['datediff_browseorder', 'page_url', 'visit_hit_time_gmt', 'ad_date', 'adid',
                     'catlg_item_id_purchase', 'catalog_item_id_ad',
                     'svc_fulfmt_type_desc', 'purchase_1p',
                     'purchase_brand', 'ad_item_brand',
                     'coop_event_id_ad', 'coop_event_id_purchase',
                     'slr_org_cd_purchase', 'sellerid_ad',
                     'order_line_nbr', 'order_no',
                     'gmv_plcd_amt', 'gmv_order_amt',
                     'source_id', 'channel_id', 'is_pla', 'is_target_all', 'is_mobile', 'f_nas',

                     'num_claimants_item', 'num_claimants_brand', 'num_claimants_coop',
                     'num_claimants_seller_item', 'num_claimants_seller_brand', 'num_claimants_seller_coop',
                     'num_claimants_seller',

                     'same_item', 'same_brand', 'same_coop', 'same_seller', 'same_seller_item',
                     'same_seller_brand', 'same_seller_coop',

                     'order_date']

coop_out = ad_trans_pairs.select(columns_raw_table)

In [122]:
ww = coop_out.toPandas()

In [123]:
ww

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_fulfmt_type_desc,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,is_mobile,f_nas,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,same_item,same_brand,same_coop,same_seller,same_seller_item,same_seller_brand,same_seller_coop,order_date
0,0,https://www.walmart.com/ip/Delta-Children-Case...,1679872739,2023-03-26,653389093_117755028669_12420145346,142918905,653389093,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,None,None,0,0,3,200010802154653,6.640000000,6.640000000,2,77,4,0,0,nas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
1,7,https://www.walmart.com/ip/Delta-Children-Haye...,1679283091,2023-03-19,1511381225_117755028669_12420145346,142918905,1511381225,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,None,None,0,0,3,200010802154653,6.640000000,6.640000000,2,77,4,0,0,nas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
2,3,https://www.walmart.com/ip/Delta-Children-Haye...,1679624348,2023-03-23,1511381225_117755028669_12420145346,142918905,1511381225,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,None,None,0,0,3,200010802154653,6.640000000,6.640000000,2,77,4,0,0,nas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
3,0,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,142918905,854547111,UNSCHEDULED_DELIVERY,1,littlestarorganic,davinci,None,None,0,0,3,200010802154653,6.640000000,6.640000000,2,27,1,1,0,oas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
4,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,142918905,997083873,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,2474,None,0,0,3,200010802154653,6.640000000,6.640000000,4,22,1,0,0,oas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
5,0,https://www.walmart.com/ip/Delta-Children-Case...,1679872739,2023-03-26,653389093_117755028669_12420145346,451560646,653389093,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,None,None,0,0,2,200010802154653,18.920000000,18.920000000,2,77,4,0,0,nas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
6,7,https://www.walmart.com/ip/Delta-Children-Haye...,1679283091,2023-03-19,1511381225_117755028669_12420145346,451560646,1511381225,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,None,None,0,0,2,200010802154653,18.920000000,18.920000000,2,77,4,0,0,nas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
7,3,https://www.walmart.com/ip/Delta-Children-Haye...,1679624348,2023-03-23,1511381225_117755028669_12420145346,451560646,1511381225,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,None,None,0,0,2,200010802154653,18.920000000,18.920000000,2,77,4,0,0,nas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
8,0,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,451560646,854547111,UNSCHEDULED_DELIVERY,1,gerber,davinci,None,None,0,0,2,200010802154653,18.920000000,18.920000000,2,27,1,1,0,oas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26
9,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,451560646,997083873,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,2474,None,0,0,2,200010802154653,18.920000000,18.920000000,4,22,1,0,0,oas,0,0,0,0,0,0,5,0,0,0,1,0,0,0,2023-03-26


In [124]:
qq

,datediff_browseorder,page_url,visit_hit_time_gmt,ad_date,adid,catlg_item_id_purchase,catalog_item_id_ad,svc_fulfmt_type_desc,purchase_1p,purchase_brand,ad_item_brand,coop_event_id_ad,coop_event_id_purchase,slr_org_cd_purchase,sellerid_ad,order_line_nbr,order_no,gmv_plcd_amt,gmv_order_amt,source_id,channel_id,is_pla,is_target_all,num_claimants_item,num_claimants_brand,num_claimants_coop,num_claimants_seller_item,num_claimants_seller_brand,num_claimants_seller_coop,num_claimants_seller,order_date
0,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,142918905,997083873,UNSCHEDULED_DELIVERY,1,littlestarorganic,deltachildren,2474,None,0,0,3,200010802154653,6.640000000,6.640000000,4,22,1,0,0,0,0,0,0,0,2,2023-03-26
1,0,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,142918905,854547111,UNSCHEDULED_DELIVERY,1,littlestarorganic,davinci,None,None,0,0,3,200010802154653,6.640000000,6.640000000,2,27,1,1,0,0,0,0,0,0,2,2023-03-26
2,0,https://www.walmart.com/ip/Delta-Children-Rich...,1679873036,2023-03-26,404313442,451560646,997083873,UNSCHEDULED_DELIVERY,1,gerber,deltachildren,2474,None,0,0,2,200010802154653,18.920000000,18.920000000,4,22,1,0,0,0,0,0,0,0,2,2023-03-26
3,0,https://www.walmart.com/ip/DaVinci-Grove-4-in-...,1679868223,2023-03-26,0,451560646,854547111,UNSCHEDULED_DELIVERY,1,gerber,davinci,None,None,0,0,2,200010802154653,18.920000000,18.920000000,2,27,1,1,0,0,0,0,0,0,2,2023-03-26


In [4]:
pt2_v1 = spark.sql("""select attributed_gmv_brand, sellerid_ad, catalog_item_id_revenue,sourceid_revenue 
from fair.pt2_14day_agg_gmv
where ad_date = '2023-03-20' and catalog_item_id_revenue='523117746'""").toPandas()
pt2_v1

,attributed_gmv_brand,sellerid_ad,catalog_item_id_revenue,sourceid_revenue
0,371.2000,0,523117746,2
1,349.0000,0,523117746,4


In [101]:
pt2_v2 = spark.sql("""select attributed_gmv_brand, adid, sellerid_ad, catalog_item_id_revenue,sourceid_revenue 
from fair.pt2_14day_agg_gmv_nas
where ad_date = '2023-03-20' and catalog_item_id_revenue='587182688'""").toPandas()
pt2_v2

,attributed_gmv_brand,adid,sellerid_ad,catalog_item_id_revenue,sourceid_revenue
0,3296.8571,349761835,0,587182688,2
1,1923.0000,349761835,0,587182688,2
2,82.4615,587182688_117755028669_12420145346,0,587182688,2
3,335.0000,587182688_117755028669_12420145346,0,587182688,2
4,1195.6923,349761835,0,587182688,2
5,0.0000,587182688_117755028669_12420145346,0,587182688,2
6,914.0000,349761835,0,587182688,4


In [24]:
914-536

378

In [77]:
v_join[(v_join['attributed_gmv_brand_v2']!=v_join['attributed_gmv_brand_v1'])&(v_join['attributed_gmv_brand_v1'].isnull()==False)&(v_join['attributed_gmv_brand_v2'].isnull()==False)&(v_join['source_id']==4)]

,catalog_item_id,source_id,attributed_gmv_brand_v1,attributed_gmv_brand_v2
74,132195807,4,0.0000,49.9900
314,587182688,4,536.0000,914.0000
358,963321253,4,0.0000,22.7500
562,437543077,4,0.0000,129.0000
613,50753947,4,0.0000,73.0000
985,523117746,4,349.0000,813.0000
1038,34621243,4,0.0000,22.4250
1072,408186523,4,0.0000,26.9600
1598,903393993,4,0.0000,35.9600
1863,454609278,4,0.0000,74.0000


# define functions to get sem, catalog, and site signals

## func to join catalog_item_id to dctr pla desktop ads

In [6]:
def join_dctr_item_id(today_ds):
    
    # this function is to retrieve the item sets that has traffic information in sem.daily_criteria_traffic_revenue in the time interval of
    # (today_ds - 139 days, today_ds - 7 days)
    # the features are extracted from (today_ds - 139 days, today_ds - 7 days), and the target values will be constructed based on (today_ds - 6days, today)
    # basicially predicting "NEXT WEEK" values (conversion rate, ordersize)
    # start from 2022-7-26 since data is only available after 0726 for v2 pla data
    partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x:x).map(lambda x : x.replace("ts=","")).max()
    
    query = spark.sql("""SELECT dctr.*,
                                coalesce(pcoc.catalog_item_id, catalog_item_id_dctr) as catalog_item_id,
                                coalesce(pcoc.us_seller_id, seller_id_dctr) as seller_id,
                                dmas.ad_item_brand brand_nm,
                                darm.division_id AS division_id,
                                darm.super_department_id AS super_department_id,
                                darm.department_id AS department_id,
                                darm.category_id AS category_id,
                                darm.sub_category_id AS sub_category_id,
                                item_dedup.id_unique_variant

                        FROM (SELECT DISTINCT adid, source_id, is_mobile, seller_id as seller_id_dctr, split(adid,'_')[0] as catalog_item_id_dctr
                              FROM sem.daily_criteria_traffic_revenue_v2_tmp
                              WHERE 
                                   customer_id_p = '1' 
                                   AND (source_id = 2 or source_id=4)
                                   AND adtype_p ='pla' 
                                   AND structure_version_p = 2
                                   AND revenue_source_p = 'fair15'
                                   AND date_string_p >= date_sub('__today__', 146) 
                                   AND date_string_p <= date_sub('__today__', 7)) dctr

                              LEFT JOIN sem.pla_catalog_offer_criteria_shopping pcoc
                              ON dctr.adid = pcoc.adid


                              LEFT JOIN(
                                        SELECT adid, last(division_id) as division_id,
                                        last(sub_category_id) as sub_category_id, last(super_department_id) as super_department_id,
                                        last(department_id) as department_id, last(category_id) as category_id
                                        FROM sem.daily_adid_rh_metrics_v2 
                                        WHERE customer_id_p=1
                                        AND structure_version_p = 2
                                        AND revenue_source_p = 'fair15'
                                        AND date_string >= date_sub('__today__', 146) 
                                        AND date_string <= date_sub('__today__', 7)
                                        GROUP BY adid
                                        ) darm
                            ON dctr.adid = darm.adid 

                              LEFT JOIN (SELECT item_id as catlg_item_id,
                                        max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
                                        FROM dmas.dfm
                                        WHERE ts='__partitionDate__'
                                        GROUP BY item_id) dmas
                              ON coalesce(pcoc.catalog_item_id, dctr.catalog_item_id_dctr) = dmas.catlg_item_id
                              
                              LEFT JOIN (SELECT item_id,id_unique_variant 
                                          FROM casesci_sem.items_w_dups_mapping_sem_offer_bidding
                                          WHERE date_string='latest') item_dedup
                              ON coalesce(pcoc.catalog_item_id, dctr.catalog_item_id_dctr) = item_dedup.item_id
                              """.replace('__today__', today_ds).replace('__partitionDate__', partitionDate))
    
    query.createOrReplaceTempView("dctr_pla")

In [8]:
def join_dctr_item_id_training(today_ds):
    
    # this function is to retrieve the item sets that has traffic information in sem.daily_criteria_traffic_revenue in the time interval of
    # (today_ds - 139 days, today_ds - 7 days)
    # the features are extracted from (today_ds - 139 days, today_ds - 7 days), and the target values will be constructed based on (today_ds - 6days, today)
    # basicially predicting "NEXT WEEK" values (conversion rate, ordersize)
    # start from 2022-7-26 since data is only available after 0726 for v2 pla data
    partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x:x).map(lambda x : x.replace("ts=","")).max()
    
    query = spark.sql("""SELECT dctr.*,
                                coalesce(pcoc.catalog_item_id, catalog_item_id_dctr) as catalog_item_id,
                                coalesce(pcoc.us_seller_id, seller_id_dctr) as seller_id,
                                dmas.ad_item_brand brand_nm,
                                darm.division_id AS division_id,
                                darm.super_department_id AS super_department_id,
                                darm.department_id AS department_id,
                                darm.category_id AS category_id,
                                darm.sub_category_id AS sub_category_id,
                                item_dedup.id_unique_variant

                        FROM (SELECT DISTINCT adid, source_id, is_mobile, seller_id as seller_id_dctr, split(adid,'_')[0] as catalog_item_id_dctr
                              FROM sem.daily_criteria_traffic_revenue_v2_tmp
                              WHERE 
                                   customer_id_p = '1' 
                                   AND (source_id = 2 or source_id=4)
                                   AND adtype_p ='pla' 
                                   AND structure_version_p = 2
                                   AND revenue_source_p = 'fair15'
                                   AND date_string_p >= date_sub('__today__', 27) 
                                   AND date_string_p <= date_sub('__today__', 0)
                                   ) dctr
                                

                              LEFT JOIN sem.pla_catalog_offer_criteria_shopping pcoc
                              ON dctr.adid = pcoc.adid


                              LEFT JOIN(
                                        SELECT adid, last(division_id) as division_id,
                                        last(sub_category_id) as sub_category_id, last(super_department_id) as super_department_id,
                                        last(department_id) as department_id, last(category_id) as category_id
                                        FROM sem.daily_adid_rh_metrics_v2 
                                        WHERE customer_id_p=1
                                        AND structure_version_p = 2
                                        AND revenue_source_p = 'fair15'
                                        AND date_string >= date_sub('__today__', 27) 
                                        AND date_string <= date_sub('__today__', 0)
                                        GROUP BY adid
                                        ) darm
                            ON dctr.adid = darm.adid 

                              LEFT JOIN (SELECT item_id as catlg_item_id,
                                        max(lower(REGEXP_REPLACE(item_brand_name, '[^0-9A-Za-z]', ''))) as ad_item_brand
                                        FROM dmas.dfm
                                        WHERE ts='__partitionDate__'
                                        GROUP BY item_id) dmas
                              ON coalesce(pcoc.catalog_item_id, dctr.catalog_item_id_dctr) = dmas.catlg_item_id
                              
                              LEFT JOIN (SELECT item_id,id_unique_variant 
                                          FROM casesci_sem.items_w_dups_mapping_sem_offer_bidding
                                          WHERE date_string='latest') item_dedup
                              ON coalesce(pcoc.catalog_item_id, dctr.catalog_item_id_dctr) = item_dedup.item_id
                              """.replace('__today__', today_ds).replace('__partitionDate__', partitionDate))
    
    query.createOrReplaceTempView("dctr_pla")

## func for sem signals

In [6]:
def get_semSignalCoop(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # this function is to retrieve the "SEM" signals of each item/ad
    # SEM signals means the performance of an item/ad in SEM channel (Search Engine Marketing)
    # signals are extracted from different time intervals as specified in the json file - 'output_sem_performance_prediction_config_train_config_20201227.json'
    
    query = """WITH             
        dctr
        AS
        (SELECT dctr_v2.adid,
                dctr_v2.source_id,
                dctr_v2.is_mobile,
                SUM(impressions) AS impressions,
                SUM(clicks) AS clicks,
                SUM(adspend) AS adspend,
                SUM(qty) AS qty,
                MAX(att_lineitems_brand) AS orders_brand,
                MAX(att_lineitems_seller) AS orders_seller,
                MAX(attributed_gmv_brand) AS gmv_revenue_adjustment_brand,
                MAX(attributed_gmv_seller) AS gmv_revenue_adjustment_seller,
                -- SUM(orders) AS orders,
                -- SUM(gmv_revenue_adjustment) AS gmv_revenue_adjustment,
                SUM(cost) AS cost,
                SUM(estimated_cp) AS estimated_cp

        FROM (SELECT * FROM sem.daily_criteria_traffic_revenue_v2_tmp
               WHERE customer_id_p = '1' 
               AND (source_id = 2 or source_id=4)
               AND adtype_p ='pla' 
               AND structure_version_p = 2
               AND revenue_source_p = 'fair15'
               AND date_string_p >= date_sub('__today__', __daysback_end__) 
               AND date_string_p <= date_sub('__today__', __daysback_start__) 
               AND date_string_p NOT IN __bad_days__) dctr_v2
        LEFT JOIN (
        SELECT adid, is_mobile, source_id, SUM(att_lineitems_brand) as att_lineitems_brand, SUM(att_lineitems_seller) as att_lineitems_seller,
                SUM(attributed_gmv_brand) as attributed_gmv_brand, SUM(attributed_gmv_seller) as attributed_gmv_seller
        FROM fair.pt3_14day_report_nas 
        WHERE
                ad_date >= date_sub('__today__', __daysback_end__) 
                AND ad_date <= date_sub('__today__', __daysback_start__) 
                AND ad_date NOT IN __bad_days__ 
                AND structure_version = 2
                AND is_pla = 1
        GROUP BY adid, is_mobile, source_id
        ) coop_14d
        ON dctr_v2.adid = coop_14d.adid
        AND dctr_v2.is_mobile = coop_14d.is_mobile
        AND dctr_v2.source_id = coop_14d.source_id

        GROUP BY dctr_v2.adid, dctr_v2.is_mobile, dctr_v2.source_id)

           --final query
           SELECT  dctr_pla.catalog_item_id AS catalog_item_id,
                   coalesce(dctr_pla.id_unique_variant, dctr_pla.catalog_item_id) AS id_unique_variant,
                   dctr_pla.seller_id AS seller_id,
                   dctr_pla.source_id AS source_id,
                   dctr_pla.adid AS adid,
                   dctr_pla.brand_nm AS brand_nm,
                   dctr_pla.division_id AS division_id,
                   dctr_pla.super_department_id AS super_department_id,
                   dctr_pla.department_id AS department_id,
                   dctr_pla.category_id AS category_id,
                   dctr_pla.sub_category_id AS sub_category_id,
                   dctr_pla.is_mobile AS is_mobile,
                   dctr.impressions/__tot_days__ AS impressions,
                   dctr.clicks/__tot_days__ AS clicks,
                   dctr.adspend/__tot_days__ AS adspend,
                   dctr.qty/__tot_days__ AS qty,
                   dctr.orders_brand/__tot_days__ AS orders_brand,
                   dctr.orders_seller/__tot_days__ AS orders_seller,
                   dctr.gmv_revenue_adjustment_seller/__tot_days__ AS gmv_revenue_adjustment_seller,
                   dctr.gmv_revenue_adjustment_brand/__tot_days__ AS gmv_revenue_adjustment_brand,
                   dctr.cost/__tot_days__ AS cost,
                   (dctr.gmv_revenue_adjustment_seller - dctr.cost)/__tot_days__ AS margin_seller,
                   (dctr.gmv_revenue_adjustment_brand - dctr.cost)/__tot_days__ AS margin_brand,
                   dctr.estimated_cp/__tot_days__ AS estimated_cp,
                   dctr.clicks/dctr.impressions AS ctr,
                   dctr.adspend/dctr.clicks AS cpc,
                   dctr.gmv_revenue_adjustment_brand/dctr.clicks AS rpc_brand,
                   dctr.gmv_revenue_adjustment_seller/dctr.clicks AS rpc_seller,
                   dctr.gmv_revenue_adjustment_brand/dctr.adspend AS roas_brand,
                   dctr.gmv_revenue_adjustment_seller/dctr.adspend AS roas_seller,
                   dctr.orders_brand/dctr.clicks AS convrt_brand,
                   dctr.orders_seller/dctr.clicks AS convrt_seller,
                   dctr.gmv_revenue_adjustment_brand/dctr.orders_brand AS ordersize_brand,
                   dctr.gmv_revenue_adjustment_seller/dctr.orders_seller AS ordersize_seller,
                   dctr.estimated_cp/dctr.orders_brand AS cpsize_brand,
                   dctr.estimated_cp/dctr.orders_seller AS cpsize_seller,
                   (dctr.gmv_revenue_adjustment_brand - dctr.cost)/dctr.clicks AS mpc_brand,
                   (dctr.gmv_revenue_adjustment_seller - dctr.cost)/dctr.clicks AS mpc_seller,
                   (dctr.gmv_revenue_adjustment_brand - dctr.cost)/dctr.orders_brand AS mpo_brand,
                   (dctr.gmv_revenue_adjustment_seller - dctr.cost)/dctr.orders_seller AS mpo_seller,
                   dctr.gmv_revenue_adjustment_brand/dctr.estimated_cp AS rpcp_brand,
                   dctr.gmv_revenue_adjustment_seller/dctr.estimated_cp AS rpcp_seller

            FROM dctr_pla LEFT JOIN dctr
                 ON dctr_pla.adid = dctr.adid
                 AND dctr_pla.source_id = dctr.source_id
                 AND dctr_pla.is_mobile = dctr.is_mobile
            """
    
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    bad_days_list = [i.replace("('",'').replace("')",'') for i in bad_days.split("', '")]
    exclude_days = len([datetime.strptime(dat, '%Y-%m-%d') for dat in bad_days_list if datetime.strptime(dat, '%Y-%m-%d').date()>=datetime.strptime(today_ds, '%Y-%m-%d').date()- \
                 timedelta(days=days_backtrack_end) and datetime.strptime(dat, '%Y-%m-%d').date()<=datetime.strptime(today_ds, '%Y-%m-%d').date()- timedelta(days=days_backtrack_start)])
    query = query.replace('__tot_days__', str(days_backtrack_end - days_backtrack_start+1-exclude_days))
    
    df = spark.sql(query).toPandas()
    return(df)

In [7]:
def get_semSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # this function is to retrieve the "SEM" signals of each item/ad
    # SEM signals means the performance of an item/ad in SEM channel (Search Engine Marketing)
    # signals are extracted from different time intervals as specified in the json file - 'output_sem_performance_prediction_config_train_config_20201227.json'
    
    query = """WITH             
            dctr
            AS
            (SELECT adid,
                    source_id,
                    is_mobile,
                    SUM(impressions) AS impressions,
                    SUM(clicks) AS clicks,
                    SUM(adspend) AS adspend,
                    SUM(qty) AS qty,
                    SUM(orders) AS orders,
                    SUM(gmv_revenue_adjustment) AS gmv_revenue_adjustment,
                    SUM(cost) AS cost,
                    SUM(estimated_cp) AS estimated_cp
                              
            FROM sem.daily_criteria_traffic_revenue_v2_tmp
            WHERE
                    customer_id_p = '1' 
                   AND (source_id = 2 or source_id=4)
                   AND adtype_p ='pla' 
                   AND structure_version_p = 2
                   AND revenue_source_p = 'fair15'
                   AND date_string_p >= date_sub('__today__', __daysback_end__) 
                   AND date_string_p <= date_sub('__today__', __daysback_start__) 
                   AND date_string_p NOT IN __bad_days__
            GROUP BY adid, is_mobile, source_id)
            
               --final query
               SELECT  dctr_pla.catalog_item_id AS catalog_item_id,
                       dctr_pla.seller_id AS seller_id,
                       dctr_pla.adid AS adid,
                       dctr_pla.brand_nm AS brand_nm,
                       dctr_pla.division_id AS division_id,
                       dctr_pla.super_department_id AS super_department_id,
                       dctr_pla.department_id AS department_id,
                       dctr_pla.category_id AS category_id,
                       dctr_pla.sub_category_id AS sub_category_id,
                       dctr_pla.source_id AS source_id,
                       dctr_pla.is_mobile AS is_mobile,
                       dctr.impressions/__tot_days__ AS impressions,
                       dctr.clicks/__tot_days__ AS clicks,
                       dctr.adspend/__tot_days__ AS adspend,
                       dctr.qty/__tot_days__ AS qty,
                       dctr.orders/__tot_days__ AS orders,
                       dctr.gmv_revenue_adjustment/__tot_days__ AS gmv_revenue_adjustment,
                       dctr.cost/__tot_days__ AS cost,
                       (dctr.gmv_revenue_adjustment - dctr.cost)/__tot_days__ AS margin,
                       dctr.estimated_cp/__tot_days__ AS estimated_cp,
                       dctr.clicks/dctr.impressions AS ctr,
                       dctr.adspend/dctr.clicks AS cpc,
                       dctr.gmv_revenue_adjustment/dctr.clicks AS rpc,
                       dctr.gmv_revenue_adjustment/dctr.adspend AS roas,
                       dctr.orders/dctr.clicks AS convrt,
                       dctr.gmv_revenue_adjustment/dctr.orders AS ordersize,
                       dctr.estimated_cp/dctr.orders AS cpsize,
                       (dctr.gmv_revenue_adjustment - dctr.cost)/dctr.clicks AS mpc,
                       (dctr.gmv_revenue_adjustment - dctr.cost)/dctr.orders AS mpo,
                       dctr.gmv_revenue_adjustment/dctr.estimated_cp AS rpcp
                       
                FROM dctr_pla LEFT JOIN dctr
                     ON dctr_pla.adid = dctr.adid
                     AND dctr_pla.source_id = dctr.source_id
                     AND dctr_pla.is_mobile = dctr.is_mobile
                """
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    bad_days_list = [i.replace("('",'').replace("')",'') for i in bad_days.split("', '")]
    exclude_days = len([datetime.strptime(dat, '%Y-%m-%d') for dat in bad_days_list if datetime.strptime(dat, '%Y-%m-%d').date()>=datetime.strptime(today_ds, '%Y-%m-%d').date()- \
                 timedelta(days=days_backtrack_end) and datetime.strptime(dat, '%Y-%m-%d').date()<=datetime.strptime(today_ds, '%Y-%m-%d').date()- timedelta(days=days_backtrack_start)])
    query = query.replace('__tot_days__', str(days_backtrack_end - days_backtrack_start+1-exclude_days))
    
    df = spark.sql(query).toPandas()
    return(df)

## func for catalog signals

In [8]:
def get_catalogSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # this function is to retrieve the "catalog" signals of each item/ad
    # catalog signals are the properties of an item recorded in the catalog system, like what category, department, etc of this item belongs to; also the price/
    # price changes of an item
    # signals are extracted from different time intervals as specified in the json file - 'output_sem_performance_prediction_config_train_config_20201227.json'

    query = """
            WITH    
            -- latest price
            tab_latest_price
            AS
            (SELECT 
                 catalog_item_id,
                 seller_id,
                 max(seller_name) as seller_name,
                 max(regexp_replace(division, "\t", "")) AS division,
                 max(regexp_replace(super_dept, "\t", "")) AS super_dept,
                 max(regexp_replace(dept, "\t", "")) AS dept,
                 max(regexp_replace(cat, "\t", "")) AS cat,
                 max(regexp_replace(subcat, "\t", "")) AS subcat,
                 max(brand_name) as brand_name,
                 max(curr_item_price) as curr_item_price,
                 max(price_change_desc) as price_change_desc,
                 max(num_appr_reviews) as num_appr_reviews,
                 max(avg_overall_rating) as avg_overall_rating,
                 max(avg_cost) as avg_cost
       
               FROM casesci_cmn.uber_item_info_dfm_accumulate
       
               WHERE ds = date_sub('__today__', 1)
               GROUP BY catalog_item_id, seller_id),
               
            -- average price
            tab_avg_price
            AS
            (SELECT catalog_item_id, seller_id,
                    AVG(curr_item_price) AS avg_item_price
                    FROM casesci_cmn.uber_item_info_dfm_accumulate
        
                    WHERE ds >= date_sub('__today__', __daysback_end__)
                      AND ds <= date_sub('__today__', __daysback_start__)
                      AND ds NOT IN __bad_days__
              
                GROUP BY catalog_item_id, seller_id)
                
            -- final query
            
            SELECT dctr_pla_distinct.catalog_item_id AS catalog_item_id,
                   dctr_pla_distinct.seller_id AS seller_id,
                   tab_latest_price.seller_name,
                   tab_latest_price.division,
                   tab_latest_price.super_dept,
                   tab_latest_price.dept,
                   tab_latest_price.cat,
                   tab_latest_price.subcat,
                   tab_latest_price.brand_name,
                   tab_latest_price.curr_item_price,
                   tab_latest_price.price_change_desc,
                   tab_latest_price.num_appr_reviews,
                   tab_latest_price.avg_overall_rating,
                   tab_latest_price.avg_cost,
                   tab_latest_price.curr_item_price/tab_avg_price.avg_item_price AS price_change_ratio,
                   tab_latest_price.curr_item_price - tab_avg_price.avg_item_price AS price_change_value,
                   tab_latest_price.curr_item_price - tab_latest_price.avg_cost AS avg_margin
                   
            FROM (SELECT DISTINCT catalog_item_id, seller_id from dctr_pla) dctr_pla_distinct 
            LEFT JOIN tab_latest_price
                 ON dctr_pla_distinct.catalog_item_id = tab_latest_price.catalog_item_id
                 AND dctr_pla_distinct.seller_id = tab_latest_price.seller_id
                 LEFT JOIN tab_avg_price
                 ON tab_latest_price.catalog_item_id = tab_avg_price.catalog_item_id
                 AND tab_latest_price.seller_id = tab_avg_price.seller_id
     """  
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    sql = spark.sql(query)
    df = sql.toPandas()
    return(df)

## func for site signals

In [9]:
def get_siteSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # this function is to retrieve the "site" signals of each item/ad
    # site signals are to measure the performance of an item/ad on whole Walmart site (SEM signals only indicate the performance of "SEM" channel)
    # signals are extracted from different time intervals as specified in the json file - 'output_sem_performance_prediction_config_train_config_20201227.json'
    
    partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x: x).map(lambda x: x.replace("ts=", "")).max()
    
    l2 = """SELECT session_id, dt, catlg_itm_id_set[0] as catalog_item_id, action_categ, action_sub_categ
                FROM us_dl_interactions_restrict.interactions_l2_session_events
                    WHERE vtc IS NOT NULL
                          AND vtc != ''
                          AND vtc != '\\N'
                          AND catlg_itm_id_set[0] IS NOT NULL
                          AND catlg_itm_id_set[0] != ''
                          AND (action_categ = 'pageView' or action_sub_categ = 'addToCart')
                          AND dt >= date_sub('__today__', __daysback_end__)
                          AND dt <= date_sub('__today__', __daysback_start__)
                          AND dt NOT IN __bad_days__
                          AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
                          AND context = 'productPage'"""
    l2 = l2.replace('__today__', today_ds)
    l2 = l2.replace('__daysback_end__', str(days_backtrack_end))
    l2 = l2.replace('__daysback_start__', str(days_backtrack_start))
    l2 = l2.replace('__bad_days__', bad_days)
    l2 = l2.replace('__partitionDate__', partitionDate)
    # session_id, dt, catalog_item_id, action_categ
    spark.sql(l2).write.mode("overwrite").partitionBy('dt', 'action_categ', 'action_sub_categ').saveAsTable('fair_dev.yp_test_interactions_l2')
    
    print('l2 saved for site signals')
    
    l1 = """SELECT distinct session_id, dt, itm_list[0]['catlgItm']['sellerId'] as sellerid, action_categ, action_sub_categ,
                            itm_list[0]['catlgItm']['productId'] as productid
                            FROM us_dl_interactions_restrict.interactions_l1_cbb_catalog
                            WHERE dt >= date_sub('__today__', __daysback_end__)
                            AND dt <= date_sub('__today__', __daysback_start__)
                            AND dt NOT IN __bad_days__
                            AND itm_list is not null
                            AND itm_list[0]['catlgItm']['sellerId'] IS NOT NULL
                            AND itm_list[0]['catlgItm']['productId'] IS NOT NULL
                            AND (action_categ = 'pageView' or action_sub_categ = 'addToCart')
                            AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
                            AND context = 'productPage'"""
    l1 = l1.replace('__today__', today_ds)
    l1 = l1.replace('__daysback_end__', str(days_backtrack_end))
    l1 = l1.replace('__daysback_start__', str(days_backtrack_start))
    l1 = l1.replace('__bad_days__', bad_days)
    l1 = l1.replace('__partitionDate__', partitionDate)
    # session_id, dt, action_categ, sellerid, productid
    spark.sql(l1).write.mode("overwrite").partitionBy('dt', 'action_categ', 'action_sub_categ').saveAsTable('fair_dev.yp_test_interactions_l1')
    
    print('l1 saved for site signals')
    
    dmas_seller = spark.sql("select * from fair_dev.yp_test_dmas_seller")
    dmas_item = spark.sql("select * from fair_dev.yp_test_dmas_item")
    l1 = """select * from fair_dev.yp_test_interactions_l1 
            WHERE dt >= date_sub('__today__', __daysback_end__)
                            AND dt <= date_sub('__today__', __daysback_start__)
                            AND dt NOT IN __bad_days__"""
    l1 = l1.replace('__today__', today_ds)
    l1 = l1.replace('__daysback_end__', str(days_backtrack_end))
    l1 = l1.replace('__daysback_start__', str(days_backtrack_start))
    l1 = l1.replace('__bad_days__', bad_days)
    l1_df = spark.sql(l1)
    
    l1_dmas = l1_df.join(broadcast(dmas_seller),dmas_seller.seller_id ==  l1_df.sellerid, how='inner').drop(l1_df.sellerid).drop(dmas_seller.seller_id)
    l1_dmas = l1_dmas.join(dmas_item,dmas_item.product_id ==  l1_dmas.productid, how='left').drop(dmas_item.product_id).drop(l1_dmas.productid)
    # session_id, dt, action_categ, us_seller_id, item_id on table l1
    l1_dmas.write.mode("overwrite").partitionBy('dt', 'action_categ', 'action_sub_categ').saveAsTable('fair_dev.yp_test_interactions_l1_dmas')
    
    print('pageview l1_dmas saved for site signals after getting seller_id and catalog_item_id from dmas table')
    
    l2 = """select * from fair_dev.yp_test_interactions_l2 
        WHERE dt >= date_sub('__today__', __daysback_end__)
                        AND dt <= date_sub('__today__', __daysback_start__)
                        AND dt NOT IN __bad_days__"""
    l2 = l2.replace('__today__', today_ds)
    l2 = l2.replace('__daysback_end__', str(days_backtrack_end))
    l2 = l2.replace('__daysback_start__', str(days_backtrack_start))
    l2 = l2.replace('__bad_days__', bad_days)
    l2_df = spark.sql(l2)
    
    l1_dmas = """select * from fair_dev.yp_test_interactions_l1_dmas 
        WHERE dt >= date_sub('__today__', __daysback_end__)
                        AND dt <= date_sub('__today__', __daysback_start__)
                        AND dt NOT IN __bad_days__"""
    l1_dmas = l1_dmas.replace('__today__', today_ds)
    l1_dmas = l1_dmas.replace('__daysback_end__', str(days_backtrack_end))
    l1_dmas = l1_dmas.replace('__daysback_start__', str(days_backtrack_start))
    l1_dmas = l1_dmas.replace('__bad_days__', bad_days)
    l1_dmas_df = spark.sql(l1_dmas)
    
    l2_l1_dmas = l2_df.join(l1_dmas_df, [(l2_df.session_id == l1_dmas_df.session_id) & (l2_df.dt == l1_dmas_df.dt) & (l2_df.action_categ == l1_dmas_df.action_categ)], how = 'left')\
    .drop(l1_dmas_df.action_categ).drop(l1_dmas_df.dt).drop(l1_dmas_df.session_id).drop(l1_dmas_df.item_id).drop(l1_dmas_df.action_sub_categ)
    l2_l1_dmas.write.mode("overwrite").partitionBy('dt', 'action_categ', 'action_sub_categ').saveAsTable('fair_dev.yp_test_interactions_l2_l1_dmas')
    # session_id, dt, action_categ, us_seller_id, catalog_item_id on l2 table
    print('l2_l1_dmas saved for site signals')
    
    l2_l1_dmas = """select * from fair_dev.yp_test_interactions_l2_l1_dmas 
        WHERE dt >= date_sub('__today__', __daysback_end__)
                        AND dt <= date_sub('__today__', __daysback_start__)
                        AND dt NOT IN __bad_days__"""
    l2_l1_dmas = l2_l1_dmas.replace('__today__', today_ds)
    l2_l1_dmas = l2_l1_dmas.replace('__daysback_end__', str(days_backtrack_end))
    l2_l1_dmas = l2_l1_dmas.replace('__daysback_start__', str(days_backtrack_start))
    l2_l1_dmas = l2_l1_dmas.replace('__bad_days__', bad_days)
    spark.sql(l2_l1_dmas).createOrReplaceTempView('l2_l1_dmas_df')
    
    
    query = """
            WITH   
            tab_product_view
            AS
            (SELECT catalog_item_id, us_seller_id AS seller_id,
                count(*) AS total_page_views
                FROM l2_l1_dmas_df
                WHERE action_categ = 'pageView'
                GROUP BY catalog_item_id, us_seller_id),
            
            tab_add_to_cart
            AS
            (SELECT catalog_item_id, us_seller_id AS seller_id,
                count(*) AS total_add_to_carts
                FROM l2_l1_dmas_df
                WHERE action_sub_categ = 'addToCart'
                GROUP BY catalog_item_id, us_seller_id),
                
            tab_auth1
            AS
            (SELECT daily_order.catalog_item_id, transaction.slr_org_cd,
               daily_order.order_nbr, 
               SUM(total_auth_amount) AS total_revenue, 
               SUM(total_auth_cost) AS total_cost 
               FROM (SELECT * FROM fair.daily_auth_order
                  WHERE auth_date >= date_sub('__today__', __daysback_end__)
                  AND auth_date <= date_sub('__today__', __daysback_start__)
                  AND auth_date NOT IN __bad_days__) daily_order

               LEFT JOIN (SELECT order_line_nbr, order_nbr,catlg_item_id,slr_org_cd 
                           FROM ww_crew_dl_rpt_vm.cnsld_order_item 
                           WHERE order_plcd_dt>= date_sub('__today__', __daysback_end__)
                           AND order_plcd_dt<= date_sub('__today__', __daysback_start__)
                           AND order_plcd_dt NOT IN __bad_days__) transaction
               ON transaction.order_nbr = daily_order.order_nbr
               AND transaction.order_line_nbr = daily_order.line_nbr

               GROUP BY daily_order.catalog_item_id, transaction.slr_org_cd, daily_order.order_nbr),

            tab_auth2
            AS
            (SELECT catalog_item_id, slr_org_cd as seller_id,
              count(*) AS total_orders,
              SUM(total_revenue) AS total_revenue,
              SUM(total_cost) AS total_cost,
              SUM(total_revenue) - SUM(total_cost) AS total_margin

            FROM

               tab_auth1

            GROUP BY catalog_item_id, slr_org_cd)
            
                -- final query
    
            SELECT dctr_pla_distinct.catalog_item_id AS catalog_item_id,
                   dctr_pla_distinct.seller_id,
                   tab_product_view.total_page_views AS total_page_views,
                   tab_add_to_cart.total_add_to_carts AS total_add_to_carts,
                   tab_auth2.total_revenue AS total_revenue,
                   tab_auth2.total_margin AS margin,
                   tab_add_to_cart.total_add_to_carts/tab_product_view.total_page_views AS atcpv,
                   tab_auth2.total_orders/tab_product_view.total_page_views AS convrt,
                   tab_auth2.total_revenue/tab_product_view.total_page_views AS rpv,
                   tab_auth2.total_margin/tab_product_view.total_page_views AS mpv,
                   tab_auth2.total_revenue/tab_auth2.total_orders AS ordersize,
                   tab_auth2.total_orders AS total_orders

            FROM (SELECT DISTINCT catalog_item_id, seller_id from dctr_pla) dctr_pla_distinct 
                 LEFT JOIN tab_product_view
                 ON dctr_pla_distinct.catalog_item_id = tab_product_view.catalog_item_id
                 AND dctr_pla_distinct.seller_id = tab_product_view.seller_id
                 LEFT JOIN tab_add_to_cart
                 ON dctr_pla_distinct.catalog_item_id = tab_add_to_cart.catalog_item_id
                 AND dctr_pla_distinct.seller_id = tab_add_to_cart.seller_id
                 LEFT JOIN tab_auth2
                 ON dctr_pla_distinct.catalog_item_id = tab_auth2.catalog_item_id
                 AND dctr_pla_distinct.seller_id = tab_auth2.seller_id
            """
    
#     query = """
#             WITH   
#             tab_product_view
#             AS
#             (SELECT catlg_itm_id_set[0] AS catalog_item_id, seller_id, 
#                 count(*) AS total_page_views

#                 FROM (SELECT * FROM us_dl_interactions_restrict.interactions_l2_session_events
#                     WHERE vtc IS NOT NULL
#                           AND vtc != ''
#                           AND vtc != '\\N'
#                           AND catlg_itm_id_set[0] IS NOT NULL
#                           AND catlg_itm_id_set[0] != ''
#                           AND action_categ = 'pageView'
#                           AND dt >= date_sub('__today__', __daysback_end__)
#                           AND dt <= date_sub('__today__', __daysback_start__)
#                           AND dt NOT IN __bad_days__
#                           AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
#                           AND context = 'productPage') interactions_l2
#                 LEFT JOIN (SELECT distinct session_id, dt, dmas.us_seller_id as seller_id 
#                             FROM us_dl_interactions_restrict.interactions_l1_cbb_catalog l1
#                             INNER JOIN (SELECT distinct seller_id, us_seller_id from dmas.dfm
#                                         WHERE ts = '__partitionDate__') dmas
#                             ON dt >= date_sub('__today__', __daysback_end__)
#                             AND dt <= date_sub('__today__', __daysback_start__)
#                             AND dt NOT IN __bad_days__
#                             AND itm_list is not null
#                             AND itm_list[0]['catlgItm']['sellerId'] IS NOT NULL
#                             AND action_categ = 'pageView'
#                             AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
#                             AND context = 'productPage'
#                             AND l1.itm_list[0]['catlgItm']['sellerId'] = dmas.seller_id) interactions_l1
#                 ON interactions_l2.session_id = interactions_l1.session_id
#                 AND interactions_l2.dt = interactions_l1.dt
                
#                 GROUP BY catlg_itm_id_set[0], seller_id),
                
#             tab_add_to_cart
#             AS
#             (SELECT catlg_itm_id_set[0] AS catalog_item_id, seller_id, 
#             count(*) AS total_add_to_carts

#         FROM (SELECT * FROM us_dl_interactions_restrict.interactions_l2_session_events
#                     WHERE vtc IS NOT NULL
#                           AND vtc != ''
#                           AND vtc != '\\N'
#                           AND catlg_itm_id_set[0] IS NOT NULL
#                           AND catlg_itm_id_set[0] != ''
#                           AND action_categ = 'addToCart'
#                           AND dt >= date_sub('__today__', __daysback_end__)
#                           AND dt <= date_sub('__today__', __daysback_start__)
#                           AND dt NOT IN __bad_days__
#                           AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
#                           AND context = 'productPage') interactions_l2
#             LEFT JOIN (SELECT distinct session_id, dt, dmas.us_seller_id as seller_id 
#                             FROM us_dl_interactions_restrict.interactions_l1_cbb_catalog l1
#                             INNER JOIN (SELECT distinct seller_id, us_seller_id from dmas.dfm
#                                         WHERE ts = '__partitionDate__') dmas
#                             ON dt >= date_sub('__today__', __daysback_end__)
#                             AND dt <= date_sub('__today__', __daysback_start__)
#                             AND dt NOT IN __bad_days__
#                             AND itm_list is not null
#                             AND itm_list[0]['catlgItm']['sellerId'] IS NOT NULL
#                             AND action_categ = 'addToCart'
#                             AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
#                             AND context = 'productPage'
#                             AND l1.itm_list[0]['catlgItm']['sellerId'] = dmas.seller_id) interactions_l1
#             ON interactions_l2.session_id = interactions_l1.session_id
#             AND interactions_l2.dt = interactions_l1.dt

#         GROUP BY catlg_itm_id_set[0], seller_id),
        
#         tab_auth1
#         AS
#         (SELECT daily_order.catalog_item_id, transaction.slr_org_cd,
#            daily_order.order_nbr, 
#            SUM(total_auth_amount) AS total_revenue, 
#            SUM(total_auth_cost) AS total_cost 
#            FROM (SELECT * FROM fair.daily_auth_order
#               WHERE auth_date >= date_sub('__today__', __daysback_end__)
#               AND auth_date <= date_sub('__today__', __daysback_start__)
#               AND auth_date NOT IN __bad_days__) daily_order
           
#            LEFT JOIN (SELECT order_line_nbr, order_nbr,catlg_item_id,slr_org_cd 
#                        FROM ww_crew_dl_rpt_vm.cnsld_order_item 
#                        WHERE order_plcd_dt>= date_sub('__today__', __daysback_end__)
#                        AND order_plcd_dt<= date_sub('__today__', __daysback_start__)
#                        AND order_plcd_dt NOT IN __bad_days__) transaction
#            ON transaction.order_nbr = daily_order.order_nbr
#            AND transaction.order_line_nbr = daily_order.line_nbr

#            GROUP BY daily_order.catalog_item_id, transaction.slr_org_cd, daily_order.order_nbr),
           
#         tab_auth2
#         AS
#         (SELECT catalog_item_id, slr_org_cd as seller_id,
#           count(*) AS total_orders,
#           SUM(total_revenue) AS total_revenue,
#           SUM(total_cost) AS total_cost,
#           SUM(total_revenue) - SUM(total_cost) AS total_margin
          
#         FROM
  
#            tab_auth1
           
#         GROUP BY catalog_item_id, slr_org_cd)
    
#     -- final query
    
#     SELECT dctr_pla_distinct.catalog_item_id AS catalog_item_id,
#            dctr_pla_distinct.seller_id,
#            tab_product_view.total_page_views AS total_page_views,
#            tab_add_to_cart.total_add_to_carts AS total_add_to_carts,
#            tab_auth2.total_revenue AS total_revenue,
#            tab_auth2.total_margin AS margin,
#            tab_add_to_cart.total_add_to_carts/tab_product_view.total_page_views AS atcpv,
#            tab_auth2.total_orders/tab_product_view.total_page_views AS convrt,
#            tab_auth2.total_revenue/tab_product_view.total_page_views AS rpv,
#            tab_auth2.total_margin/tab_product_view.total_page_views AS mpv,
#            tab_auth2.total_revenue/tab_auth2.total_orders AS ordersize,
#            tab_auth2.total_orders AS total_orders
          
#     FROM (SELECT DISTINCT catalog_item_id, seller_id from dctr_pla) dctr_pla_distinct 
#          LEFT JOIN tab_product_view
#          ON dctr_pla_distinct.catalog_item_id = tab_product_view.catalog_item_id
#          AND dctr_pla_distinct.seller_id = tab_product_view.seller_id
#          LEFT JOIN tab_add_to_cart
#          ON dctr_pla_distinct.catalog_item_id = tab_add_to_cart.catalog_item_id
#          AND dctr_pla_distinct.seller_id = tab_add_to_cart.seller_id
#          LEFT JOIN tab_auth2
#          ON dctr_pla_distinct.catalog_item_id = tab_auth2.catalog_item_id
#          AND dctr_pla_distinct.seller_id = tab_auth2.seller_id
        
#         """
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    query = query.replace('__partitionDate__', partitionDate)
    sql = spark.sql(query)
    df = sql.toPandas()
    return(df)

In [10]:
def get_siteSignal_item(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # this function is to retrieve the "site" signals of each item/ad
    # site signals are to measure the performance of an item/ad on whole Walmart site (SEM signals only indicate the performance of "SEM" channel)
    # signals are extracted from different time intervals as specified in the json file - 'output_sem_performance_prediction_config_train_config_20201227.json'
    
    query = """
            WITH   
            tab_product_view
            AS
            (SELECT catlg_itm_id_set[0] AS catalog_item_id,
                count(*) AS total_page_views

                FROM us_dl_interactions_restrict.interactions_l2_session_events 

                WHERE vtc IS NOT NULL
                      AND vtc != ''
                      AND vtc != '\\N'
                      AND catlg_itm_id_set[0] IS NOT NULL
                      AND catlg_itm_id_set[0] != ''
                      AND action_categ = 'pageView'
                      AND dt >= date_sub('__today__', __daysback_end__)
                      AND dt <= date_sub('__today__', __daysback_start__)
                      AND dt NOT IN __bad_days__
                      AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
                      AND context = 'productPage'
                GROUP BY catlg_itm_id_set[0]),
                
            tab_add_to_cart
            AS
            (SELECT catlg_itm_id_set[0] AS catalog_item_id,
            count(*) AS total_add_to_carts

        FROM us_dl_interactions_restrict.interactions_l2_session_events

        WHERE vtc IS NOT NULL
              AND vtc != ''
              AND vtc != '\\N'
              AND catlg_itm_id_set[0] IS NOT NULL
              AND catlg_itm_id_set[0] != ''
              AND action_sub_categ = 'addToCart'
              AND dt >= date_sub('__today__', __daysback_end__)
              AND dt <= date_sub('__today__', __daysback_start__)
              AND dt NOT IN __bad_days__
              AND pipeline in ('PULSE_USOA_RWEB', 'ANIVIA_USOA_APP') 
              AND context = 'productPage'
        GROUP BY catlg_itm_id_set[0]),
        
        tab_auth1
        AS
        (SELECT catalog_item_id, 
           order_nbr, 
           SUM(total_auth_amount) AS total_revenue, 
           SUM(total_auth_cost) AS total_cost 
           FROM fair.daily_auth_order 
           WHERE 
              auth_date >= date_sub('__today__', __daysback_end__)
              AND auth_date <= date_sub('__today__', __daysback_start__)
              AND auth_date NOT IN __bad_days__
           GROUP BY catalog_item_id, order_nbr),
           
        tab_auth2
        AS
        (SELECT catalog_item_id,
          count(*) AS total_orders,
          SUM(total_revenue) AS total_revenue,
          SUM(total_cost) AS total_cost,
          SUM(total_revenue) - SUM(total_cost) AS total_margin
          
        FROM
  
           tab_auth1
           
        GROUP BY catalog_item_id)
    
    -- final query
    
    SELECT dctr_pla.catalog_item_id AS catalog_item_id,
           dctr_pla.seller_id AS seller_id,
           tab_product_view.total_page_views AS total_page_views,
           tab_add_to_cart.total_add_to_carts AS total_add_to_carts,
           tab_auth2.total_revenue AS total_revenue,
           tab_auth2.total_margin AS margin,
           tab_add_to_cart.total_add_to_carts/tab_product_view.total_page_views AS atcpv,
           tab_auth2.total_orders/tab_product_view.total_page_views AS convrt,
           tab_auth2.total_revenue/tab_product_view.total_page_views AS rpv,
           tab_auth2.total_margin/tab_product_view.total_page_views AS mpv,
           tab_auth2.total_revenue/tab_auth2.total_orders AS ordersize,
           tab_auth2.total_orders AS total_orders
          
    FROM dctr_pla LEFT JOIN tab_product_view
         ON dctr_pla.catalog_item_id = tab_product_view.catalog_item_id
         LEFT JOIN tab_add_to_cart
         ON tab_product_view.catalog_item_id = tab_add_to_cart.catalog_item_id
         LEFT JOIN tab_auth2
         ON tab_add_to_cart.catalog_item_id = tab_auth2.catalog_item_id
        
        """
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    
    sql = spark.sql(query)
    df = sql.toPandas()
    return(df)

# func for bounce signals

In [11]:
def get_bounceSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # bounce signals are to measure non-effective visit in SEM channel as well as whole walmart site
    # a bounce is a non-effective visit - for example viewed but not added to cart
    
    partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x: x).map(lambda x: x.replace("ts=", "")).max()
    
    l1 = """SELECT distinct session_id, dt, itm_list[0]['catlgItm']['sellerId'] as sellerid, action_categ,
                            itm_list[0]['catlgItm']['productId'] as productid
                            FROM us_dl_interactions_restrict.interactions_l1_cbb_catalog
                            WHERE dt >= date_sub('__today__', __daysback_end__)
                            AND dt <= date_sub('__today__', __daysback_start__)
                            AND dt NOT IN __bad_days__
                            AND pipeline = 'PULSE_USOA_RWEB'
                            AND itm_list is not null
                            AND itm_list[0]['catlgItm']['sellerId'] IS NOT NULL
                            AND itm_list[0]['catlgItm']['productId'] IS NOT NULL"""
    l1 = l1.replace('__today__', today_ds)
    l1 = l1.replace('__daysback_end__', str(days_backtrack_end))
    l1 = l1.replace('__daysback_start__', str(days_backtrack_start))
    l1 = l1.replace('__bad_days__', bad_days)
    l1 = l1.replace('__partitionDate__', partitionDate)
    # session_id, dt, action_categ, sellerid, productid
    spark.sql(l1).write.mode("overwrite").partitionBy('dt', 'action_categ').saveAsTable('fair_dev.yp_test_bounce_interactions_l1')
    
    print('l1 saved for bounce signals')
    
    dmas_seller = spark.sql("select * from fair_dev.yp_test_dmas_seller")
    dmas_item = spark.sql("select * from fair_dev.yp_test_dmas_item")
    l1 = """select * from fair_dev.yp_test_bounce_interactions_l1 
            WHERE dt >= date_sub('__today__', __daysback_end__)
                            AND dt <= date_sub('__today__', __daysback_start__)
                            AND dt NOT IN __bad_days__"""
    l1 = l1.replace('__today__', today_ds)
    l1 = l1.replace('__daysback_end__', str(days_backtrack_end))
    l1 = l1.replace('__daysback_start__', str(days_backtrack_start))
    l1 = l1.replace('__bad_days__', bad_days)
    l1_df = spark.sql(l1)
    
    l1_dmas = l1_df.join(broadcast(dmas_seller),dmas_seller.seller_id ==  l1_df.sellerid, how='inner').drop(l1_df.sellerid).drop(dmas_seller.seller_id)
    l1_dmas = l1_dmas.join(dmas_item,dmas_item.product_id ==  l1_dmas.productid, how='left').drop(dmas_item.product_id).drop(l1_dmas.productid)
    # session_id, dt, action_categ, us_seller_id, item_id on table l1
    l1_dmas.write.mode("overwrite").partitionBy('dt', 'action_categ').saveAsTable('fair_dev.yp_test_bounce_interactions_l1_dmas')
    
    print('pageview l1_dmas saved for bounce signals after getting seller_id and catalog_item_id from dmas table')
    
    l1_dmas = """select * from fair_dev.yp_test_bounce_interactions_l1_dmas 
        WHERE dt >= date_sub('__today__', __daysback_end__)
                        AND dt <= date_sub('__today__', __daysback_start__)
                        AND dt NOT IN __bad_days__"""
    l1_dmas = l1_dmas.replace('__today__', today_ds)
    l1_dmas = l1_dmas.replace('__daysback_end__', str(days_backtrack_end))
    l1_dmas = l1_dmas.replace('__daysback_start__', str(days_backtrack_start))
    l1_dmas = l1_dmas.replace('__bad_days__', bad_days)
    l1_dmas_df = spark.sql(l1_dmas)
    
    spark.sql(l1_dmas).createOrReplaceTempView('l1_dmas')
    
    query = """
            SELECT dctr_pla_distinct.catalog_item_id,
                      dctr_pla_distinct.seller_id, 
                      SUM(num_bounces) AS bounces,
                      SUM(num_visits) AS visits,
                      SUM(num_sem_visits) AS sem_visits,
                      SUM(num_sem_bounces) AS sem_bounces,
                      SUM(num_bounces)/SUM(num_visits) AS bounce_rate,
                      SUM(num_sem_bounces)/SUM(num_sem_visits) AS sem_bounce_rate
       
               FROM
               (SELECT DISTINCT session_id, 
                        mkt_veh, 
                        total_non_performance_metrics_view,
                        1 AS num_visits,
                        IF (total_non_performance_metrics_view = 1, 1,  0) AS num_bounces,
                        IF (mkt_veh = 'Paid:SEM', 1, 0) AS num_sem_visits,
                        IF (total_non_performance_metrics_view = 1 AND mkt_veh = 'Paid:SEM', 1, 0) AS num_sem_bounces
        
                 FROM us_dl_interactions_restrict.interactions_l3_session_stats_daily l3
        
                 WHERE dt >= date_sub('__today__', __daysback_end__)
                       AND dt <= date_sub('__today__', __daysback_start__)
                       AND dt NOT IN __bad_days__
                       AND pipeline = 'PULSE_USOA_RWEB')  cbb_l3
            
               LEFT JOIN l1_dmas cbb_l1
           
                ON cbb_l3.session_id = cbb_l1.session_id
    
                RIGHT JOIN (SELECT DISTINCT catalog_item_id, seller_id from dctr_pla) dctr_pla_distinct 
    
                ON dctr_pla_distinct.catalog_item_id = cbb_l1.item_id
                AND dctr_pla_distinct.seller_id = cbb_l1.us_seller_id
                
                GROUP BY dctr_pla_distinct.catalog_item_id, dctr_pla_distinct.seller_id
    
        """ 
    
#     query = """WITH 
#                l2_withsellerid 
#                AS
#                (SELECT DISTINCT l2.session_id, l2.catalog_item_id, l1.seller_id
#                 FROM 
#                 (SELECT DISTINCT catlg_itm_id_set[0] AS catalog_item_id, session_id
#                     FROM us_dl_interactions_restrict.interactions_l2_session_events
#                     WHERE dt >= date_sub('__today__', __daysback_end__)
#                     AND dt <= date_sub('__today__', __daysback_start__)
#                     AND dt NOT IN __bad_days__
#                     AND pipeline = 'PULSE_USOA_RWEB') l2
#                 INNER JOIN (
#                         SELECT distinct session_id, dt, dmas.us_seller_id as seller_id 
#                         FROM us_dl_interactions_restrict.interactions_l1_cbb_catalog l1
#                         INNER JOIN (
#                                     SELECT distinct seller_id, us_seller_id from dmas.dfm
#                                     WHERE ts = '__partitionDate__'
#                                     ) dmas
#                         ON dt >= date_sub('__today__', __daysback_end__)
#                         AND dt <= date_sub('__today__', __daysback_start__)
#                         AND dt NOT IN __bad_days__
#                         AND itm_list is not null
#                         AND itm_list[0]['catlgItm']['sellerId'] IS NOT NULL
#                         AND pipeline = 'PULSE_USOA_RWEB'
#                         AND l1.itm_list[0]['catlgItm']['sellerId'] = dmas.seller_id
#                     ) l1_withseller
#                 ON l2.session_id = l1_withseller.session_id
#                 AND l2.dt = l1_withseller.dt
                
                
#                 -- final query
    
#             SELECT dctr_pla_distinct.catalog_item_id,
#                       dctr_pla_distinct.seller_id, 
#                       SUM(num_bounces) AS bounces,
#                       SUM(num_visits) AS visits,
#                       SUM(num_sem_visits) AS sem_visits,
#                       SUM(num_sem_bounces) AS sem_bounces,
#                       SUM(num_bounces)/SUM(num_visits) AS bounce_rate,
#                       SUM(num_sem_bounces)/SUM(num_sem_visits) AS sem_bounce_rate
       
#                FROM
#                (SELECT DISTINCT session_id, 
#                         mkt_veh, 
#                         total_non_performance_metrics_view,
#                         1 AS num_visits,
#                         IF (total_non_performance_metrics_view = 1, 1,  0) AS num_bounces,
#                         IF (mkt_veh = 'Paid:SEM', 1, 0) AS num_sem_visits,
#                         IF (total_non_performance_metrics_view = 1 AND mkt_veh = 'Paid:SEM', 1, 0) AS num_sem_bounces
        
#                  FROM us_dl_interactions_restrict.interactions_l3_session_stats_daily l3
        
#                  WHERE dt >= date_sub('__today__', __daysback_end__)
#                        AND dt <= date_sub('__today__', __daysback_start__)
#                        AND dt NOT IN __bad_days__
#                        AND pipeline = 'PULSE_USOA_RWEB')  cbb_l3
            
#                LEFT JOIN l2_withsellerid cbb_l2
           
#                 ON cbb_l3.session_id = cbb_l2.session_id
    
#                 RIGHT JOIN (SELECT DISTINCT catalog_item_id, seller_id from dctr_pla) dctr_pla_distinct 
    
#                 ON dctr_pla_distinct.catalog_item_id = cbb_l2.catalog_item_id
#                 AND dctr_pla_distinct.seller_id = cbb_l2.seller_id
                
#                 GROUP BY dctr_pla_distinct.catalog_item_id, dctr_pla_distinct.seller_id
    
#         """ 
    
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    query = query.replace('__partitionDate__', partitionDate)
    
    sql = spark.sql(query)
    df = sql.toPandas()
    return(df)

In [12]:
def get_bounceSignal_item(today_ds, days_backtrack_end, days_backtrack_start, bad_days):
    
    # bounce signals are to measure non-effective visit in SEM channel as well as whole walmart site
    # a bounce is a non-effective visit - for example viewed but not added to cart
    query = """
               WITH 
               l3
               AS
               (SELECT DISTINCT session_id, 
                        mkt_veh, 
                        total_non_performance_metrics_view,
                        1 AS num_visits,
                        IF (total_non_performance_metrics_view = 1, 1,  0) AS num_bounces,
                        IF (mkt_veh = 'Paid:SEM', 1, 0) AS num_sem_visits,
                        IF (total_non_performance_metrics_view = 1 AND mkt_veh = 'Paid:SEM', 1, 0) AS num_sem_bounces
        
                 FROM us_dl_interactions_restrict.interactions_l3_session_stats_daily
        
                 WHERE dt >= date_sub('__today__', __daysback_end__)
                       AND dt <= date_sub('__today__', __daysback_start__)
                       AND dt NOT IN __bad_days__
                       AND pipeline = 'PULSE_USOA_RWEB'),
                       
                l3_item
                AS
                (SELECT catlg_itm_id_set[0] AS catalog_item_id,
                      SUM(num_bounces) AS bounces,
                      SUM(num_visits) AS visits,
                      SUM(num_sem_visits) AS sem_visits,
                      SUM(num_sem_bounces) AS sem_bounces,
                      SUM(num_bounces)/SUM(num_visits) AS bounce_rate,
                      SUM(num_sem_bounces)/SUM(num_sem_visits) AS sem_bounce_rate
                     
                FROM (SELECT DISTINCT catlg_itm_id_set, session_id from 
                      us_dl_interactions_restrict.interactions_l2_session_events
                      WHERE dt >= date_sub('__today__', __daysback_end__)
                      AND dt <= date_sub('__today__', __daysback_start__)
                      AND dt NOT IN __bad_days__
                      AND pipeline in ('PULSE_USOA_RWEB')
                      AND context = 'productPage') l2
                RIGHT JOIN l3
                ON l2.session_id = l3.session_id
                GROUP BY catlg_itm_id_set[0])
                
                -- final query
                
                SELECT dctr_pla.catalog_item_id, 
                       dctr_pla.seller_id,
                       l3_item.bounces,
                       l3_item.visits,
                       l3_item.sem_visits,
                       l3_item.sem_bounces,
                       l3_item.bounce_rate,
                       l3_item.sem_bounce_rate
                FROM dctr_pla
                LEFT JOIN l3_item
                ON dctr_pla.catalog_item_id = l3_item.catalog_item_id
    
        """ 
    
    query = query.replace('__today__', today_ds)
    query = query.replace('__daysback_end__', str(days_backtrack_end))
    query = query.replace('__daysback_start__', str(days_backtrack_start))
    query = query.replace('__bad_days__', bad_days)
    
    sql = spark.sql(query)
    df = sql.toPandas()
    return(df)

# query features

In [35]:
from datetime import datetime,timedelta

start_date = '2023-01-28'
gcp_directory = 'gs://msc_fair_airflow/rpc_model/'+start_date+'/'
ref_dates = []
for x in range(0, 5):
    ref_dates.append(str(datetime.strptime(start_date, '%Y-%m-%d').date() - timedelta(days=7)*x))
# ref_dates.append(str(datetime.strptime(start_date, '%Y-%m-%d').date() + timedelta(days=7)))

# config = load_json(gcp_directory.replace(start_date+'/','')+ 'rpc_time_window_config.json')
config = load_json(gcp_directory.replace(start_date+'/','')+ '1month_time_window.json')

In [36]:
ref_dates

['2023-01-28', '2023-01-21', '2023-01-14', '2023-01-07', '2022-12-31']

In [34]:
config

[{'com.walmartlabs.crm.unifiers.sem.SemSignalsUnifier': [{'name': 'l1',
    'condition': "date_string_p BETWEEN date_sub('2020-12-25', 0+0) AND date_sub('2020-12-25', -6) AND date_string_p NOT IN ('2019-11-30', '2019-11-28', '2019-11-29', '2019-11-27', '2020-02-25', '2020-02-26', '2020-02-27', '2019-12-03', '2019-12-02', '2019-12-01')"},
   {'name': 'l2',
    'condition': "date_string_p BETWEEN date_sub('2020-12-25', 0+0) AND date_sub('2020-12-25', -13) AND date_string_p NOT IN ('2019-11-30', '2019-11-28', '2019-11-29', '2019-11-27', '2020-02-25', '2020-02-26', '2020-02-27', '2019-12-03', '2019-12-02', '2019-12-01')"},
   {'name': 'l3',
    'condition': "date_string_p BETWEEN date_sub('2020-12-25', 0+0) AND date_sub('2020-12-25', -20) AND date_string_p NOT IN ('2019-11-30', '2019-11-28', '2019-11-29', '2019-11-27', '2020-02-25', '2020-02-26', '2020-02-27', '2019-12-03', '2019-12-02', '2019-12-01')"},
   {'name': 'l4',
    'condition': "date_string_p BETWEEN date_sub('2020-12-25', 0+0) 

In [12]:
# bad days are typically holidays when RPC/target value pattern are very different from normal days to be excluded from training data
bad_days = """('2022-11-07', '2022-11-14', '2022-11-21', '2022-11-23', '2022-11-24', '2022-11-25')"""

# loop through configuration files to get features

In [25]:
config_sem = list(config[0].values())[0]
config_catalog = list(config[1].values())[0]
config_site = list(config[2].values())[0]
config_bounce = list(config[3].values())[0]

In [ ]:
# starttime = time.time()
# partitionDate = spark.sql("""show partitions dmas.dfm""").rdd.flatMap(lambda x: x).map(lambda x: x.replace("ts=", "")).max()

# bb = """SELECT distinct seller_id, us_seller_id from dmas.dfm WHERE ts = '__partitionDate__'"""
# bb = bb.replace('__partitionDate__', partitionDate)
# spark.sql(bb).write.mode("overwrite").saveAsTable('fair_dev.yp_test_dmas_seller')

# bb = """SELECT distinct item_id, product_id from dmas.dfm WHERE ts = '__partitionDate__'"""
# bb = bb.replace('__partitionDate__', partitionDate)
# spark.sql(bb).write.mode("overwrite").saveAsTable('fair_dev.yp_test_dmas_item')

# endtime = time.time()
# print('time used for creating dmas copy table'+': ' + str(endtime - starttime))

for today_ds in ref_dates:
    starttime = time.time()
    print('Querying data for %s' % today_ds)
    df_feat_all = pd.DataFrame()
    
    # create table view of all catalog_item_ids of interest (today_ds -139 :  today_ds-7)
    if today_ds>start_date:
        join_dctr_item_id(today_ds)
    else:
        join_dctr_item_id_training(today_ds)
    
    ###############################################################
    #*********************** SEM signals***************************
    
    # get target values
    ##{'name': 'l',
    #'condition': "date_string_p BETWEEN date_sub('2020-12-25', 0 + 6) AND date_sub('2020-12-25', 0) AND date_string_p NOT IN ('2019-11-30', '2019-11-28', '2019-11-29', '2019-11-27', '2020-02-25', '2020-02-26', '2020-02-27', '2019-12-03', '2019-12-02', '2019-12-01')"}

    config_tmp = config_sem[0]     
    name = config_tmp['name']
    condition = config_tmp['condition']
    
    #    ["('2020-12-25', 0 + 6)",
    # "('2020-12-25', 0)",
    # "('2019-11-30', '2019-11-28', '2019-11-29', '2019-11-27', '2020-02-25', '2020-02-26', '2020-02-27', '2019-12-03', '2019-12-02', '2019-12-01')"]
    backdays = re.findall(r'\(.*?\)', condition) #
    
    days_backtrack_end = backdays[0][1:-1].split(',')[1].split('+')
    days_backtrack_end = int(days_backtrack_end[0]) + int(days_backtrack_end[1])
    
    days_backtrack_start = int(backdays[1][1:-1].split(',')[1])
    
    # ordersize               double,
    # convrt                  double,
    # catalog_item_id         long,
    # adid                    long,
    df_tmp = get_semSignalCoop(today_ds, days_backtrack_end, days_backtrack_start, bad_days)
    df_tmp = df_tmp[['catalog_item_id', 'id_unique_variant', 'seller_id', 'brand_nm', 'adid', 'source_id', 'is_mobile','division_id',\
                    'super_department_id','department_id','category_id','sub_category_id']].copy()

    upload_csv_to_cloud_storage(df_tmp, gcp_directory+'sample_data_individual_feat/df_header_'+today_ds+'.csv')
    del df_tmp
    endtime = time.time()
    print('time used for creating header table for date '+today_ds+': ' + str(endtime - starttime))
    
    
    for idx in tqdm(range(len(config_sem))):
        print('processing sem and sem_coop features for date '+today_ds+' and name is '+config_sem[idx]['name'])
        config_tmp = config_sem[idx] 
        name = config_tmp['name']
        condition = config_tmp['condition']
    
        backdays = re.findall(r'\(.*?\)', condition) #
    
        days_backtrack_end = backdays[0][1:-1].split(',')[1].split('+')
        days_backtrack_end = int(days_backtrack_end[0]) + int(days_backtrack_end[1])
    
        days_backtrack_start = int(backdays[1][1:-1].split(',')[1])
    
        # sem_is_mobile_w2        int,
        # sem_impressions_w2      double,
        # sem_clicks_w2           double,
        # sem_adspend_w2          double,
        # sem_qty_w2              double,
        # sem_revenue_w2          double,
        # sem_orders_w2           double,
        # sem_margin_w2           double,
        # sem_ctr_w2              double,
        # sem_cpc_w2              double,
        # sem_rpc_w2              double,
        # sem_roas_w2             double,
        # sem_convrt_w2           double,
        # sem_ordersize_w2        double,
        # sem_mpc_w2              double,
        # sem_mpo_w2              double, 
#         df_tmp = get_semSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days)
        
#         df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'source_id', 'adid', 'is_mobile', 'impressions', 'clicks', 'adspend', 'qty', 'gmv_revenue_adjustment', 'orders', 'margin',\
#                             'ctr', 'cpc', 'rpc', 'roas', 'convrt', 'ordersize', 'mpc', 'mpo']].copy()

#         columns = df_tmp.columns
#         for col in columns[5:]:
#             df_tmp.rename(columns = {col: 'sem_'+col+'_'+name}, inplace=True)
        
#         upload_csv_to_cloud_storage(df_tmp, gcp_directory+'sample_data_individual_feat/sem_'+name+'_'+today_ds+'.csv')
#         del df_tmp
#         endtime = time.time()
#         print('time used for sem signals for evergreen for date '+today_ds+' and name is '+config_sem[idx]['name']+': ' + str(endtime - starttime))
        
        # get sem coop features
        starttime = time.time()
        df_tmp = get_semSignalCoop(today_ds, days_backtrack_end, days_backtrack_start, bad_days)

        df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'source_id', 'adid', 'is_mobile', 'impressions', 'clicks', 'adspend', 'qty', 'gmv_revenue_adjustment_brand','cost',\
                         'gmv_revenue_adjustment_seller', 'orders_brand', 'orders_seller', 'margin_brand', 'margin_seller',\
                    'ctr', 'cpc', 'rpc_brand', 'rpc_seller', 'roas_brand', 'roas_seller', 'convrt_brand', 'convrt_seller', 'ordersize_brand', 'ordersize_seller',\
                         'mpc_brand', 'mpc_seller', 'mpo_brand', 'mpo_seller']].copy()

        columns = df_tmp.columns
        for col in columns[5:]:
            df_tmp.rename(columns = {col: 'sem_'+col+'_'+name}, inplace=True)
        
        upload_csv_to_cloud_storage(df_tmp, gcp_directory+'sample_data_individual_feat/semcoop_'+name+'_'+today_ds+'.csv')
        del df_tmp
        endtime = time.time()
        print('time used for sem signals for coop for date '+today_ds+' and name is '+config_sem[idx]['name']+': ' + str(endtime - starttime))
        
    ###############################################################
    #*********************** catalog signals***********************
    
    print('querying catalog signals...')
    for config_tmp in config_catalog:
        print('processing catalog features for date '+today_ds+' and name is '+config_tmp['name'])
        starttime = time.time()
        name = config_tmp['name']
        condition = config_tmp['condition']
    
        backdays = re.findall(r'\(.*?\)', condition) #
    
        days_backtrack_end = backdays[0][1:-1].split(',')[1].split('+')
        days_backtrack_end = int(days_backtrack_end[0]) + int(days_backtrack_end[1])
    
        days_backtrack_start = int(backdays[1][1:-1].split(',')[1])
    
        # uber_curr_item_price_w1  double,
        # uber_num_appr_reviews_w1 double,
        # uber_avg_overall_rating_w1 double,
        # uber_price_change_ratio_w1 double,
        # uber_price_change_value_w1 double,
        # uber_avg_margin_w1         double,
        
        # "uber_division_w1"
        # "uber_super_dept_w1"
        # "uber_dept_w1"
        # "uber_cat_w1"
        # "uber_subcat_w1"
    
        df_tmp = get_catalogSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days)
        

        df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'division','super_dept','dept','cat','subcat','curr_item_price', 'num_appr_reviews', 'avg_overall_rating', 'price_change_ratio',\
                            'price_change_value', 'avg_margin']].copy()

        columns = df_tmp.columns
        for col in columns[2:]:
            df_tmp.rename(columns={col: 'uber_'+col+'_'+name}, inplace=True)
        
        upload_csv_to_cloud_storage(df_tmp, gcp_directory+'sample_data_individual_feat/catalog_'+name+'_'+today_ds+'.csv')
        del df_tmp
        endtime = time.time()
        print('time used for catalog signals for date '+today_ds+' and name is '+config_tmp['name']+': ' + str(endtime - starttime))

    ###############################################################
    #*********************** site signals**************************
    print('querying site signals...')
    for config_tmp in config_site:
        name = config_tmp['name']
        condition = config_tmp['condition']
    
        backdays = re.findall(r'\(.*?\)', condition) #
    
        days_backtrack_end = backdays[0][1:-1].split(',')[1].split('+')
        days_backtrack_end = int(days_backtrack_end[0]) + int(days_backtrack_end[1])
    
        days_backtrack_start = int(backdays[1][1:-1].split(',')[1])
    
#         # site_page_views_w2         double,
#         # site_add_to_carts_w2       double,
#         # site_orders_w2             double,
#         # site_revenue_w2            double,
#         # site_atcpv_w2              double,
#         # site_convrt_w2             double,
#         # site_rpv_w2                double,
#         # site_mpv_w2                double,
    
#         df_tmp = get_siteSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days)
        
#         df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'total_page_views', 'total_add_to_carts', 'total_orders', 'total_revenue',\
#                             'atcpv', 'convrt', 'rpv', 'mpv']].copy()
#         columns = df_tmp.columns
#         for col in columns[2:]:
#             df_tmp.rename(columns={col: 'site_'+col+'_'+name}, inplace=True)
        
#         df_tmp.to_csv('../rpc_model/sample_data_individual_feat/site_'+name+'_'+today_ds+'.csv')
#         del df_tmp
#         endtime = time.time()
#         print('time used for site offer level signals for date '+today_ds+' and name is '+config_tmp['name']+': ' + str(endtime - starttime))
        
        print('processing site item level features for date '+today_ds+' and name is '+config_tmp['name'])
        starttime = time.time()
        df_tmp = get_siteSignal_item(today_ds, days_backtrack_end, days_backtrack_start, bad_days)
        df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'margin', 'total_page_views', 'total_add_to_carts', 'total_orders', 'total_revenue',\
                            'atcpv', 'convrt', 'rpv', 'mpv']].copy()
        columns = df_tmp.columns
        for col in columns[2:]:
            df_tmp.rename(columns={col: 'site_'+col+'_'+name}, inplace=True)
        
        upload_csv_to_cloud_storage(df_tmp, gcp_directory+'sample_data_individual_feat/site_item_'+name+'_'+today_ds+'.csv')
        del df_tmp
        endtime = time.time()
        print('time used for site item level signals for date '+today_ds+' and name is '+config_tmp['name']+': ' + str(endtime - starttime))
    ###############################################################
    #*********************** bounce signals**************************
    print('querying bounce signals...')
    for config_tmp in config_bounce:
        name = config_tmp['name']
        condition = config_tmp['condition']
    
        backdays = re.findall(r'\(.*?\)', condition) #
    
        days_backtrack_end = backdays[0][1:-1].split(',')[1].split('+')
        days_backtrack_end = int(days_backtrack_end[0]) + int(days_backtrack_end[1])
    
        days_backtrack_start = int(backdays[1][1:-1].split(',')[1])
    
#         # bc_bounce_rate_w1
#         # bc_bounces_w1
#         # bc_sem_bounce_rate_w1
#         # bc_sem_bounces_w1
#         # bc_sem_visits_w1
#         # bc_visits_w1

#         df_tmp = get_bounceSignal(today_ds, days_backtrack_end, days_backtrack_start, bad_days)

#         df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'bounce_rate', 'bounces', 'sem_bounce_rate', 'sem_bounces',\
#                             'visits', 'sem_visits']].copy()
#         columns = df_tmp.columns
#         for col in columns[2:]:
#             df_tmp.rename(columns={col: 'bc_'+col+'_'+name}, inplace=True)
        
#         df_tmp.to_csv('../rpc_model/sample_data_individual_feat/bounce_'+name+'_'+today_ds+'.csv')
#         del df_tmp
#         endtime = time.time()
#         print('time used for bounce offer signals for date '+today_ds+' and name is '+config_tmp['name']+': ' + str(endtime - starttime))
        print('processing bounce item features for date '+today_ds+' and name is '+config_tmp['name'])
        starttime = time.time()
        df_tmp = get_bounceSignal_item(today_ds, days_backtrack_end, days_backtrack_start, bad_days)

        df_tmp = df_tmp[['catalog_item_id', 'seller_id', 'bounce_rate', 'bounces', 'sem_bounce_rate', 'sem_bounces',\
                            'visits', 'sem_visits']].copy()
        columns = df_tmp.columns
        for col in columns[2:]:
            df_tmp.rename(columns={col: 'bc_'+col+'_'+name}, inplace=True)
        
        upload_csv_to_cloud_storage(df_tmp, gcp_directory+'sample_data_individual_feat/bounce_item_'+name+'_'+today_ds+'.csv')
        del df_tmp
        endtime = time.time()
        print('time used for bounce item signals for date '+today_ds+' and name is '+config_tmp['name']+': ' + str(endtime - starttime))

dmas table renew done
Querying data for 2022-10-16
querying sem signals...
querying site signals...
processing site features for date 2022-10-16 and name is w2
l2 saved for site signals


In [98]:
# upload_local_to_cloud_storage('/home/y0c07y1/rpc_feature_extraction/query/test.csv', 'gs://msc_fair_airflow/kk/test1.csv')

# client = storage.Client()
# bucket = client.get_bucket('msc_fair_airflow')
# # list all objects in the directory
# blobs = bucket.list_blobs(prefix='logs')
# for blob in blobs:
#     blob.delete()

In [17]:
# today_ds = '2022-03-21'

# # insert data into table
# pd_df = pd.read_csv('data_lia_37/df_feat_all_'+today_ds+'.csv', index_col=0)
# pd_df['adid'] = pd_df['adid'].astype('str')
        
# sparkDF=spark.createDataFrame(pd_df) 
# sparkDF.registerTempTable("feat") 

# query = """INSERT INTO TABLE casesci_sem.lia_rpc_feature_test_notxt_cfl
#                 PARTITION(date_string = '__today__')
#                 SELECT * FROM feat"""

# query = query.replace('__today__', today_ds)
# spark.sql(query)     

In [18]:
# for config_tmp in config_site:
            
#         name = config_tmp['name']
#         condition = config_tmp['condition']
    
#         backdays = re.findall(r'\(.*?\)', condition) #
    
#         days_backtrack_end = backdays[0][1:-1].split(',')[1].split('+')
#         days_backtrack_end = int(days_backtrack_end[0]) + int(days_backtrack_end[1])
    
#         days_backtrack_start = int(backdays[1][1:-1].split(',')[1])